In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns
from random import seed

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import  MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import Imputer

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, LSTM, BatchNormalization
from keras import layers
from keras import optimizers
from keras import callbacks
from keras import models

Using TensorFlow backend.


In [2]:
#Generate Dataset
#Airfoil Self-Noise
df_airfoil=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Airfoil_self_noise.csv')

#Automobile
df_automobile=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Automobile_data.txt')

#Autompg
df_autompg=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Auto-mpg.csv')

#ChallengerUSA
df_challenger=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Challenger USA Space Shuttle O-Ring.csv')

#Combine Cycle Power
df_combine=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Combine Cycle Power Plant.csv')

#Computer Hardware
df_computer=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Computer Hardware.csv')

#Concrete Compressive Strength
df_concrete_compressive_strength=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Compressive Strength.csv')

#Concrete Slump Test
df_concrete1=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Slump Test.csv')
df_concrete2=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Slump Test.csv')
df_concrete3=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Slump Test.csv')

#Energy Efficiency
df_energy1=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')
df_energy2=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')

#Fertility Diagnosis
df_fertility=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\fertility_Diagnosis.csv')

#Forest Fires
df_forest=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\forestfires.csv')

#Housing
df_housing=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Housing.csv')

#Istanbul Stock Exchange
df_istanbul=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Istanbul Stock Exchange.csv')

#WDBC
df_wdbc=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\wdbc.csv')

In [3]:
#remove Nan columns of df_energy1
df_energy1=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')
df_energy1.drop(df_energy1.columns[11],inplace=True, axis = 1)
df_energy1.drop(df_energy1.columns[10],inplace=True, axis = 1)

#remove Nan columns of df_energy2
df_energy2=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')
df_energy2.drop(df_energy2.columns[11],inplace=True, axis = 1)
df_energy2.drop(df_energy2.columns[10],inplace=True, axis = 1)

#remove question marks
#df_automobile
imp= Imputer(missing_values="NaN", strategy="mean")
df_automobile=df_automobile.replace("?", np.NaN)
df_automobile['num-of-doors'] = df_automobile['num-of-doors'].fillna("four")
for i in df_automobile.columns:
    if df_automobile[i].isnull().sum()>0:
        df_automobile[i]=imp.fit_transform(df_automobile[[i]])
#df_automobile = df_automobile[(df_automobile.astype(str) != '?').all(axis=1)]
#df_automobile.reset_index(drop=True, inplace=True)
df_automobile = df_automobile.drop(['engine-location'], axis=1)
#df_autompg
df_autompg = df_autompg[(df_autompg.astype(str) != '?').all(axis=1)]
df_autompg.reset_index(drop=True, inplace=True)

#convert string to number
le=LabelEncoder()
#df_automobile
string_automobile=df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']]
df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']]=string_automobile.apply(le.fit_transform)
encoder=df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']].astype(np.int64)
df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']]=encoder
a=df_automobile[['normalized-losses','horsepower','price']].astype(np.int64)
b=df_automobile[['bore','stroke','peak-rpm']].astype(float)
df_automobile[['normalized-losses','horsepower','price']]=a
df_automobile[['bore','stroke','peak-rpm']]=b
#df_computer
string_computer=df_computer[['Vendor','Model']]
df_computer[['Vendor','Model']]=string_computer.apply(le.fit_transform)
df_computer[['Vendor','Model']]=df_computer[['Vendor','Model']].astype(np.int64)

#dropna spesific column & special preprocessing
#df_forest
df_forest=df_forest[(df_forest['area'].astype(int) != 0)] 
df_forest.reset_index(drop=True, inplace=True)
num_features=10
dimension=(num_features+1)*9+10
population_size=(2*dimension+1)*2
df_forest=df_forest[:population_size]

C:\Users\user\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [4]:
def preprocessing_data1(df,num_features):
    dimension=(num_features+1)*9+10
    population_size=(2*dimension+1)*2
    df=df.dropna()
    df=df[:population_size]
    df
    return df

In [5]:
def feature_selection(df):
    cor=df.corr()
    plt.figure(figsize=(100,50))
    sns.set(font_scale=3)
    sns.heatmap(cor, annot=True, cmap=plt.cm.Blues)
    plt.show()
    return cor

In [6]:
def train_test_split1(x, y):
    x_train, x_test, y_train, y_test= train_test_split(x, y, test_size=0.15, random_state=42)
    return x_train, x_test, y_train, y_test

In [7]:
def preprocessing_data2(df):
    train_size = 0.85*len(df)
    train_size = int(train_size)
    df_train = df[:train_size]
    df_test = df[train_size:]
    print(len(df_test))
    print(len(df_train))
    validation_size=len(df_test)/len(df_train)
    print(validation_size)
    return df_train, df_test, validation_size 

In [8]:
#df_airfoil=preprocessing_data1(df_airfoil,5)
#df_automobile=preprocessing_data1(df_automobile,22)
#df_autompg=preprocessing_data1(df_autompg,7)
#df_challenger=preprocessing_data1(df_challenger,3)
#df_combine=preprocessing_data1(df_combine,4)
#df_computer=preprocessing_data1(df_computer,9)
#df_concrete_compressive_strength=preprocessing_data1(df_concrete_compressive_strength,8)
#df_concrete1=preprocessing_data1(df_concrete1,7)
#df_concrete2=preprocessing_data1(df_concrete2,7)
#df_concrete3=preprocessing_data1(df_concrete3,7)
#df_energy1=preprocessing_data1(df_energy1,8)
#df_energy2=preprocessing_data1(df_energy2,8)

#df_fertility=preprocessing_data1(df_fertility,9)
for i in df_fertility:
    df_fertility.Diagnosis[df_fertility.Diagnosis == 'N'] = 1
    df_fertility.Diagnosis[df_fertility.Diagnosis == 'O'] = 2
df_fertility['Diagnosis']=df_fertility['Diagnosis'].astype(np.int64)

#df_forest=preprocessing_data1(df_forest,10)
#df_housing=preprocessing_data1(df_housing,13)

#df_istanbul=preprocessing_data1(df_istanbul,7)
new_header = df_istanbul.iloc[0]
df_istanbul = df_istanbul[1:]
df_istanbul.columns = new_header
df_istanbul[['ISE1','ISE2','SP','DAX','FTSE','NIKKEI','BOVESPA','EU','EM']]=df_istanbul[['ISE1','ISE2','SP','DAX','FTSE','NIKKEI','BOVESPA','EU','EM']].astype(np.float64)


#df_wdbc=preprocessing_data1(df_wdbc,10)
df_wdbc.drop(df_wdbc.columns[0], axis=1, inplace = True)
df_wdbc.columns=['id','diagnosis','radius_mean','texture_mean','perimeter_mean','area_mean','smoothness_mean','compactness_mean','concavity_mean','concave points_mean','symmetry_mean','fractal_dimension_mean','radius_se','texture_se','perimeter_se','area_se','smoothness_se','compactness_se','concavity_se','concave points_se','symmetry_se','fractal_dimension_se','radius_worst','texture_worst','perimeter_worst','area_worst','smoothness_worst','compactness_worst','concavity_worst','concave points_worst','symmetry_worst','fractal_dimension_worst']
for i in df_wdbc:
    df_wdbc.diagnosis[df_wdbc.diagnosis == 'M'] = 1
    df_wdbc.diagnosis[df_wdbc.diagnosis == 'B'] = 2
df_wdbc['diagnosis']=df_wdbc['diagnosis'].astype(np.object)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a 

In [9]:
print(df_airfoil.head())
print(df_autompg.head())
print(df_automobile.head())
print(df_challenger.head())

   Frequency  Angle of Attack  Chord length  Free-stream velocity  \
0        800              0.0        0.3048                  71.3   
1       1000              0.0        0.3048                  71.3   
2       1250              0.0        0.3048                  71.3   
3       1600              0.0        0.3048                  71.3   
4       2000              0.0        0.3048                  71.3   

   Suction side displacement thickness  Scaled sound pressure level  
0                             0.002663                      126.201  
1                             0.002663                      125.201  
2                             0.002663                      125.951  
3                             0.002663                      127.591  
4                             0.002663                      127.461  
    Mpg  Cylinders  Displacement Horsepower  Weight  Acceleration  Model Year  \
0  18.0          8         307.0        130    3504          12.0          70   
1  

In [10]:
#Train and Test 17 datasets
#Airfoil
x_airfoil = df_airfoil.drop(['Scaled sound pressure level'], axis = 1)
y_airfoil = df_airfoil['Scaled sound pressure level']
#Wdbc
x_wdbc = df_wdbc[['radius_mean','perimeter_mean','area_mean','concavity_mean','concave points_mean','radius_worst','perimeter_worst','area_worst','concave points_worst']]
y_wdbc = df_wdbc['diagnosis']
labelencoder_Y = LabelEncoder()
y_wdbc = labelencoder_Y.fit_transform(y_wdbc)
#Automobile
x_automobile = df_automobile.drop(['price','make','body-style'], axis = 1)
y_automobile = df_automobile['price']
y_automobile = np.reshape(y_automobile.values, (-1,1))
#Autompg
x_autompg = df_autompg.drop(['Mpg','Car Name'], axis = 1)
y_autompg = df_autompg['Mpg']
#Challenger
x_challenger = df_challenger.drop(['Number of O-rings','Temporal Order of Flight'], axis = 1)
y_challenger = df_challenger['Number of O-rings']
#Combine
x_combine = df_combine.drop(['PE'], axis = 1)
y_combine = df_combine['PE']
y_combine = np.reshape(y_combine.values, (-1,1))
#Computer
x_computer = df_computer.drop(['ERP'], axis = 1)
y_computer = df_computer['ERP']
y_computer = np.reshape(y_computer.values, (-1,1))
#Concrete_compressive_strength
x_concrete_compressive_strength = df_concrete_compressive_strength.drop(['CCS'], axis = 1)
y_concrete_compressive_strength = df_concrete_compressive_strength['CCS']
y_concrete_compressive_strength = np.reshape(y_concrete_compressive_strength.values, (-1,1))
#Concrete1
x_concrete1 = df_concrete1.drop(['No','Compressive Strength (28-day)(Mpa)'], axis = 1)
y_concrete1 = df_concrete1['Compressive Strength (28-day)(Mpa)']
#Concrete2
x_concrete2 = df_concrete2.drop(['No','SLUMP(cm)'], axis = 1)
y_concrete2 = df_concrete2['SLUMP(cm)']
y_concrete2 = np.reshape(y_concrete2.values, (-1,1))
#Concrete3
x_concrete3 = df_concrete3.drop(['No','FLOW(cm)'], axis = 1)
y_concrete3 = df_concrete3['FLOW(cm)']
y_concrete3 = np.reshape(y_concrete3.values, (-1,1))
#Energy1
x_energy1 = df_energy1.drop(['Y1'], axis = 1)
y_energy1 = df_energy1['Y1']
#Energy2
x_energy2 = df_energy2.drop(['Y2'], axis = 1)
y_energy2 = df_energy2['Y2']
#Fertility
x_fertility = df_fertility.drop(['Diagnosis'], axis = 1)
y_fertility = df_fertility['Diagnosis']
#Forest
x_forest = df_forest.drop(['month','day','area'], axis = 1)
y_forest = df_forest['area']
y_forest = np.reshape(y_forest.values, (-1,1))
#Housing
x_housing = df_housing.drop(['MEDV'], axis = 1)
y_housing = df_housing['MEDV']
#Istanbul
x_istanbul = df_istanbul.drop(['date','ISE1','ISE2'], axis = 1)
y_istanbul = df_istanbul[['ISE1','ISE2']]
y_istanbul['mean'] = y_istanbul.mean(axis=1)
y_istanbul = y_istanbul['mean']

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [11]:
x_train_airfoil, x_test_airfoil, y_train_airfoil, y_test_airfoil = train_test_split1(x_airfoil, y_airfoil)
x_train_automobile, x_test_automobile, y_train_automobile, y_test_automobile = train_test_split1(x_automobile, y_automobile)
x_train_autompg, x_test_autompg, y_train_autompg, y_test_autompg = train_test_split1(x_autompg, y_autompg)
x_train_challenger, x_test_challenger, y_train_challenger, y_test_challenger = train_test_split1(x_challenger, y_challenger)
x_train_combine, x_test_combine, y_train_combine, y_test_combine = train_test_split1(x_combine, y_combine)
x_train_computer, x_test_computer, y_train_computer, y_test_computer = train_test_split1(x_computer, y_computer)
x_train_concrete_compressive_strength, x_test_concrete_compressive_strength, y_train_concrete_compressive_strength, y_test_concrete_compressive_strength = train_test_split1(x_concrete_compressive_strength, y_concrete_compressive_strength)
x_train_concrete1, x_test_concrete1, y_train_concrete1, y_test_concrete1 = train_test_split1(x_concrete1, y_concrete1)
x_train_concrete2, x_test_concrete2, y_train_concrete2, y_test_concrete2 = train_test_split1(x_concrete2, y_concrete2)
x_train_concrete3, x_test_concrete3, y_train_concrete3, y_test_concrete3 = train_test_split1(x_concrete3, y_concrete3)
x_train_energy1, x_test_energy1, y_train_energy1, y_test_energy1 = train_test_split1(x_energy1, y_energy1)
x_train_energy2, x_test_energy2, y_train_energy2, y_test_energy2 = train_test_split1(x_energy2, y_energy2)
x_train_fertility, x_test_fertility, y_train_fertility, y_test_fertility = train_test_split1(x_fertility, y_fertility)
x_train_forest, x_test_forest, y_train_forest, y_test_forest = train_test_split1(x_forest, y_forest)
x_train_housing, x_test_housing, y_train_housing, y_test_housing = train_test_split1(x_housing, y_housing)
x_train_istanbul, x_test_istanbul, y_train_istanbul, y_test_istanbul = train_test_split1(x_istanbul, y_istanbul)
x_train_wdbc, x_test_wdbc, y_train_wdbc, y_test_wdbc = train_test_split1(x_wdbc, y_wdbc)

In [12]:
def train_val_split(x_train, y_train):
    X_train, X_val, Y_train, Y_val= train_test_split(x_train, y_train, test_size=0.176, random_state=42)
    return X_train, X_val, Y_train, Y_val

In [13]:
def standard_scaler(X_train, X_val, x_test):
    sc=StandardScaler()
    X_train=X_train.values
    X_val=X_val.values
    x_test=x_test.values
    X_scale_train=sc.fit(X_train)
    #X_scale_val=sc.fit(X_val)
    #x_scale_test=sc.fit(x_test)
    X_train=X_scale_train.transform(X_train)
    X_val=X_scale_train.transform(X_val)
    x_test=X_scale_train.transform(x_test)
    return X_train, X_val, x_test, X_scale_train

In [14]:
def min_max_scaler_automobile(X_train, X_val, x_test, Y_train, Y_val, y_test):
    minmax_x=MinMaxScaler()
    minmax_y=MinMaxScaler()
    X_train=X_train.values
    X_val=X_val.values
    x_test=x_test.values
    X_scale_train=minmax_x.fit(X_train)
    #X_scale_val=min_max.fit(X_val)
    #x_scale_test=min_max.fit(x_test)
    Y_scale_train=minmax_y.fit(Y_train)
    #Y_scale_val=min_max.fit(Y_val)
    #y_scale_test=min_max.fit(y_test)
    X_train=X_scale_train.transform(X_train)
    X_val=X_scale_train.transform(X_val)
    x_test=X_scale_train.transform(x_test)
    Y_train=Y_scale_train.transform(Y_train)
    Y_val=Y_scale_train.transform(Y_val)
    y_test=Y_scale_train.transform(y_test)
    return X_train, X_val, x_test, Y_train, Y_val, y_test, X_scale_train, Y_scale_train

In [15]:
def min_max_scaler(X_train, X_val, x_test, Y_train, Y_val, y_test):
    min_max=MinMaxScaler()
    X_train=X_train.values
    X_val=X_val.values
    x_test=x_test.values
    X_scale_train=min_max.fit(X_train)
    X_scale_val=min_max.fit(X_val)
    x_scale_test=min_max.fit(x_test)
    Y_scale_train=min_max.fit(Y_train)
    Y_scale_val=min_max.fit(Y_val)
    y_scale_test=min_max.fit(y_test)
    X_train=X_scale_train.transform(X_train)
    X_val=X_scale_val.transform(X_val)
    x_test=x_scale_test.transform(x_test)
    Y_train=Y_scale_train.transform(Y_train)
    Y_val=Y_scale_train.transform(Y_val)
    y_test=y_scale_test.transform(y_test)
    return X_train, X_val, x_test, Y_train, Y_val, y_test, min_max

In [16]:
def normalization(X_train, X_val, x_test, Y_train, Y_val, y_test):
    normalizer=Normalizer()
    X_train=normalizer.fit_transform(X_train)
    X_val=normalizer.fit_transform(X_val)
    x_test=normalizer.fit_transform(x_test)
    Y_train=normalizer.fit_transform(Y_train)
    Y_val=normalizer.fit_transform(Y_val)
    y_test=normalizer.fit_transform(y_test)
    return X_train, X_val, x_test, Y_train, Y_val, y_test

In [17]:
#Optimizer list
adam0 = optimizers.Adam(lr=0.1)
adam1 = optimizers.Adam(lr=0.001)
adam2 = optimizers.Adam(lr=0.01)
adam3 = optimizers.Adam(lr=0.0001)
adam4 = optimizers.Adam(lr=0.00001)
sgd = optimizers.SGD(lr=0.00001, decay=1e-4, momentum=0.9, nesterov=True)
RMSprop= optimizers.RMSprop(lr=0.01,rho=0.9)
adagrad = optimizers.Adagrad(lr=0.01)
adadelta = optimizers.Adadelta(lr=1.0)

In [18]:
def NN_model_structure_regression_1(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 5, input_dim = (X_train.shape[1])))
    model.add(Activation('tanh'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 5))
    model.add(Activation('linear'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate

In [19]:
def NN_model_structure_regression_2(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 5, input_dim = (X_train.shape[1])))
    model.add(Activation('tanh'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 10))
    model.add(Activation('linear'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate

In [20]:
def NN_model_structure_regression_3(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 5, input_dim = (X_train.shape[1])))
    model.add(Activation('tanh'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 15))
    model.add(Activation('linear'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate

In [21]:
def NN_model_structure_regression_4(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 10, input_dim = (X_train.shape[1])))
    model.add(Activation('tanh'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 5))
    model.add(Activation('linear'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate

In [22]:
def NN_model_structure_regression_5(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 10, input_dim = (X_train.shape[1])))
    model.add(Activation('tanh'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 10))
    model.add(Activation('linear'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate

In [23]:
def NN_model_structure_regression_6(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 10, input_dim = (X_train.shape[1])))
    model.add(Activation('tanh'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 15))
    model.add(Activation('linear'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate

In [24]:
X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil = train_val_split(x_train_airfoil, y_train_airfoil)
X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile = train_val_split(x_train_automobile, y_train_automobile)
X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg = train_val_split(x_train_autompg, y_train_autompg)
X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger = train_val_split(x_train_challenger, y_train_challenger)
X_train_combine, X_val_combine, Y_train_combine, Y_val_combine = train_val_split(x_train_combine, y_train_combine)
X_train_computer, X_val_computer, Y_train_computer, Y_val_computer = train_val_split(x_train_computer, y_train_computer)
X_train_concrete_compressive_strength, X_val_concrete_compressive_strength, Y_train_concrete_compressive_strength, Y_val_concrete_compressive_strength = train_val_split(x_train_concrete_compressive_strength, y_train_concrete_compressive_strength)
X_train_concrete1, X_val_concrete1, Y_train_concrete1, Y_val_concrete1 = train_val_split(x_train_concrete1, y_train_concrete1)
X_train_concrete2, X_val_concrete2, Y_train_concrete2, Y_val_concrete2 = train_val_split(x_train_concrete2, y_train_concrete2)
X_train_concrete3, X_val_concrete3, Y_train_concrete3, Y_val_concrete3 = train_val_split(x_train_concrete3, y_train_concrete3)
X_train_energy1, X_val_energy1, Y_train_energy1, Y_val_energy1 = train_val_split(x_train_energy1, y_train_energy1)
X_train_energy2, X_val_energy2, Y_train_energy2, Y_val_energy2 = train_val_split(x_train_energy2, y_train_energy2)
X_train_fertility, X_val_fertility, Y_train_fertility, Y_val_fertility = train_val_split(x_train_fertility, y_train_fertility)
X_train_forest, X_val_forest, Y_train_forest, Y_val_forest = train_val_split(x_train_forest, y_train_forest)
X_train_housing, X_val_housing, Y_train_housing, Y_val_housing = train_val_split(x_train_housing, y_train_housing)
X_train_istanbul, X_val_istanbul, Y_train_istanbul, Y_val_istanbul = train_val_split(x_train_istanbul, y_train_istanbul)
X_train_wdbc, X_val_wdbc, Y_train_wdbc, Y_val_wdbc = train_val_split(x_train_wdbc, y_train_wdbc)

In [25]:
X_train_airfoil, X_val_airfoil, x_test_airfoil, X_scale_train_airfoil=standard_scaler(X_train_airfoil, X_val_airfoil, x_test_airfoil)
#X_train_automobile, X_val_automobile, x_test_automobile=standard_scaler(X_train_automobile, X_val_automobile, x_test_automobile)
#X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile = normalization(X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile)
X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile, X_scale_train_automobile, Y_scale_train_automobile=min_max_scaler_automobile(X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile)
X_train_autompg, X_val_autompg, x_test_autompg, X_scale_train_autompg=standard_scaler(X_train_autompg, X_val_autompg, x_test_autompg)
X_train_challenger, X_val_challenger, x_test_challenger, X_scale_train_challenger=standard_scaler(X_train_challenger, X_val_challenger, x_test_challenger)
X_train_combine, X_val_combine, x_test_combine, Y_train_combine, Y_val_combine, y_test_combine, min_max_combine=min_max_scaler(X_train_combine, X_val_combine, x_test_combine, Y_train_combine, Y_val_combine, y_test_combine)
X_train_computer, X_val_computer, x_test_computer, Y_train_computer, Y_val_computer, y_test_computer, min_max_computer=min_max_scaler(X_train_computer, X_val_computer, x_test_computer, Y_train_computer, Y_val_computer, y_test_computer)
X_train_concrete_compressive_strength, X_val_concrete_compressive_strength, x_test_concrete_compressive_strength, Y_train_concrete_compressive_strength, Y_val_concrete_compressive_strength, y_test_concrete_compressive_strength, min_max_concrete_compressive_strength=min_max_scaler(X_train_concrete_compressive_strength, X_val_concrete_compressive_strength, x_test_concrete_compressive_strength, Y_train_concrete_compressive_strength, Y_val_concrete_compressive_strength, y_test_concrete_compressive_strength)
X_train_concrete1, X_val_concrete1, x_test_concrete1, X_scale_train_concrete1=standard_scaler(X_train_concrete1, X_val_concrete1, x_test_concrete1)
X_train_concrete2, X_val_concrete2, x_test_concrete2, Y_train_concrete2, Y_val_concrete2, y_test_concrete2, min_max_concrete2=min_max_scaler(X_train_concrete2, X_val_concrete2, x_test_concrete2, Y_train_concrete2, Y_val_concrete2, y_test_concrete2)
X_train_concrete3, X_val_concrete3, x_test_concrete3, Y_train_concrete3, Y_val_concrete3, y_test_concrete3, min_max_concrete3=min_max_scaler(X_train_concrete3, X_val_concrete3, x_test_concrete3, Y_train_concrete3, Y_val_concrete3, y_test_concrete3)
X_train_energy1, X_val_energy1, x_test_energy1, X_scale_train_energy1=standard_scaler(X_train_energy1, X_val_energy1, x_test_energy1)
X_train_energy2, X_val_energy2, x_test_energy2, X_scale_train_energy2=standard_scaler(X_train_energy2, X_val_energy2, x_test_energy2)
X_train_fertility, X_val_fertility, x_test_fertility, X_scale_train_fertility=standard_scaler(X_train_fertility, X_val_fertility, x_test_fertility)
X_train_forest, X_val_forest, x_test_forest, Y_train_forest, Y_val_forest, y_test_forest, min_max_forest=min_max_scaler(X_train_forest, X_val_forest, x_test_forest, Y_train_forest, Y_val_forest, y_test_forest)
X_train_housing, X_val_housing, x_test_housing, X_scale_train_housing=standard_scaler(X_train_housing, X_val_housing, x_test_housing)
X_train_istanbul, X_val_istanbul, x_test_istanbul, X_scale_train_istanbul=standard_scaler(X_train_istanbul, X_val_istanbul, x_test_istanbul)

In [26]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression_1(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_1 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_2 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 6         
Total params: 66
Trainable params: 66
Non-trainable params: 0
_________________________________________________________________

Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 413us/step - lo

1052/1052 [==============================] - 0s 103us/step - loss: 21.8189 - val_loss: 25.8302
Epoch 66/1000
1052/1052 [==============================] - 0s 90us/step - loss: 21.6687 - val_loss: 25.5780
Epoch 67/1000
1052/1052 [==============================] - 0s 90us/step - loss: 21.8684 - val_loss: 25.4291
Epoch 68/1000
1052/1052 [==============================] - 0s 91us/step - loss: 22.2347 - val_loss: 25.9070
Epoch 69/1000
1052/1052 [==============================] - 0s 88us/step - loss: 22.2256 - val_loss: 27.0164
Epoch 70/1000
1052/1052 [==============================] - 0s 89us/step - loss: 21.6548 - val_loss: 25.2628
Epoch 71/1000
1052/1052 [==============================] - 0s 86us/step - loss: 22.0248 - val_loss: 26.1881
Epoch 72/1000
1052/1052 [==============================] - 0s 93us/step - loss: 22.1866 - val_loss: 26.7755
Epoch 73/1000
1052/1052 [==============================] - 0s 87us/step - loss: 22.1624 - val_loss: 25.4510
Epoch 74/1000
1052/1052 [================

1052/1052 [==============================] - 0s 82us/step - loss: 21.9409 - val_loss: 25.3485
Epoch 141/1000
1052/1052 [==============================] - 0s 92us/step - loss: 21.8431 - val_loss: 25.2772
Epoch 142/1000
1052/1052 [==============================] - 0s 90us/step - loss: 22.1705 - val_loss: 25.7889
Epoch 143/1000
1052/1052 [==============================] - 0s 88us/step - loss: 21.9447 - val_loss: 26.4543
Epoch 144/1000
1052/1052 [==============================] - 0s 85us/step - loss: 21.9474 - val_loss: 28.5328
Epoch 145/1000
1052/1052 [==============================] - 0s 83us/step - loss: 21.5222 - val_loss: 25.6529
Epoch 146/1000
1052/1052 [==============================] - 0s 88us/step - loss: 21.3511 - val_loss: 25.1365
Epoch 147/1000
1052/1052 [==============================] - 0s 90us/step - loss: 21.5129 - val_loss: 25.2407
Epoch 148/1000
1052/1052 [==============================] - 0s 86us/step - loss: 21.1355 - val_loss: 25.4170
Epoch 149/1000
1052/1052 [========

Epoch 215/1000
1052/1052 [==============================] - 0s 90us/step - loss: 16.4091 - val_loss: 19.3091
Epoch 216/1000
1052/1052 [==============================] - 0s 89us/step - loss: 16.2331 - val_loss: 18.7772
Epoch 217/1000
1052/1052 [==============================] - 0s 94us/step - loss: 16.1009 - val_loss: 18.9124
Epoch 218/1000
1052/1052 [==============================] - 0s 94us/step - loss: 16.1182 - val_loss: 17.9723
Epoch 219/1000
1052/1052 [==============================] - 0s 100us/step - loss: 17.2945 - val_loss: 18.0153
Epoch 220/1000
1052/1052 [==============================] - 0s 95us/step - loss: 16.1028 - val_loss: 18.6424
Epoch 221/1000
1052/1052 [==============================] - 0s 86us/step - loss: 16.1527 - val_loss: 20.3667
Epoch 222/1000
1052/1052 [==============================] - 0s 87us/step - loss: 16.1241 - val_loss: 18.2488
Epoch 223/1000
1052/1052 [==============================] - 0s 90us/step - loss: 15.9012 - val_loss: 17.8857
Epoch 224/1000
105

1052/1052 [==============================] - 0s 96us/step - loss: 11.6019 - val_loss: 14.0129
Epoch 291/1000
1052/1052 [==============================] - 0s 97us/step - loss: 12.0357 - val_loss: 14.6469
Epoch 292/1000
1052/1052 [==============================] - 0s 96us/step - loss: 11.4194 - val_loss: 14.3274
Epoch 293/1000
1052/1052 [==============================] - 0s 86us/step - loss: 11.1342 - val_loss: 13.8545
Epoch 294/1000
1052/1052 [==============================] - 0s 84us/step - loss: 11.0349 - val_loss: 14.4555
Epoch 295/1000
1052/1052 [==============================] - 0s 89us/step - loss: 11.2962 - val_loss: 16.2567
Epoch 296/1000
1052/1052 [==============================] - 0s 94us/step - loss: 11.4917 - val_loss: 14.9377
Epoch 297/1000
1052/1052 [==============================] - 0s 107us/step - loss: 11.2923 - val_loss: 13.9733
Epoch 298/1000
1052/1052 [==============================] - 0s 88us/step - loss: 10.7739 - val_loss: 15.6253
Epoch 299/1000
1052/1052 [=======

...

1052/1052 [==============================] - 0s 101us/step - loss: 8.2287 - val_loss: 11.9286
Epoch 746/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.5341 - val_loss: 9.9358
Epoch 747/1000
1052/1052 [==============================] - 0s 100us/step - loss: 8.2565 - val_loss: 9.2276
Epoch 748/1000
1052/1052 [==============================] - 0s 94us/step - loss: 8.5396 - val_loss: 9.4354
Epoch 749/1000
1052/1052 [==============================] - 0s 96us/step - loss: 8.7299 - val_loss: 11.5559
Epoch 750/1000
1052/1052 [==============================] - 0s 97us/step - loss: 8.2918 - val_loss: 10.7911
Epoch 751/1000
1052/1052 [==============================] - 0s 83us/step - loss: 8.5837 - val_loss: 10.8651
Epoch 752/1000
1052/1052 [==============================] - 0s 100us/step - loss: 8.3889 - val_loss: 10.1249
Epoch 753/1000
1052/1052 [==============================] - 0s 114us/step - loss: 8.0872 - val_loss: 9.8907
Epoch 754/1000
1052/1052 [=================

1052/1052 [==============================] - 0s 83us/step - loss: 8.2609 - val_loss: 10.0803
Epoch 822/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.6713 - val_loss: 9.7574
Epoch 823/1000
1052/1052 [==============================] - 0s 82us/step - loss: 8.3535 - val_loss: 9.3647
Epoch 824/1000
1052/1052 [==============================] - 0s 82us/step - loss: 8.6118 - val_loss: 9.9119
Epoch 825/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.1845 - val_loss: 9.9741
Epoch 826/1000
1052/1052 [==============================] - 0s 103us/step - loss: 8.0096 - val_loss: 9.6660
Epoch 827/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.1112 - val_loss: 11.8681
Epoch 828/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.4288 - val_loss: 9.5294
Epoch 829/1000
1052/1052 [==============================] - 0s 96us/step - loss: 8.0483 - val_loss: 9.9668
Epoch 830/1000
1052/1052 [=======================

1052/1052 [==============================] - 0s 82us/step - loss: 8.1989 - val_loss: 9.9455
Epoch 898/1000
1052/1052 [==============================] - 0s 82us/step - loss: 7.9651 - val_loss: 9.6084
Epoch 899/1000
1052/1052 [==============================] - 0s 81us/step - loss: 8.0728 - val_loss: 9.8195
Epoch 900/1000
1052/1052 [==============================] - 0s 81us/step - loss: 8.1723 - val_loss: 9.1905
Epoch 901/1000
1052/1052 [==============================] - 0s 82us/step - loss: 8.5549 - val_loss: 12.1195
Epoch 902/1000
1052/1052 [==============================] - 0s 80us/step - loss: 8.2622 - val_loss: 9.2596
Epoch 903/1000
1052/1052 [==============================] - 0s 81us/step - loss: 8.7079 - val_loss: 9.2537
Epoch 904/1000
1052/1052 [==============================] - 0s 80us/step - loss: 8.0780 - val_loss: 9.9712
Epoch 905/1000
1052/1052 [==============================] - 0s 78us/step - loss: 8.1443 - val_loss: 10.5614
Epoch 906/1000
1052/1052 [========================

1052/1052 [==============================] - 0s 80us/step - loss: 7.9576 - val_loss: 9.3327
Epoch 974/1000
1052/1052 [==============================] - 0s 82us/step - loss: 8.1678 - val_loss: 10.4639
Epoch 975/1000
1052/1052 [==============================] - 0s 80us/step - loss: 8.1096 - val_loss: 10.2543
Epoch 976/1000
1052/1052 [==============================] - 0s 83us/step - loss: 8.1724 - val_loss: 9.7887
Epoch 977/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.0855 - val_loss: 8.9587
Epoch 978/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.1421 - val_loss: 9.5087
Epoch 979/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.2025 - val_loss: 9.4070
Epoch 980/1000
1052/1052 [==============================] - 0s 82us/step - loss: 8.2269 - val_loss: 9.1249
Epoch 981/1000
1052/1052 [==============================] - 0s 80us/step - loss: 8.1775 - val_loss: 10.6029
Epoch 982/1000
1052/1052 [=======================

[array([[ 1.810115  ,  4.0667405 ,  0.16914245,  3.162663  , -0.7241936 ],
        [-0.5076961 , -0.20897667,  0.19581285, -0.06318165, -1.4072537 ],
        [-0.40682298,  0.7359196 ,  0.3011046 , -0.0130247 , -1.5635636 ],
        [-0.0192558 , -0.13605331, -0.1302032 , -0.06391427,  0.25787455],
        [ 0.29347178,  0.33252844,  0.18295312,  2.2943096 ,  0.28922522]],
       dtype=float32),
 array([ 1.2996395 ,  4.9890785 , -1.3994079 ,  4.6891246 , -0.70023715],
       dtype=float32),
 array([[ 0.20652682, -1.0497646 ,  1.0188239 ,  0.85498446,  1.1505967 ],
        [-3.4394212 ,  2.8564396 , -3.1773133 , -2.0947242 , -2.672332  ],
        [ 1.842096  , -2.9491358 ,  1.4664241 ,  2.847502  ,  2.6628861 ],
        [-3.049095  ,  2.6243596 , -3.0126631 , -3.0480802 , -3.2229526 ],
        [-0.31583336, -0.19730633, -1.0590692 , -1.187005  , -0.178946  ]],
       dtype=float32),
 array([-2.5857222,  2.273334 , -2.3287945, -2.523248 , -2.6682775],
       dtype=float32),
 array([[-2.4

In [27]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_structure1_3rd.h5')

In [28]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression_2(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_3 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                60        
_________________________________________________________________
activation_4 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 11        
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 189us/step - l

Epoch 67/1000
1052/1052 [==============================] - 0s 83us/step - loss: 20.6521 - val_loss: 26.3383
Epoch 68/1000
1052/1052 [==============================] - 0s 80us/step - loss: 20.7552 - val_loss: 25.5481
Epoch 69/1000
1052/1052 [==============================] - 0s 84us/step - loss: 20.8277 - val_loss: 25.1402
Epoch 70/1000
1052/1052 [==============================] - 0s 86us/step - loss: 20.1330 - val_loss: 24.8556
Epoch 71/1000
1052/1052 [==============================] - 0s 85us/step - loss: 19.9762 - val_loss: 24.8127
Epoch 72/1000
1052/1052 [==============================] - 0s 83us/step - loss: 20.1916 - val_loss: 24.3869
Epoch 73/1000
1052/1052 [==============================] - 0s 86us/step - loss: 19.4102 - val_loss: 23.8619
Epoch 74/1000
1052/1052 [==============================] - 0s 85us/step - loss: 19.3077 - val_loss: 24.0906
Epoch 75/1000
1052/1052 [==============================] - 0s 84us/step - loss: 19.2462 - val_loss: 23.7728
Epoch 76/1000
1052/1052 [===

1052/1052 [==============================] - 0s 83us/step - loss: 13.1983 - val_loss: 17.2161
Epoch 143/1000
1052/1052 [==============================] - 0s 82us/step - loss: 13.4008 - val_loss: 16.6903
Epoch 144/1000
1052/1052 [==============================] - 0s 87us/step - loss: 13.0282 - val_loss: 17.6244
Epoch 145/1000
1052/1052 [==============================] - 0s 84us/step - loss: 13.6634 - val_loss: 15.1846
Epoch 146/1000
1052/1052 [==============================] - 0s 84us/step - loss: 12.8961 - val_loss: 16.6296
Epoch 147/1000
1052/1052 [==============================] - 0s 83us/step - loss: 12.8617 - val_loss: 15.8365
Epoch 148/1000
1052/1052 [==============================] - 0s 83us/step - loss: 12.2763 - val_loss: 15.7743
Epoch 149/1000
1052/1052 [==============================] - 0s 80us/step - loss: 12.9562 - val_loss: 17.5477
Epoch 150/1000
1052/1052 [==============================] - 0s 85us/step - loss: 12.5139 - val_loss: 14.9249
Epoch 151/1000
1052/1052 [========

1052/1052 [==============================] - 0s 100us/step - loss: 10.2515 - val_loss: 12.7849
Epoch 218/1000
1052/1052 [==============================] - 0s 90us/step - loss: 9.5771 - val_loss: 13.0380
Epoch 219/1000
1052/1052 [==============================] - 0s 82us/step - loss: 9.3011 - val_loss: 12.1941
Epoch 220/1000
1052/1052 [==============================] - 0s 82us/step - loss: 9.2599 - val_loss: 12.2450
Epoch 221/1000
1052/1052 [==============================] - 0s 95us/step - loss: 9.6466 - val_loss: 12.9941
Epoch 222/1000
1052/1052 [==============================] - 0s 91us/step - loss: 9.7286 - val_loss: 15.0525
Epoch 223/1000
1052/1052 [==============================] - 0s 82us/step - loss: 9.9976 - val_loss: 11.6805
Epoch 224/1000
1052/1052 [==============================] - 0s 83us/step - loss: 9.8784 - val_loss: 11.8130
Epoch 225/1000
1052/1052 [==============================] - 0s 80us/step - loss: 9.4565 - val_loss: 11.6692
Epoch 226/1000
1052/1052 [===============

Epoch 293/1000
1052/1052 [==============================] - 0s 94us/step - loss: 9.4253 - val_loss: 11.8808
Epoch 294/1000
1052/1052 [==============================] - 0s 105us/step - loss: 8.9571 - val_loss: 10.9171
Epoch 295/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.7015 - val_loss: 11.1073
Epoch 296/1000
1052/1052 [==============================] - 0s 82us/step - loss: 8.7883 - val_loss: 10.5478
Epoch 297/1000
1052/1052 [==============================] - 0s 80us/step - loss: 8.6160 - val_loss: 11.3605
Epoch 298/1000
1052/1052 [==============================] - 0s 82us/step - loss: 9.1475 - val_loss: 10.0956
Epoch 299/1000
1052/1052 [==============================] - 0s 81us/step - loss: 8.5105 - val_loss: 11.5995
Epoch 300/1000
1052/1052 [==============================] - 0s 80us/step - loss: 8.9146 - val_loss: 11.6933
Epoch 301/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.6635 - val_loss: 10.9313
Epoch 302/1000
1052/1052 [=

...

1052/1052 [==============================] - 0s 91us/step - loss: 8.6099 - val_loss: 10.1828
Epoch 750/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.1231 - val_loss: 9.2433
Epoch 751/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.0632 - val_loss: 10.3897
Epoch 752/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.1482 - val_loss: 10.0361
Epoch 753/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.3104 - val_loss: 9.2326
Epoch 754/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.2089 - val_loss: 9.9834
Epoch 755/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.0537 - val_loss: 9.3207
Epoch 756/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.2231 - val_loss: 11.8079
Epoch 757/1000
1052/1052 [==============================] - 0s 97us/step - loss: 8.6433 - val_loss: 9.8077
Epoch 758/1000
1052/1052 [======================

1052/1052 [==============================] - 0s 91us/step - loss: 9.1463 - val_loss: 10.6148
Epoch 826/1000
1052/1052 [==============================] - 0s 99us/step - loss: 8.6708 - val_loss: 8.9477
Epoch 827/1000
1052/1052 [==============================] - 0s 99us/step - loss: 8.3857 - val_loss: 9.8790
Epoch 828/1000
1052/1052 [==============================] - 0s 97us/step - loss: 8.0077 - val_loss: 10.3369
Epoch 829/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.5471 - val_loss: 9.9398
Epoch 830/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.2701 - val_loss: 9.1274
Epoch 831/1000
1052/1052 [==============================] - 0s 93us/step - loss: 8.4282 - val_loss: 10.0108
Epoch 832/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.1799 - val_loss: 9.1844
Epoch 833/1000
1052/1052 [==============================] - 0s 94us/step - loss: 8.7566 - val_loss: 9.3118
Epoch 834/1000
1052/1052 [=======================

1052/1052 [==============================] - 0s 86us/step - loss: 7.9967 - val_loss: 9.1428
Epoch 902/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.1330 - val_loss: 9.8152
Epoch 903/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.0844 - val_loss: 10.1372
Epoch 904/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.2976 - val_loss: 9.9998
Epoch 905/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.1942 - val_loss: 9.1891
Epoch 906/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.2006 - val_loss: 9.9403
Epoch 907/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.0829 - val_loss: 10.4732
Epoch 908/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.2750 - val_loss: 9.0838
Epoch 909/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.0495 - val_loss: 9.8408
Epoch 910/1000
1052/1052 [========================

1052/1052 [==============================] - 0s 85us/step - loss: 8.0042 - val_loss: 9.6143
Epoch 978/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.7015 - val_loss: 11.5872
Epoch 979/1000
1052/1052 [==============================] - 0s 83us/step - loss: 8.2088 - val_loss: 9.3087
Epoch 980/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.0416 - val_loss: 10.4447
Epoch 981/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.1524 - val_loss: 9.2050
Epoch 982/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.1187 - val_loss: 8.8649
Epoch 983/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.1899 - val_loss: 9.4459
Epoch 984/1000
1052/1052 [==============================] - 0s 83us/step - loss: 8.1833 - val_loss: 8.9990
Epoch 985/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.3699 - val_loss: 9.5650
Epoch 986/1000
1052/1052 [========================

[array([[-3.37024   , -1.8646775 ,  3.9129682 ,  0.7887742 ,  0.16711254],
        [ 0.08870032,  0.51897234, -0.2069316 ,  1.6172274 ,  0.16246352],
        [ 0.02301222,  0.41476265,  0.7276812 ,  1.7831668 ,  0.28526914],
        [ 0.07521304,  0.01172149, -0.14392282, -0.2910428 , -0.11031183],
        [-2.422672  , -0.30629653,  0.31992775, -0.46970576,  0.16660307]],
       dtype=float32),
 array([-4.8658543, -1.3226448,  4.880152 ,  0.745566 , -1.1464118],
       dtype=float32),
 array([[ 1.3732266 , -1.8020537 , -2.286285  ,  1.5273933 ,  1.7434138 ,
          1.1416508 ,  2.2840726 ,  1.5929793 , -2.0280218 ,  1.6114348 ],
        [-0.25837895,  0.42347786,  0.47322845, -0.96187097, -0.6763954 ,
         -0.7621005 , -0.30140808, -0.8166893 ,  0.72778213, -0.6212528 ],
        [-2.1621978 ,  2.522412  ,  2.075805  , -1.8423162 , -1.9116014 ,
         -1.8033962 , -2.2672977 , -2.008114  ,  2.4968674 , -1.8061601 ],
        [ 0.64213854, -0.7641411 ,  0.07502489, -0.3540214 ,  

In [29]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_structure2_3rd.h5')

In [30]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression_3(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_5 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_8 (Dense)              (None, 15)                90        
_________________________________________________________________
activation_6 (Activation)    (None, 15)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 16        
Total params: 136
Trainable params: 136
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 213us/step - l

1052/1052 [==============================] - 0s 85us/step - loss: 16.8512 - val_loss: 22.1974
Epoch 68/1000
1052/1052 [==============================] - 0s 86us/step - loss: 16.6209 - val_loss: 19.8053
Epoch 69/1000
1052/1052 [==============================] - 0s 85us/step - loss: 16.1418 - val_loss: 19.8559
Epoch 70/1000
1052/1052 [==============================] - 0s 81us/step - loss: 15.8433 - val_loss: 19.7265
Epoch 71/1000
1052/1052 [==============================] - 0s 86us/step - loss: 15.7838 - val_loss: 19.1839
Epoch 72/1000
1052/1052 [==============================] - 0s 85us/step - loss: 15.6668 - val_loss: 19.3731
Epoch 73/1000
1052/1052 [==============================] - 0s 84us/step - loss: 15.6559 - val_loss: 18.9092
Epoch 74/1000
1052/1052 [==============================] - 0s 85us/step - loss: 15.6605 - val_loss: 19.3108
Epoch 75/1000
1052/1052 [==============================] - 0s 82us/step - loss: 15.4723 - val_loss: 19.0944
Epoch 76/1000
1052/1052 [=================

1052/1052 [==============================] - 0s 83us/step - loss: 12.6328 - val_loss: 14.6304
Epoch 143/1000
1052/1052 [==============================] - 0s 88us/step - loss: 11.6214 - val_loss: 14.5312
Epoch 144/1000
1052/1052 [==============================] - 0s 87us/step - loss: 11.3396 - val_loss: 13.9855
Epoch 145/1000
1052/1052 [==============================] - 0s 82us/step - loss: 11.6969 - val_loss: 14.6622
Epoch 146/1000
1052/1052 [==============================] - 0s 84us/step - loss: 11.3914 - val_loss: 13.9142
Epoch 147/1000
1052/1052 [==============================] - 0s 83us/step - loss: 11.0772 - val_loss: 13.5802
Epoch 148/1000
1052/1052 [==============================] - 0s 85us/step - loss: 11.3965 - val_loss: 14.6678
Epoch 149/1000
1052/1052 [==============================] - 0s 87us/step - loss: 11.5762 - val_loss: 13.5769
Epoch 150/1000
1052/1052 [==============================] - 0s 86us/step - loss: 11.5596 - val_loss: 14.2433
Epoch 151/1000
1052/1052 [========

1052/1052 [==============================] - 0s 89us/step - loss: 10.1204 - val_loss: 12.1757
Epoch 218/1000
1052/1052 [==============================] - 0s 86us/step - loss: 10.4031 - val_loss: 12.1821
Epoch 219/1000
1052/1052 [==============================] - 0s 88us/step - loss: 9.8621 - val_loss: 12.3079
Epoch 220/1000
1052/1052 [==============================] - 0s 86us/step - loss: 10.1921 - val_loss: 12.2251
Epoch 221/1000
1052/1052 [==============================] - 0s 85us/step - loss: 10.3782 - val_loss: 12.4062
Epoch 222/1000
1052/1052 [==============================] - 0s 89us/step - loss: 10.6167 - val_loss: 12.2659
Epoch 223/1000
1052/1052 [==============================] - 0s 93us/step - loss: 10.0440 - val_loss: 12.2011
Epoch 224/1000
1052/1052 [==============================] - 0s 89us/step - loss: 10.0319 - val_loss: 11.9119
Epoch 225/1000
1052/1052 [==============================] - 0s 86us/step - loss: 9.7746 - val_loss: 12.7299
Epoch 226/1000
1052/1052 [==========

Epoch 293/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.6377 - val_loss: 9.8146
Epoch 294/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.6088 - val_loss: 10.8688
Epoch 295/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.3971 - val_loss: 10.4111
Epoch 296/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.8407 - val_loss: 11.1344
Epoch 297/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.7871 - val_loss: 9.5014
Epoch 298/1000
1052/1052 [==============================] - 0s 83us/step - loss: 9.1950 - val_loss: 10.2397
Epoch 299/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.8130 - val_loss: 10.2329
Epoch 300/1000
1052/1052 [==============================] - 0s 89us/step - loss: 9.1247 - val_loss: 11.0063
Epoch 301/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.6598 - val_loss: 9.5322
Epoch 302/1000
1052/1052 [=====

...

1052/1052 [==============================] - 0s 91us/step - loss: 8.7200 - val_loss: 12.0851
Epoch 750/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.0865 - val_loss: 8.4977
Epoch 751/1000
1052/1052 [==============================] - 0s 86us/step - loss: 7.4169 - val_loss: 9.4073
Epoch 752/1000
1052/1052 [==============================] - 0s 84us/step - loss: 7.5269 - val_loss: 9.4155
Epoch 753/1000
1052/1052 [==============================] - 0s 86us/step - loss: 7.2050 - val_loss: 10.3421
Epoch 754/1000
1052/1052 [==============================] - 0s 82us/step - loss: 7.4698 - val_loss: 9.3928
Epoch 755/1000
1052/1052 [==============================] - 0s 84us/step - loss: 7.6281 - val_loss: 8.6749
Epoch 756/1000
1052/1052 [==============================] - 0s 84us/step - loss: 7.4892 - val_loss: 8.8792
Epoch 757/1000
1052/1052 [==============================] - 0s 84us/step - loss: 7.6193 - val_loss: 9.2099
Epoch 758/1000
1052/1052 [========================

1052/1052 [==============================] - 0s 86us/step - loss: 8.0573 - val_loss: 8.7503
Epoch 826/1000
1052/1052 [==============================] - 0s 89us/step - loss: 7.7234 - val_loss: 9.6442
Epoch 827/1000
1052/1052 [==============================] - 0s 86us/step - loss: 7.3495 - val_loss: 8.5533
Epoch 828/1000
1052/1052 [==============================] - 0s 86us/step - loss: 7.6867 - val_loss: 9.4821
Epoch 829/1000
1052/1052 [==============================] - 0s 85us/step - loss: 7.7106 - val_loss: 8.7314
Epoch 830/1000
1052/1052 [==============================] - 0s 88us/step - loss: 7.4494 - val_loss: 8.3039
Epoch 831/1000
1052/1052 [==============================] - 0s 83us/step - loss: 7.0676 - val_loss: 9.8126
Epoch 832/1000
1052/1052 [==============================] - 0s 86us/step - loss: 7.2980 - val_loss: 9.1743
Epoch 833/1000
1052/1052 [==============================] - 0s 88us/step - loss: 7.4064 - val_loss: 10.0873
Epoch 834/1000
1052/1052 [=========================

1052/1052 [==============================] - 0s 89us/step - loss: 7.4195 - val_loss: 8.4086
Epoch 902/1000
1052/1052 [==============================] - 0s 91us/step - loss: 7.3312 - val_loss: 8.6147
Epoch 903/1000
1052/1052 [==============================] - 0s 92us/step - loss: 7.4754 - val_loss: 10.3001
Epoch 904/1000
1052/1052 [==============================] - 0s 95us/step - loss: 7.6725 - val_loss: 8.6203
Epoch 905/1000
1052/1052 [==============================] - 0s 88us/step - loss: 7.5897 - val_loss: 9.0609
Epoch 906/1000
1052/1052 [==============================] - 0s 81us/step - loss: 7.3948 - val_loss: 8.5443
Epoch 907/1000
1052/1052 [==============================] - 0s 93us/step - loss: 7.5050 - val_loss: 10.3258
Epoch 908/1000
1052/1052 [==============================] - 0s 93us/step - loss: 7.7464 - val_loss: 8.7551
Epoch 909/1000
1052/1052 [==============================] - 0s 99us/step - loss: 7.5826 - val_loss: 9.1383
Epoch 910/1000
1052/1052 [========================

1052/1052 [==============================] - 0s 99us/step - loss: 7.6574 - val_loss: 9.9845
Epoch 978/1000
1052/1052 [==============================] - 0s 92us/step - loss: 8.4568 - val_loss: 9.1954
Epoch 979/1000
1052/1052 [==============================] - 0s 94us/step - loss: 7.9439 - val_loss: 10.0103
Epoch 980/1000
1052/1052 [==============================] - 0s 94us/step - loss: 8.1677 - val_loss: 10.4508
Epoch 981/1000
1052/1052 [==============================] - 0s 82us/step - loss: 7.7686 - val_loss: 8.8410
Epoch 982/1000
1052/1052 [==============================] - 0s 83us/step - loss: 7.2700 - val_loss: 9.3975
Epoch 983/1000
1052/1052 [==============================] - 0s 79us/step - loss: 7.5758 - val_loss: 9.3276
Epoch 984/1000
1052/1052 [==============================] - 0s 87us/step - loss: 7.5144 - val_loss: 8.4517
Epoch 985/1000
1052/1052 [==============================] - 0s 91us/step - loss: 7.4926 - val_loss: 8.2935
Epoch 986/1000
1052/1052 [========================

[array([[-0.1592215 ,  2.7216642 ,  3.0954537 ,  2.01829   , -0.6039481 ],
        [-0.24132442, -0.24869639, -0.11973082,  0.12322515, -0.2548509 ],
        [-0.6290021 , -0.13947985,  1.0468621 ,  0.10290544,  0.6356868 ],
        [ 0.0943072 , -0.10151696, -0.1323905 , -0.09238867,  0.03579548],
        [-0.12589242,  0.2791103 ,  0.1332975 ,  1.3031863 , -1.2103715 ]],
       dtype=float32),
 array([-0.2911458,  1.4950378,  4.6018014,  2.9649096, -1.0657979],
       dtype=float32),
 array([[-8.0463886e-01,  7.0006633e-01,  8.8542283e-01, -1.0699587e+00,
          9.8526567e-02,  7.7192599e-01,  4.9396464e-01, -1.0851717e+00,
         -6.2383717e-01, -7.2217178e-01,  3.0238974e-01, -6.3772529e-01,
          9.5855737e-01, -4.5049158e-01,  7.2859538e-01],
        [ 8.3909827e-01, -1.0890035e-03, -7.8165668e-01,  5.8485681e-01,
         -4.1357982e-01, -4.5805448e-01, -6.3674384e-01,  2.8160796e-01,
          2.9775521e-01,  1.5908298e-01, -6.1002725e-01,  2.4680129e-01,
         -6.5

In [31]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_structure3_3rd.h5')

In [32]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression_4(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_7 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 5)                 55        
_________________________________________________________________
activation_8 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 6         
Total params: 121
Trainable params: 121
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 230us/step - l

1052/1052 [==============================] - 0s 89us/step - loss: 14.6723 - val_loss: 18.6771
Epoch 68/1000
1052/1052 [==============================] - 0s 87us/step - loss: 14.7482 - val_loss: 17.4464
Epoch 69/1000
1052/1052 [==============================] - 0s 88us/step - loss: 14.7385 - val_loss: 18.2627
Epoch 70/1000
1052/1052 [==============================] - 0s 89us/step - loss: 14.9484 - val_loss: 18.4233
Epoch 71/1000
1052/1052 [==============================] - 0s 82us/step - loss: 15.0993 - val_loss: 17.5494
Epoch 72/1000
1052/1052 [==============================] - 0s 90us/step - loss: 14.3347 - val_loss: 18.0984
Epoch 73/1000
1052/1052 [==============================] - 0s 89us/step - loss: 14.9251 - val_loss: 17.1572
Epoch 74/1000
1052/1052 [==============================] - 0s 92us/step - loss: 15.6063 - val_loss: 19.1451
Epoch 75/1000
1052/1052 [==============================] - 0s 88us/step - loss: 14.4885 - val_loss: 18.3102
Epoch 76/1000
1052/1052 [=================

1052/1052 [==============================] - 0s 99us/step - loss: 12.3784 - val_loss: 14.7699
Epoch 143/1000
1052/1052 [==============================] - 0s 95us/step - loss: 11.7007 - val_loss: 14.8595
Epoch 144/1000
1052/1052 [==============================] - 0s 86us/step - loss: 11.3505 - val_loss: 14.8699
Epoch 145/1000
1052/1052 [==============================] - 0s 87us/step - loss: 11.1963 - val_loss: 14.1974
Epoch 146/1000
1052/1052 [==============================] - 0s 87us/step - loss: 10.9086 - val_loss: 14.3775
Epoch 147/1000
1052/1052 [==============================] - 0s 89us/step - loss: 10.9211 - val_loss: 15.1848
Epoch 148/1000
1052/1052 [==============================] - 0s 87us/step - loss: 10.8625 - val_loss: 15.6891
Epoch 149/1000
1052/1052 [==============================] - 0s 87us/step - loss: 10.9348 - val_loss: 13.0638
Epoch 150/1000
1052/1052 [==============================] - 0s 86us/step - loss: 10.8488 - val_loss: 13.4029
Epoch 151/1000
1052/1052 [========

Epoch 218/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.4358 - val_loss: 10.1153
Epoch 219/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.4396 - val_loss: 9.7094
Epoch 220/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.4304 - val_loss: 9.8664
Epoch 221/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.2414 - val_loss: 10.3432
Epoch 222/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.5393 - val_loss: 11.7304
Epoch 223/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.5389 - val_loss: 11.2897
Epoch 224/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.4710 - val_loss: 10.1635
Epoch 225/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.2858 - val_loss: 10.3395
Epoch 226/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.1297 - val_loss: 10.2709
Epoch 227/1000
1052/1052 [====

1052/1052 [==============================] - 0s 92us/step - loss: 8.0294 - val_loss: 9.3895
Epoch 295/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.3591 - val_loss: 10.0686
Epoch 296/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.4104 - val_loss: 10.2780
Epoch 297/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.1250 - val_loss: 9.2946
Epoch 298/1000
1052/1052 [==============================] - 0s 95us/step - loss: 8.2751 - val_loss: 9.6055
Epoch 299/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.4049 - val_loss: 11.5794
Epoch 300/1000
1052/1052 [==============================] - 0s 95us/step - loss: 8.4748 - val_loss: 9.6444
Epoch 301/1000
1052/1052 [==============================] - 0s 93us/step - loss: 8.4957 - val_loss: 11.1378
Epoch 302/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.2689 - val_loss: 11.4461
Epoch 303/1000
1052/1052 [=====================

...

1052/1052 [==============================] - 0s 89us/step - loss: 8.2517 - val_loss: 9.8937
Epoch 751/1000
1052/1052 [==============================] - 0s 88us/step - loss: 7.9098 - val_loss: 9.1219
Epoch 752/1000
1052/1052 [==============================] - 0s 88us/step - loss: 7.8171 - val_loss: 9.1783
Epoch 753/1000
1052/1052 [==============================] - 0s 101us/step - loss: 7.7509 - val_loss: 8.5392
Epoch 754/1000
1052/1052 [==============================] - 0s 92us/step - loss: 7.7504 - val_loss: 8.5309
Epoch 755/1000
1052/1052 [==============================] - 0s 86us/step - loss: 7.9633 - val_loss: 9.0423
Epoch 756/1000
1052/1052 [==============================] - 0s 84us/step - loss: 7.9724 - val_loss: 8.6091
Epoch 757/1000
1052/1052 [==============================] - 0s 88us/step - loss: 7.8439 - val_loss: 8.3766
Epoch 758/1000
1052/1052 [==============================] - 0s 85us/step - loss: 7.9750 - val_loss: 8.6102
Epoch 759/1000
1052/1052 [=========================

1052/1052 [==============================] - 0s 95us/step - loss: 7.8244 - val_loss: 9.4946
Epoch 827/1000
1052/1052 [==============================] - 0s 93us/step - loss: 8.6457 - val_loss: 8.6803
Epoch 828/1000
1052/1052 [==============================] - 0s 93us/step - loss: 7.7420 - val_loss: 8.8203
Epoch 829/1000
1052/1052 [==============================] - 0s 97us/step - loss: 7.5324 - val_loss: 8.6396
Epoch 830/1000
1052/1052 [==============================] - 0s 91us/step - loss: 7.5348 - val_loss: 8.8138
Epoch 831/1000
1052/1052 [==============================] - 0s 92us/step - loss: 7.5930 - val_loss: 8.3698
Epoch 832/1000
1052/1052 [==============================] - 0s 93us/step - loss: 7.7278 - val_loss: 8.8195
Epoch 833/1000
1052/1052 [==============================] - 0s 94us/step - loss: 7.8493 - val_loss: 9.0456
Epoch 834/1000
1052/1052 [==============================] - 0s 95us/step - loss: 7.6359 - val_loss: 9.0975
Epoch 835/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 87us/step - loss: 7.7910 - val_loss: 9.2320
Epoch 903/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.4536 - val_loss: 8.8459
Epoch 904/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.4282 - val_loss: 8.9795
Epoch 905/1000
1052/1052 [==============================] - 0s 88us/step - loss: 7.7315 - val_loss: 8.6621
Epoch 906/1000
1052/1052 [==============================] - 0s 86us/step - loss: 7.4979 - val_loss: 8.6590
Epoch 907/1000
1052/1052 [==============================] - 0s 83us/step - loss: 7.5048 - val_loss: 8.7529
Epoch 908/1000
1052/1052 [==============================] - 0s 89us/step - loss: 7.7972 - val_loss: 8.9751
Epoch 909/1000
1052/1052 [==============================] - 0s 87us/step - loss: 7.9600 - val_loss: 8.7861
Epoch 910/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.3348 - val_loss: 8.7773
Epoch 911/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 82us/step - loss: 7.8563 - val_loss: 8.5993
Epoch 979/1000
1052/1052 [==============================] - 0s 83us/step - loss: 7.5810 - val_loss: 9.4839
Epoch 980/1000
1052/1052 [==============================] - 0s 85us/step - loss: 7.5733 - val_loss: 9.4875
Epoch 981/1000
1052/1052 [==============================] - 0s 87us/step - loss: 7.4459 - val_loss: 8.6366
Epoch 982/1000
1052/1052 [==============================] - 0s 87us/step - loss: 7.3368 - val_loss: 9.1941
Epoch 983/1000
1052/1052 [==============================] - 0s 85us/step - loss: 7.4932 - val_loss: 8.6597
Epoch 984/1000
1052/1052 [==============================] - 0s 88us/step - loss: 7.6968 - val_loss: 9.3397
Epoch 985/1000
1052/1052 [==============================] - 0s 85us/step - loss: 7.2640 - val_loss: 8.8419
Epoch 986/1000
1052/1052 [==============================] - 0s 85us/step - loss: 7.3551 - val_loss: 8.5069
Epoch 987/1000
1052/1052 [==========================

[array([[ 0.29754686, -4.5374136 , -1.8018076 , -0.48563626, -0.40350598,
          0.43346035,  4.8322835 ,  0.47281808, -0.22530831, -2.5244021 ],
        [ 0.16159979, -0.1929506 ,  0.08117052, -0.0918889 , -1.6149272 ,
          0.9668325 , -0.16320778,  0.4064741 , -0.07328901,  0.20474364],
        [ 0.3463221 , -0.33221087,  0.7638202 , -0.3557692 , -1.954208  ,
          1.4365307 ,  1.1966695 ,  0.57852226, -0.26220226,  0.22818694],
        [-0.97523445,  0.176037  , -0.653972  ,  0.14045495,  0.29302204,
          0.2096561 , -0.21638705, -1.0314046 ,  0.939379  ,  0.05163943],
        [ 0.4154829 , -3.722454  ,  0.29697382, -0.60596424,  0.7041925 ,
          0.56830657,  0.3034414 ,  0.3608603 , -0.4313825 , -0.21422835]],
       dtype=float32),
 array([-3.9069293 , -5.3981533 , -4.4114375 ,  0.27840617, -1.1511276 ,
        -4.455686  ,  5.6199117 , -3.8311985 ,  4.14112   , -1.3430954 ],
       dtype=float32),
 array([[-9.8712832e-01,  1.7560444e+00,  2.3376446e+00,  1.9

In [33]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_structure4_3rd.h5')

In [34]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression_5(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_9 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 10)                110       
_________________________________________________________________
activation_10 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 11        
Total params: 181
Trainable params: 181
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 243us/step - l

1052/1052 [==============================] - 0s 85us/step - loss: 13.9165 - val_loss: 16.7549
Epoch 68/1000
1052/1052 [==============================] - 0s 87us/step - loss: 13.9349 - val_loss: 17.0671
Epoch 69/1000
1052/1052 [==============================] - 0s 85us/step - loss: 14.0035 - val_loss: 18.4682
Epoch 70/1000
1052/1052 [==============================] - 0s 84us/step - loss: 13.4367 - val_loss: 16.7486
Epoch 71/1000
1052/1052 [==============================] - 0s 89us/step - loss: 14.2649 - val_loss: 17.3470
Epoch 72/1000
1052/1052 [==============================] - 0s 88us/step - loss: 13.7686 - val_loss: 16.4662
Epoch 73/1000
1052/1052 [==============================] - 0s 90us/step - loss: 13.6939 - val_loss: 17.6556
Epoch 74/1000
1052/1052 [==============================] - 0s 87us/step - loss: 13.4045 - val_loss: 16.2460
Epoch 75/1000
1052/1052 [==============================] - 0s 81us/step - loss: 13.2750 - val_loss: 16.8770
Epoch 76/1000
1052/1052 [=================

1052/1052 [==============================] - 0s 94us/step - loss: 9.3035 - val_loss: 11.3482
Epoch 143/1000
1052/1052 [==============================] - 0s 90us/step - loss: 9.3199 - val_loss: 12.3512
Epoch 144/1000
1052/1052 [==============================] - 0s 87us/step - loss: 9.4082 - val_loss: 11.5237
Epoch 145/1000
1052/1052 [==============================] - 0s 91us/step - loss: 9.5921 - val_loss: 11.5009
Epoch 146/1000
1052/1052 [==============================] - 0s 89us/step - loss: 9.4691 - val_loss: 14.3723
Epoch 147/1000
1052/1052 [==============================] - 0s 83us/step - loss: 9.7703 - val_loss: 12.0054
Epoch 148/1000
1052/1052 [==============================] - 0s 85us/step - loss: 9.9200 - val_loss: 11.3656
Epoch 149/1000
1052/1052 [==============================] - 0s 89us/step - loss: 9.3100 - val_loss: 13.6796
Epoch 150/1000
1052/1052 [==============================] - 0s 84us/step - loss: 9.5045 - val_loss: 11.7841
Epoch 151/1000
1052/1052 [=================

1052/1052 [==============================] - 0s 87us/step - loss: 8.8080 - val_loss: 10.9657
Epoch 219/1000
1052/1052 [==============================] - 0s 87us/step - loss: 9.2481 - val_loss: 9.9716
Epoch 220/1000
1052/1052 [==============================] - 0s 88us/step - loss: 9.1592 - val_loss: 10.1301
Epoch 221/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.9822 - val_loss: 11.6855
Epoch 222/1000
1052/1052 [==============================] - 0s 86us/step - loss: 9.6531 - val_loss: 10.0216
Epoch 223/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.4770 - val_loss: 9.8717
Epoch 224/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.6715 - val_loss: 9.7647
Epoch 225/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.4632 - val_loss: 11.5566
Epoch 226/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.8660 - val_loss: 9.8385
Epoch 227/1000
1052/1052 [=====================

1052/1052 [==============================] - 0s 86us/step - loss: 7.9680 - val_loss: 9.2967
Epoch 295/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.4390 - val_loss: 10.0623
Epoch 296/1000
1052/1052 [==============================] - 0s 88us/step - loss: 7.8995 - val_loss: 9.2059
Epoch 297/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.0028 - val_loss: 8.8332
Epoch 298/1000
1052/1052 [==============================] - 0s 89us/step - loss: 8.3026 - val_loss: 9.6711
Epoch 299/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.1124 - val_loss: 8.8523
Epoch 300/1000
1052/1052 [==============================] - 0s 98us/step - loss: 8.6262 - val_loss: 9.9803
Epoch 301/1000
1052/1052 [==============================] - 0s 97us/step - loss: 8.1254 - val_loss: 9.6091
Epoch 302/1000
1052/1052 [==============================] - 0s 97us/step - loss: 7.9059 - val_loss: 9.2817
Epoch 303/1000
1052/1052 [=========================

...

1052/1052 [==============================] - 0s 85us/step - loss: 5.7027 - val_loss: 6.6058
Epoch 751/1000
1052/1052 [==============================] - 0s 86us/step - loss: 6.1489 - val_loss: 6.1026
Epoch 752/1000
1052/1052 [==============================] - 0s 86us/step - loss: 6.1395 - val_loss: 6.9404
Epoch 753/1000
1052/1052 [==============================] - 0s 88us/step - loss: 5.9735 - val_loss: 7.4024
Epoch 754/1000
1052/1052 [==============================] - 0s 91us/step - loss: 6.4334 - val_loss: 7.3495
Epoch 755/1000
1052/1052 [==============================] - 0s 86us/step - loss: 6.0460 - val_loss: 6.0840
Epoch 756/1000
1052/1052 [==============================] - 0s 88us/step - loss: 5.7173 - val_loss: 6.5417
Epoch 757/1000
1052/1052 [==============================] - 0s 90us/step - loss: 5.7198 - val_loss: 6.1341
Epoch 758/1000
1052/1052 [==============================] - 0s 86us/step - loss: 5.9236 - val_loss: 6.4056
Epoch 759/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 89us/step - loss: 5.6818 - val_loss: 6.1882
Epoch 827/1000
1052/1052 [==============================] - 0s 90us/step - loss: 5.6532 - val_loss: 6.8136
Epoch 828/1000
1052/1052 [==============================] - 0s 91us/step - loss: 6.0830 - val_loss: 7.8821
Epoch 829/1000
1052/1052 [==============================] - 0s 86us/step - loss: 6.1121 - val_loss: 6.7204
Epoch 830/1000
1052/1052 [==============================] - 0s 88us/step - loss: 5.9365 - val_loss: 5.9652
Epoch 831/1000
1052/1052 [==============================] - 0s 88us/step - loss: 5.7479 - val_loss: 5.9378
Epoch 832/1000
1052/1052 [==============================] - 0s 85us/step - loss: 5.5729 - val_loss: 6.1087
Epoch 833/1000
1052/1052 [==============================] - 0s 83us/step - loss: 5.6958 - val_loss: 6.3672
Epoch 834/1000
1052/1052 [==============================] - 0s 95us/step - loss: 5.6671 - val_loss: 6.6735
Epoch 835/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 94us/step - loss: 5.4587 - val_loss: 5.7877
Epoch 903/1000
1052/1052 [==============================] - 0s 99us/step - loss: 5.3730 - val_loss: 6.2196
Epoch 904/1000
1052/1052 [==============================] - 0s 93us/step - loss: 5.4832 - val_loss: 5.7972
Epoch 905/1000
1052/1052 [==============================] - 0s 93us/step - loss: 6.0994 - val_loss: 7.6965
Epoch 906/1000
1052/1052 [==============================] - 0s 93us/step - loss: 6.0543 - val_loss: 5.8133
Epoch 907/1000
1052/1052 [==============================] - 0s 100us/step - loss: 5.6260 - val_loss: 5.5127
Epoch 908/1000
1052/1052 [==============================] - 0s 91us/step - loss: 5.6201 - val_loss: 6.0474
Epoch 909/1000
1052/1052 [==============================] - 0s 91us/step - loss: 5.7215 - val_loss: 7.3808
Epoch 910/1000
1052/1052 [==============================] - 0s 92us/step - loss: 5.3453 - val_loss: 5.5765
Epoch 911/1000
1052/1052 [=========================

1052/1052 [==============================] - 0s 95us/step - loss: 5.8506 - val_loss: 6.0835
Epoch 979/1000
1052/1052 [==============================] - 0s 98us/step - loss: 5.8016 - val_loss: 8.0876
Epoch 980/1000
1052/1052 [==============================] - 0s 98us/step - loss: 6.0019 - val_loss: 6.5943
Epoch 981/1000
1052/1052 [==============================] - 0s 98us/step - loss: 5.6720 - val_loss: 6.3669
Epoch 982/1000
1052/1052 [==============================] - 0s 100us/step - loss: 5.7851 - val_loss: 5.3237
Epoch 983/1000
1052/1052 [==============================] - 0s 103us/step - loss: 5.4257 - val_loss: 5.5493
Epoch 984/1000
1052/1052 [==============================] - 0s 92us/step - loss: 5.4376 - val_loss: 5.4502
Epoch 985/1000
1052/1052 [==============================] - 0s 85us/step - loss: 5.6196 - val_loss: 5.8631
Epoch 986/1000
1052/1052 [==============================] - 0s 90us/step - loss: 5.5497 - val_loss: 5.9774
Epoch 987/1000
1052/1052 [========================

[array([[-4.54894   ,  0.756763  , -3.8925295 ,  3.8499572 , -0.9689762 ,
         -0.74325883,  3.0303266 , -2.8322222 ,  0.4462247 , -0.24653126],
        [ 0.18181923,  0.40499443, -0.01979484, -0.13261867, -0.8788064 ,
         -0.72159886,  0.4383766 ,  0.06843939,  1.008557  , -0.49589366],
        [-0.2688333 ,  0.7279026 ,  0.08457325,  1.4294361 , -0.24042922,
         -0.67887473,  1.9895313 , -0.16964094,  2.0019002 , -1.6644448 ],
        [ 0.39877042, -0.9538043 , -0.08044741, -0.17802957,  0.31061208,
          1.0346493 , -0.13067292,  0.08017372,  0.31834114,  0.10459998],
        [-4.4781966 ,  1.0186802 , -1.2349389 , -0.08900424, -1.3028768 ,
         -0.46460965,  0.40499592, -0.24094865,  0.6062392 ,  0.09481625]],
       dtype=float32),
 array([-5.4819655, -2.6828554, -4.714987 ,  5.7202344, -0.5550474,
         4.9839916,  3.4100168, -1.5411057, -5.476604 , -1.1657927],
       dtype=float32),
 array([[-2.7653232e-01, -1.5489629e-01, -8.0517429e-01,  3.7374225e-01

In [35]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_structure5_3rd.h5')

In [36]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression_6(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_11 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_17 (Dense)             (None, 15)                165       
_________________________________________________________________
activation_12 (Activation)   (None, 15)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 16        
Total params: 241
Trainable params: 241
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 264us/step - l

1052/1052 [==============================] - 0s 88us/step - loss: 11.5437 - val_loss: 13.8580
Epoch 68/1000
1052/1052 [==============================] - 0s 87us/step - loss: 11.0222 - val_loss: 13.3494
Epoch 69/1000
1052/1052 [==============================] - 0s 87us/step - loss: 10.7669 - val_loss: 13.7037
Epoch 70/1000
1052/1052 [==============================] - 0s 88us/step - loss: 10.9134 - val_loss: 13.3723
Epoch 71/1000
1052/1052 [==============================] - 0s 88us/step - loss: 10.6771 - val_loss: 12.4935
Epoch 72/1000
1052/1052 [==============================] - 0s 87us/step - loss: 10.7026 - val_loss: 12.2793
Epoch 73/1000
1052/1052 [==============================] - 0s 88us/step - loss: 10.3515 - val_loss: 12.7129
Epoch 74/1000
1052/1052 [==============================] - 0s 87us/step - loss: 10.0240 - val_loss: 11.9841
Epoch 75/1000
1052/1052 [==============================] - 0s 86us/step - loss: 9.8368 - val_loss: 12.4332
Epoch 76/1000
1052/1052 [==================

1052/1052 [==============================] - 0s 97us/step - loss: 8.0691 - val_loss: 11.4400
Epoch 144/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.0513 - val_loss: 10.0370
Epoch 145/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.1593 - val_loss: 8.5590
Epoch 146/1000
1052/1052 [==============================] - 0s 85us/step - loss: 7.8440 - val_loss: 8.8009
Epoch 147/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.0731 - val_loss: 9.0106
Epoch 148/1000
1052/1052 [==============================] - 0s 88us/step - loss: 8.1084 - val_loss: 10.4891
Epoch 149/1000
1052/1052 [==============================] - 0s 86us/step - loss: 7.6464 - val_loss: 10.1711
Epoch 150/1000
1052/1052 [==============================] - 0s 88us/step - loss: 7.9633 - val_loss: 8.5759
Epoch 151/1000
1052/1052 [==============================] - 0s 89us/step - loss: 7.4278 - val_loss: 9.0228
Epoch 152/1000
1052/1052 [======================

1052/1052 [==============================] - 0s 93us/step - loss: 7.9296 - val_loss: 9.6600
Epoch 220/1000
1052/1052 [==============================] - 0s 89us/step - loss: 7.7239 - val_loss: 7.9962
Epoch 221/1000
1052/1052 [==============================] - 0s 94us/step - loss: 7.1559 - val_loss: 8.8249
Epoch 222/1000
1052/1052 [==============================] - 0s 90us/step - loss: 7.2146 - val_loss: 8.6822
Epoch 223/1000
1052/1052 [==============================] - 0s 86us/step - loss: 7.4438 - val_loss: 10.2214
Epoch 224/1000
1052/1052 [==============================] - 0s 90us/step - loss: 7.3045 - val_loss: 7.9629
Epoch 225/1000
1052/1052 [==============================] - 0s 94us/step - loss: 7.1947 - val_loss: 8.5502
Epoch 226/1000
1052/1052 [==============================] - 0s 93us/step - loss: 7.0591 - val_loss: 9.8903
Epoch 227/1000
1052/1052 [==============================] - 0s 93us/step - loss: 7.7344 - val_loss: 9.7342
Epoch 228/1000
1052/1052 [=========================

1052/1052 [==============================] - 0s 91us/step - loss: 6.3347 - val_loss: 7.2811
Epoch 296/1000
1052/1052 [==============================] - 0s 88us/step - loss: 6.6024 - val_loss: 8.2179
Epoch 297/1000
1052/1052 [==============================] - 0s 101us/step - loss: 6.5964 - val_loss: 7.1848
Epoch 298/1000
1052/1052 [==============================] - 0s 100us/step - loss: 6.7308 - val_loss: 7.8411
Epoch 299/1000
1052/1052 [==============================] - 0s 99us/step - loss: 6.6480 - val_loss: 8.4133
Epoch 300/1000
1052/1052 [==============================] - 0s 115us/step - loss: 6.5275 - val_loss: 7.0430
Epoch 301/1000
1052/1052 [==============================] - 0s 94us/step - loss: 6.6669 - val_loss: 7.0769
Epoch 302/1000
1052/1052 [==============================] - 0s 100us/step - loss: 7.8875 - val_loss: 7.6614
Epoch 303/1000
1052/1052 [==============================] - 0s 101us/step - loss: 7.0369 - val_loss: 8.5731
Epoch 304/1000
1052/1052 [=====================

...

1052/1052 [==============================] - 0s 85us/step - loss: 5.4954 - val_loss: 5.9034
Epoch 752/1000
1052/1052 [==============================] - 0s 84us/step - loss: 5.5246 - val_loss: 5.3447
Epoch 753/1000
1052/1052 [==============================] - 0s 86us/step - loss: 5.3409 - val_loss: 7.0784
Epoch 754/1000
1052/1052 [==============================] - 0s 83us/step - loss: 5.3999 - val_loss: 6.0439
Epoch 755/1000
1052/1052 [==============================] - 0s 86us/step - loss: 5.3835 - val_loss: 5.6755
Epoch 756/1000
1052/1052 [==============================] - 0s 89us/step - loss: 5.1325 - val_loss: 5.8073
Epoch 757/1000
1052/1052 [==============================] - 0s 86us/step - loss: 5.0660 - val_loss: 5.5006
Epoch 758/1000
1052/1052 [==============================] - 0s 89us/step - loss: 5.5132 - val_loss: 5.4107
Epoch 759/1000
1052/1052 [==============================] - 0s 86us/step - loss: 5.2771 - val_loss: 7.0542
Epoch 760/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 87us/step - loss: 5.1363 - val_loss: 5.0006
Epoch 828/1000
1052/1052 [==============================] - 0s 87us/step - loss: 5.1859 - val_loss: 5.1717
Epoch 829/1000
1052/1052 [==============================] - 0s 86us/step - loss: 5.0495 - val_loss: 5.0752
Epoch 830/1000
1052/1052 [==============================] - 0s 90us/step - loss: 4.9099 - val_loss: 5.3570
Epoch 831/1000
1052/1052 [==============================] - 0s 85us/step - loss: 4.9449 - val_loss: 5.8890
Epoch 832/1000
1052/1052 [==============================] - 0s 84us/step - loss: 5.5213 - val_loss: 6.5549
Epoch 833/1000
1052/1052 [==============================] - 0s 89us/step - loss: 5.1677 - val_loss: 5.9382
Epoch 834/1000
1052/1052 [==============================] - 0s 87us/step - loss: 5.1563 - val_loss: 5.5431
Epoch 835/1000
1052/1052 [==============================] - 0s 85us/step - loss: 5.3019 - val_loss: 6.1061
Epoch 836/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 88us/step - loss: 5.1594 - val_loss: 5.3330
Epoch 904/1000
1052/1052 [==============================] - 0s 86us/step - loss: 4.7963 - val_loss: 5.3092
Epoch 905/1000
1052/1052 [==============================] - 0s 88us/step - loss: 5.3664 - val_loss: 5.3385
Epoch 906/1000
1052/1052 [==============================] - 0s 86us/step - loss: 5.2306 - val_loss: 5.6840
Epoch 907/1000
1052/1052 [==============================] - 0s 85us/step - loss: 4.9137 - val_loss: 5.3474
Epoch 908/1000
1052/1052 [==============================] - 0s 87us/step - loss: 4.9092 - val_loss: 5.5185
Epoch 909/1000
1052/1052 [==============================] - 0s 87us/step - loss: 5.5226 - val_loss: 5.5262
Epoch 910/1000
1052/1052 [==============================] - 0s 88us/step - loss: 5.1131 - val_loss: 5.3397
Epoch 911/1000
1052/1052 [==============================] - 0s 88us/step - loss: 4.9431 - val_loss: 5.6887
Epoch 912/1000
1052/1052 [==========================

1052/1052 [==============================] - 0s 86us/step - loss: 5.1851 - val_loss: 5.4642
Epoch 980/1000
1052/1052 [==============================] - 0s 90us/step - loss: 5.0646 - val_loss: 4.8452
Epoch 981/1000
1052/1052 [==============================] - 0s 83us/step - loss: 4.8691 - val_loss: 4.7939
Epoch 982/1000
1052/1052 [==============================] - 0s 89us/step - loss: 5.0935 - val_loss: 5.1617
Epoch 983/1000
1052/1052 [==============================] - 0s 89us/step - loss: 5.0309 - val_loss: 5.5500
Epoch 984/1000
1052/1052 [==============================] - 0s 87us/step - loss: 5.1455 - val_loss: 5.9252
Epoch 985/1000
1052/1052 [==============================] - 0s 96us/step - loss: 5.3570 - val_loss: 6.7392
Epoch 986/1000
1052/1052 [==============================] - 0s 87us/step - loss: 5.2662 - val_loss: 5.2603
Epoch 987/1000
1052/1052 [==============================] - 0s 87us/step - loss: 4.7807 - val_loss: 5.5327
Epoch 988/1000
1052/1052 [==========================

[array([[ 2.6125572 ,  0.6085845 ,  2.7822814 , -2.4259644 ,  4.2845445 ,
         -0.21569136,  3.9998188 ,  6.5871935 , -0.6558927 , -0.05071361],
        [ 0.17132154,  1.0442256 , -0.19284351, -0.05391383, -0.41260397,
          0.06051411,  0.9131112 , -0.2531927 , -0.7837451 , -0.5594172 ],
        [ 0.05192109,  1.8361807 , -0.08993945, -0.27352968,  0.6893319 ,
          0.83255756,  3.7348065 ,  0.14145748, -0.31194958, -2.0381355 ],
        [ 0.3232264 , -0.35635132, -0.10920472,  0.19332628, -0.23116262,
          0.15806773, -0.22525413, -0.04669935,  0.23658814,  0.0241079 ],
        [ 0.14833792,  0.5581028 ,  2.536186  , -0.04362506,  0.31994945,
         -1.1427736 , -0.0293513 ,  0.6382023 , -1.1412896 ,  1.021813  ]],
       dtype=float32),
 array([ 3.644897  , -5.051995  ,  4.0899477 , -0.25660986,  5.363279  ,
         2.338008  ,  5.258464  ,  3.7073808 , -0.39563835, -1.3181163 ],
       dtype=float32),
 array([[-1.8507884 ,  1.1801091 ,  1.3356609 ,  1.8140212 , 

In [37]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_structure6_3rd.h5')

In [38]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression_1(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_13 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_20 (Dense)             (None, 5)                 30        
_________________________________________________________________
activation_14 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 6         
Total params: 76
Trainable params: 76
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 791us/step - loss: 4

274/274 [==============================] - 0s 96us/step - loss: 7.1654 - val_loss: 9.5756
Epoch 70/1000
274/274 [==============================] - 0s 95us/step - loss: 7.1469 - val_loss: 9.6365
Epoch 71/1000
274/274 [==============================] - 0s 96us/step - loss: 7.1812 - val_loss: 9.6717
Epoch 72/1000
274/274 [==============================] - 0s 100us/step - loss: 7.3573 - val_loss: 9.6414
Epoch 73/1000
274/274 [==============================] - 0s 95us/step - loss: 7.2169 - val_loss: 9.5676
Epoch 74/1000
274/274 [==============================] - 0s 97us/step - loss: 7.2365 - val_loss: 9.6420
Epoch 75/1000
274/274 [==============================] - 0s 100us/step - loss: 7.3634 - val_loss: 9.5454
Epoch 76/1000
274/274 [==============================] - 0s 94us/step - loss: 7.3125 - val_loss: 9.7430
Epoch 77/1000
274/274 [==============================] - 0s 96us/step - loss: 7.1993 - val_loss: 9.5799
Epoch 78/1000
274/274 [==============================] - 0s 98us/step - loss

274/274 [==============================] - 0s 98us/step - loss: 6.9575 - val_loss: 9.4089
Epoch 148/1000
274/274 [==============================] - 0s 100us/step - loss: 6.9826 - val_loss: 9.4383
Epoch 149/1000
274/274 [==============================] - 0s 96us/step - loss: 6.9823 - val_loss: 9.2834
Epoch 150/1000
274/274 [==============================] - 0s 95us/step - loss: 7.0736 - val_loss: 9.4579
Epoch 151/1000
274/274 [==============================] - 0s 91us/step - loss: 6.9331 - val_loss: 9.6032
Epoch 152/1000
274/274 [==============================] - 0s 97us/step - loss: 7.0686 - val_loss: 9.4772
Epoch 153/1000
274/274 [==============================] - 0s 100us/step - loss: 6.9741 - val_loss: 9.4094
Epoch 154/1000
274/274 [==============================] - 0s 100us/step - loss: 6.9905 - val_loss: 9.3318
Epoch 155/1000
274/274 [==============================] - 0s 101us/step - loss: 6.9391 - val_loss: 9.4342
Epoch 156/1000
274/274 [==============================] - 0s 95us/

274/274 [==============================] - 0s 95us/step - loss: 6.7492 - val_loss: 9.4198
Epoch 226/1000
274/274 [==============================] - 0s 96us/step - loss: 6.7137 - val_loss: 9.1672
Epoch 227/1000
274/274 [==============================] - 0s 102us/step - loss: 6.8527 - val_loss: 8.9594
Epoch 228/1000
274/274 [==============================] - 0s 102us/step - loss: 6.6569 - val_loss: 9.1574
Epoch 229/1000
274/274 [==============================] - 0s 94us/step - loss: 6.7254 - val_loss: 9.1276
Epoch 230/1000
274/274 [==============================] - 0s 95us/step - loss: 6.7372 - val_loss: 9.0944
Epoch 231/1000
274/274 [==============================] - 0s 98us/step - loss: 6.6993 - val_loss: 8.9480
Epoch 232/1000
274/274 [==============================] - 0s 105us/step - loss: 6.7095 - val_loss: 9.1149
Epoch 233/1000
274/274 [==============================] - 0s 95us/step - loss: 6.7512 - val_loss: 9.2690
Epoch 234/1000
274/274 [==============================] - 0s 98us/s

274/274 [==============================] - 0s 92us/step - loss: 6.6248 - val_loss: 9.2368
Epoch 304/1000
274/274 [==============================] - 0s 95us/step - loss: 6.6978 - val_loss: 9.1925
Epoch 305/1000
274/274 [==============================] - 0s 107us/step - loss: 6.5751 - val_loss: 9.3224
Epoch 306/1000
274/274 [==============================] - 0s 98us/step - loss: 6.4459 - val_loss: 9.3508
Epoch 307/1000
274/274 [==============================] - 0s 95us/step - loss: 6.4068 - val_loss: 9.2028
Epoch 308/1000
274/274 [==============================] - 0s 102us/step - loss: 6.3692 - val_loss: 9.2702
Epoch 309/1000
274/274 [==============================] - 0s 97us/step - loss: 6.4630 - val_loss: 9.1697
Epoch 310/1000
274/274 [==============================] - 0s 98us/step - loss: 6.4913 - val_loss: 9.1385
Epoch 311/1000
274/274 [==============================] - 0s 97us/step - loss: 6.6002 - val_loss: 9.1209
Epoch 312/1000
274/274 [==============================] - 0s 95us/st

...

Epoch 692/1000
274/274 [==============================] - 0s 95us/step - loss: 5.4458 - val_loss: 8.9351
Epoch 693/1000
274/274 [==============================] - 0s 91us/step - loss: 5.5465 - val_loss: 9.0282
Epoch 694/1000
274/274 [==============================] - 0s 95us/step - loss: 5.5129 - val_loss: 8.9963
Epoch 695/1000
274/274 [==============================] - 0s 95us/step - loss: 5.6010 - val_loss: 9.1300
Epoch 696/1000
274/274 [==============================] - 0s 91us/step - loss: 5.4480 - val_loss: 8.8513
Epoch 697/1000
274/274 [==============================] - 0s 95us/step - loss: 5.4571 - val_loss: 8.9504
Epoch 698/1000
274/274 [==============================] - 0s 100us/step - loss: 5.6087 - val_loss: 9.1801
Epoch 699/1000
274/274 [==============================] - 0s 92us/step - loss: 5.8187 - val_loss: 9.0559
Epoch 700/1000
274/274 [==============================] - 0s 102us/step - loss: 5.9141 - val_loss: 9.2475
Epoch 701/1000
274/274 [=============================

Epoch 770/1000
274/274 [==============================] - 0s 102us/step - loss: 5.5333 - val_loss: 9.1072
Epoch 771/1000
274/274 [==============================] - 0s 91us/step - loss: 5.4599 - val_loss: 9.1859
Epoch 772/1000
274/274 [==============================] - 0s 98us/step - loss: 5.4860 - val_loss: 9.1622
Epoch 773/1000
274/274 [==============================] - 0s 98us/step - loss: 5.3770 - val_loss: 9.3215
Epoch 774/1000
274/274 [==============================] - 0s 101us/step - loss: 5.4228 - val_loss: 9.2790
Epoch 775/1000
274/274 [==============================] - 0s 101us/step - loss: 5.4105 - val_loss: 9.2726
Epoch 776/1000
274/274 [==============================] - 0s 98us/step - loss: 5.3991 - val_loss: 9.2457
Epoch 777/1000
274/274 [==============================] - 0s 93us/step - loss: 5.4965 - val_loss: 9.0602
Epoch 778/1000
274/274 [==============================] - 0s 94us/step - loss: 5.4414 - val_loss: 9.2328
Epoch 779/1000
274/274 [============================

Epoch 848/1000
274/274 [==============================] - 0s 95us/step - loss: 5.3654 - val_loss: 9.3777
Epoch 849/1000
274/274 [==============================] - 0s 106us/step - loss: 5.4176 - val_loss: 9.3345
Epoch 850/1000
274/274 [==============================] - 0s 113us/step - loss: 5.4050 - val_loss: 9.2642
Epoch 851/1000
274/274 [==============================] - 0s 106us/step - loss: 5.5295 - val_loss: 9.4127
Epoch 852/1000
274/274 [==============================] - 0s 124us/step - loss: 5.4043 - val_loss: 9.4440
Epoch 853/1000
274/274 [==============================] - 0s 96us/step - loss: 5.4335 - val_loss: 9.2308
Epoch 854/1000
274/274 [==============================] - 0s 98us/step - loss: 5.3505 - val_loss: 9.4313
Epoch 855/1000
274/274 [==============================] - 0s 95us/step - loss: 5.4410 - val_loss: 9.4292
Epoch 856/1000
274/274 [==============================] - 0s 95us/step - loss: 5.4517 - val_loss: 9.3366
Epoch 857/1000
274/274 [===========================

Epoch 926/1000
274/274 [==============================] - 0s 95us/step - loss: 5.4686 - val_loss: 9.5744
Epoch 927/1000
274/274 [==============================] - 0s 91us/step - loss: 5.4009 - val_loss: 9.3223
Epoch 928/1000
274/274 [==============================] - 0s 98us/step - loss: 5.3790 - val_loss: 9.3449
Epoch 929/1000
274/274 [==============================] - 0s 98us/step - loss: 5.4219 - val_loss: 9.3266
Epoch 930/1000
274/274 [==============================] - 0s 84us/step - loss: 5.3543 - val_loss: 9.6408
Epoch 931/1000
274/274 [==============================] - 0s 98us/step - loss: 5.4928 - val_loss: 9.7386
Epoch 932/1000
274/274 [==============================] - 0s 98us/step - loss: 5.4778 - val_loss: 9.5614
Epoch 933/1000
274/274 [==============================] - 0s 98us/step - loss: 5.5193 - val_loss: 9.1687
Epoch 934/1000
274/274 [==============================] - 0s 95us/step - loss: 5.4711 - val_loss: 9.2697
Epoch 935/1000
274/274 [==============================]

[array([[ 0.5274079 , -2.2242153 , -0.47249725,  3.0854359 ,  3.8636408 ],
        [-0.07253052, -1.5150435 , -0.23374613,  1.5209925 , -2.3716407 ],
        [-0.70851517,  0.2405941 ,  0.06291824, -0.63690674, -0.23099397],
        [-0.31885052,  1.9019178 ,  0.5942567 , -1.3293526 ,  3.2043407 ],
        [ 0.7839201 , -0.43210658,  0.35512224, -0.12916407, -0.09189379],
        [-0.49207363,  3.7632198 ,  0.10980529,  1.5938578 , -1.1129472 ],
        [-0.44705078,  2.754573  , -1.1258641 , -0.07482101, -0.253989  ]],
       dtype=float32),
 array([-0.39699763,  3.5524423 , -1.4585446 ,  1.8059983 ,  0.79468274],
       dtype=float32),
 array([[-0.7945633 ,  0.3808262 ,  0.9267694 ,  0.21312577, -0.4951309 ],
        [-0.7461719 ,  0.5085992 ,  0.8118609 , -0.06394607,  0.4326445 ],
        [ 1.0534099 , -1.0619574 , -1.3301113 , -0.21242328,  1.2452883 ],
        [-0.08860403,  1.3125558 ,  1.3770329 ,  0.53465635, -0.6791965 ],
        [ 0.9282504 , -0.9339215 , -0.9350389 , -0.575

In [39]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_structure1_3rd.h5')

In [40]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression_2(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_15 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_23 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_16 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 11        
Total params: 111
Trainable params: 111
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 872us/step - loss:

274/274 [==============================] - 0s 98us/step - loss: 6.7623 - val_loss: 9.3053
Epoch 70/1000
274/274 [==============================] - 0s 95us/step - loss: 6.8248 - val_loss: 9.4809
Epoch 71/1000
274/274 [==============================] - 0s 95us/step - loss: 6.7799 - val_loss: 9.4915
Epoch 72/1000
274/274 [==============================] - 0s 97us/step - loss: 6.6736 - val_loss: 9.4827
Epoch 73/1000
274/274 [==============================] - 0s 98us/step - loss: 6.6494 - val_loss: 9.5042
Epoch 74/1000
274/274 [==============================] - 0s 95us/step - loss: 6.8203 - val_loss: 9.6835
Epoch 75/1000
274/274 [==============================] - 0s 102us/step - loss: 6.8188 - val_loss: 9.3299
Epoch 76/1000
274/274 [==============================] - 0s 100us/step - loss: 6.7558 - val_loss: 9.3789
Epoch 77/1000
274/274 [==============================] - 0s 100us/step - loss: 6.6303 - val_loss: 9.3343
Epoch 78/1000
274/274 [==============================] - 0s 98us/step - los

274/274 [==============================] - 0s 102us/step - loss: 6.7727 - val_loss: 9.2705
Epoch 148/1000
274/274 [==============================] - 0s 109us/step - loss: 6.8044 - val_loss: 9.0973
Epoch 149/1000
274/274 [==============================] - 0s 107us/step - loss: 6.6389 - val_loss: 9.0863
Epoch 150/1000
274/274 [==============================] - 0s 105us/step - loss: 6.5772 - val_loss: 8.9084
Epoch 151/1000
274/274 [==============================] - 0s 113us/step - loss: 6.2875 - val_loss: 9.2659
Epoch 152/1000
274/274 [==============================] - 0s 108us/step - loss: 6.3923 - val_loss: 8.9364
Epoch 153/1000
274/274 [==============================] - 0s 106us/step - loss: 6.5347 - val_loss: 9.0187
Epoch 154/1000
274/274 [==============================] - 0s 103us/step - loss: 6.4413 - val_loss: 8.9490
Epoch 155/1000
274/274 [==============================] - 0s 113us/step - loss: 6.3774 - val_loss: 9.0237
Epoch 156/1000
274/274 [==============================] - 0s 

Epoch 225/1000
274/274 [==============================] - 0s 95us/step - loss: 6.1947 - val_loss: 9.1367
Epoch 226/1000
274/274 [==============================] - 0s 102us/step - loss: 6.2139 - val_loss: 9.1769
Epoch 227/1000
274/274 [==============================] - 0s 95us/step - loss: 6.3630 - val_loss: 9.1433
Epoch 228/1000
274/274 [==============================] - 0s 95us/step - loss: 6.3021 - val_loss: 9.1130
Epoch 229/1000
274/274 [==============================] - 0s 95us/step - loss: 6.3711 - val_loss: 9.0433
Epoch 230/1000
274/274 [==============================] - 0s 95us/step - loss: 6.3400 - val_loss: 9.2219
Epoch 231/1000
274/274 [==============================] - 0s 98us/step - loss: 6.2365 - val_loss: 9.2372
Epoch 232/1000
274/274 [==============================] - 0s 97us/step - loss: 6.2748 - val_loss: 9.1607
Epoch 233/1000
274/274 [==============================] - 0s 98us/step - loss: 6.3251 - val_loss: 9.2243
Epoch 234/1000
274/274 [==============================

Epoch 303/1000
274/274 [==============================] - 0s 102us/step - loss: 6.2998 - val_loss: 9.2036
Epoch 304/1000
274/274 [==============================] - 0s 95us/step - loss: 6.1480 - val_loss: 9.1781
Epoch 305/1000
274/274 [==============================] - 0s 98us/step - loss: 6.1479 - val_loss: 9.3106
Epoch 306/1000
274/274 [==============================] - 0s 102us/step - loss: 6.3110 - val_loss: 9.1321
Epoch 307/1000
274/274 [==============================] - 0s 93us/step - loss: 6.0461 - val_loss: 9.2809
Epoch 308/1000
274/274 [==============================] - 0s 98us/step - loss: 6.0834 - val_loss: 9.0649
Epoch 309/1000
274/274 [==============================] - 0s 95us/step - loss: 6.1353 - val_loss: 9.2520
Epoch 310/1000
274/274 [==============================] - 0s 95us/step - loss: 6.3027 - val_loss: 9.1970
Epoch 311/1000
274/274 [==============================] - 0s 95us/step - loss: 6.3126 - val_loss: 9.1541
Epoch 312/1000
274/274 [=============================

...

Epoch 692/1000
274/274 [==============================] - 0s 95us/step - loss: 5.8915 - val_loss: 9.4838
Epoch 693/1000
274/274 [==============================] - 0s 91us/step - loss: 6.2690 - val_loss: 9.3031
Epoch 694/1000
274/274 [==============================] - 0s 98us/step - loss: 5.9519 - val_loss: 9.3171
Epoch 695/1000
274/274 [==============================] - 0s 98us/step - loss: 5.8596 - val_loss: 9.1808
Epoch 696/1000
274/274 [==============================] - 0s 95us/step - loss: 5.8792 - val_loss: 9.2364
Epoch 697/1000
274/274 [==============================] - 0s 95us/step - loss: 5.8922 - val_loss: 9.2780
Epoch 698/1000
274/274 [==============================] - 0s 98us/step - loss: 6.1377 - val_loss: 9.2147
Epoch 699/1000
274/274 [==============================] - 0s 95us/step - loss: 5.8772 - val_loss: 9.2876
Epoch 700/1000
274/274 [==============================] - 0s 98us/step - loss: 5.9039 - val_loss: 9.1062
Epoch 701/1000
274/274 [==============================]

Epoch 770/1000
274/274 [==============================] - 0s 95us/step - loss: 5.8648 - val_loss: 9.3303
Epoch 771/1000
274/274 [==============================] - 0s 98us/step - loss: 5.7571 - val_loss: 9.4868
Epoch 772/1000
274/274 [==============================] - 0s 98us/step - loss: 5.8602 - val_loss: 9.1795
Epoch 773/1000
274/274 [==============================] - 0s 91us/step - loss: 5.8436 - val_loss: 9.3297
Epoch 774/1000
274/274 [==============================] - 0s 99us/step - loss: 5.8601 - val_loss: 9.1927
Epoch 775/1000
274/274 [==============================] - 0s 95us/step - loss: 5.7829 - val_loss: 9.4093
Epoch 776/1000
274/274 [==============================] - 0s 98us/step - loss: 5.8350 - val_loss: 9.3354
Epoch 777/1000
274/274 [==============================] - 0s 91us/step - loss: 5.8616 - val_loss: 9.3145
Epoch 778/1000
274/274 [==============================] - 0s 100us/step - loss: 5.9127 - val_loss: 9.2872
Epoch 779/1000
274/274 [==============================

Epoch 848/1000
274/274 [==============================] - 0s 102us/step - loss: 5.9425 - val_loss: 9.7161
Epoch 849/1000
274/274 [==============================] - 0s 93us/step - loss: 5.7775 - val_loss: 9.3503
Epoch 850/1000
274/274 [==============================] - 0s 95us/step - loss: 5.9409 - val_loss: 9.6020
Epoch 851/1000
274/274 [==============================] - 0s 95us/step - loss: 5.8317 - val_loss: 9.4316
Epoch 852/1000
274/274 [==============================] - 0s 93us/step - loss: 5.7880 - val_loss: 9.4736
Epoch 853/1000
274/274 [==============================] - 0s 98us/step - loss: 5.8896 - val_loss: 9.3967
Epoch 854/1000
274/274 [==============================] - 0s 95us/step - loss: 5.8096 - val_loss: 9.4385
Epoch 855/1000
274/274 [==============================] - 0s 95us/step - loss: 5.8670 - val_loss: 9.3929
Epoch 856/1000
274/274 [==============================] - 0s 101us/step - loss: 5.7611 - val_loss: 9.4465
Epoch 857/1000
274/274 [=============================

Epoch 926/1000
274/274 [==============================] - 0s 95us/step - loss: 5.8002 - val_loss: 9.3408
Epoch 927/1000
274/274 [==============================] - 0s 93us/step - loss: 5.9355 - val_loss: 9.6990
Epoch 928/1000
274/274 [==============================] - 0s 95us/step - loss: 5.7653 - val_loss: 9.5184
Epoch 929/1000
274/274 [==============================] - 0s 94us/step - loss: 5.7390 - val_loss: 9.5268
Epoch 930/1000
274/274 [==============================] - 0s 101us/step - loss: 5.8148 - val_loss: 9.6185
Epoch 931/1000
274/274 [==============================] - 0s 95us/step - loss: 5.7838 - val_loss: 9.4309
Epoch 932/1000
274/274 [==============================] - 0s 102us/step - loss: 5.8798 - val_loss: 9.6976
Epoch 933/1000
274/274 [==============================] - 0s 95us/step - loss: 5.9729 - val_loss: 9.4546
Epoch 934/1000
274/274 [==============================] - 0s 99us/step - loss: 5.8452 - val_loss: 9.5767
Epoch 935/1000
274/274 [=============================

[array([[-1.3467336e+00,  1.4211567e-01,  2.8313897e+00, -4.1423708e-01,
          2.5285000e-01],
        [-3.5401517e-01, -2.6235349e+00, -3.0402675e+00,  8.7620091e-01,
         -7.2671734e-02],
        [ 3.9375806e-01,  2.6986074e+00,  2.7425830e+00,  5.9640501e-05,
         -6.4852637e-01],
        [ 2.3905623e+00,  3.1529481e+00,  2.1616662e+00, -4.2550385e-01,
         -1.3310755e+00],
        [-3.9253780e-01, -9.6870571e-01,  5.9021419e-01,  6.3443786e-01,
         -1.0992393e+00],
        [-1.2700504e+00, -1.1725215e+00, -8.2102263e-01, -3.4282988e-01,
          9.0340483e-01],
        [-1.9713761e-01,  8.7818503e-01, -2.4003906e+00, -1.3872278e+00,
         -2.4778748e-01]], dtype=float32),
 array([ 2.3409479, -0.6539924,  1.0343283, -2.7042737,  3.1972566],
       dtype=float32),
 array([[-0.08755091, -0.5618288 ,  0.07350245,  0.8922274 , -0.46773246,
          0.57798326, -0.6071717 , -0.59325105, -0.8586033 ,  0.7576529 ],
        [ 0.21049668,  0.5123015 ,  0.7655772 ,  

In [41]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_structure2_3rd.h5')

In [42]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression_3(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_17 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_26 (Dense)             (None, 15)                90        
_________________________________________________________________
activation_18 (Activation)   (None, 15)                0         
_________________________________________________________________
dense_27 (Dense)             (None, 1)                 16        
Total params: 146
Trainable params: 146
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 920us/step - loss:

274/274 [==============================] - 0s 116us/step - loss: 7.5942 - val_loss: 10.7619
Epoch 69/1000
274/274 [==============================] - 0s 120us/step - loss: 7.5763 - val_loss: 11.4313
Epoch 70/1000
274/274 [==============================] - 0s 113us/step - loss: 7.7538 - val_loss: 11.2069
Epoch 71/1000
274/274 [==============================] - 0s 111us/step - loss: 7.9884 - val_loss: 11.6083
Epoch 72/1000
274/274 [==============================] - 0s 113us/step - loss: 8.5190 - val_loss: 11.4477
Epoch 73/1000
274/274 [==============================] - 0s 109us/step - loss: 7.9678 - val_loss: 11.4632
Epoch 74/1000
274/274 [==============================] - 0s 113us/step - loss: 7.5793 - val_loss: 11.1382
Epoch 75/1000
274/274 [==============================] - 0s 113us/step - loss: 7.0080 - val_loss: 11.2672
Epoch 76/1000
274/274 [==============================] - 0s 116us/step - loss: 7.1500 - val_loss: 10.8927
Epoch 77/1000
274/274 [==============================] - 0s 

274/274 [==============================] - 0s 102us/step - loss: 7.5344 - val_loss: 10.9598
Epoch 146/1000
274/274 [==============================] - 0s 101us/step - loss: 6.9177 - val_loss: 10.3767
Epoch 147/1000
274/274 [==============================] - 0s 97us/step - loss: 7.1095 - val_loss: 11.0250
Epoch 148/1000
274/274 [==============================] - 0s 98us/step - loss: 6.8196 - val_loss: 10.4080
Epoch 149/1000
274/274 [==============================] - 0s 91us/step - loss: 6.9003 - val_loss: 10.4945
Epoch 150/1000
274/274 [==============================] - 0s 98us/step - loss: 6.7272 - val_loss: 10.2194
Epoch 151/1000
274/274 [==============================] - 0s 97us/step - loss: 6.7745 - val_loss: 10.5570
Epoch 152/1000
274/274 [==============================] - 0s 94us/step - loss: 6.7420 - val_loss: 10.2252
Epoch 153/1000
274/274 [==============================] - 0s 97us/step - loss: 6.7657 - val_loss: 10.1094
Epoch 154/1000
274/274 [==============================] - 0

274/274 [==============================] - 0s 93us/step - loss: 6.3659 - val_loss: 8.9101
Epoch 223/1000
274/274 [==============================] - 0s 105us/step - loss: 6.5563 - val_loss: 8.7463
Epoch 224/1000
274/274 [==============================] - 0s 102us/step - loss: 6.5256 - val_loss: 8.8634
Epoch 225/1000
274/274 [==============================] - 0s 98us/step - loss: 6.5610 - val_loss: 8.8648
Epoch 226/1000
274/274 [==============================] - 0s 102us/step - loss: 7.0061 - val_loss: 9.3301
Epoch 227/1000
274/274 [==============================] - 0s 95us/step - loss: 6.8980 - val_loss: 8.7381
Epoch 228/1000
274/274 [==============================] - 0s 91us/step - loss: 6.6834 - val_loss: 8.8971
Epoch 229/1000
274/274 [==============================] - 0s 98us/step - loss: 6.7540 - val_loss: 8.8019
Epoch 230/1000
274/274 [==============================] - 0s 95us/step - loss: 6.4842 - val_loss: 8.7743
Epoch 231/1000
274/274 [==============================] - 0s 102us/

Epoch 300/1000
274/274 [==============================] - 0s 98us/step - loss: 6.2072 - val_loss: 7.8744
Epoch 301/1000
274/274 [==============================] - 0s 98us/step - loss: 6.1048 - val_loss: 8.0763
Epoch 302/1000
274/274 [==============================] - 0s 100us/step - loss: 6.2796 - val_loss: 8.0055
Epoch 303/1000
274/274 [==============================] - 0s 95us/step - loss: 6.0807 - val_loss: 7.8353
Epoch 304/1000
274/274 [==============================] - 0s 98us/step - loss: 6.1834 - val_loss: 7.9578
Epoch 305/1000
274/274 [==============================] - 0s 96us/step - loss: 6.3802 - val_loss: 8.3451
Epoch 306/1000
274/274 [==============================] - 0s 102us/step - loss: 6.0860 - val_loss: 7.8121
Epoch 307/1000
274/274 [==============================] - 0s 95us/step - loss: 6.1100 - val_loss: 8.0623
Epoch 308/1000
274/274 [==============================] - 0s 96us/step - loss: 6.2325 - val_loss: 7.7795
Epoch 309/1000
274/274 [=============================

...

Epoch 765/1000
274/274 [==============================] - 0s 102us/step - loss: 5.3275 - val_loss: 9.3015
Epoch 766/1000
274/274 [==============================] - 0s 98us/step - loss: 5.3195 - val_loss: 9.1878
Epoch 767/1000
274/274 [==============================] - 0s 99us/step - loss: 5.2662 - val_loss: 9.5126
Epoch 768/1000
274/274 [==============================] - 0s 98us/step - loss: 5.2031 - val_loss: 9.6267
Epoch 769/1000
274/274 [==============================] - 0s 93us/step - loss: 5.2043 - val_loss: 9.3937
Epoch 770/1000
274/274 [==============================] - 0s 95us/step - loss: 5.1557 - val_loss: 9.6063
Epoch 771/1000
274/274 [==============================] - 0s 98us/step - loss: 5.2253 - val_loss: 9.5117
Epoch 772/1000
274/274 [==============================] - 0s 93us/step - loss: 5.3595 - val_loss: 9.6661
Epoch 773/1000
274/274 [==============================] - 0s 97us/step - loss: 5.2755 - val_loss: 9.4968
Epoch 774/1000
274/274 [==============================

Epoch 843/1000
274/274 [==============================] - 0s 98us/step - loss: 5.4265 - val_loss: 9.5495
Epoch 844/1000
274/274 [==============================] - 0s 95us/step - loss: 5.1100 - val_loss: 9.9151
Epoch 845/1000
274/274 [==============================] - 0s 100us/step - loss: 5.4694 - val_loss: 9.9184
Epoch 846/1000
274/274 [==============================] - 0s 102us/step - loss: 5.2155 - val_loss: 10.0334
Epoch 847/1000
274/274 [==============================] - 0s 99us/step - loss: 5.2865 - val_loss: 9.5874
Epoch 848/1000
274/274 [==============================] - 0s 100us/step - loss: 5.2525 - val_loss: 10.0206
Epoch 849/1000
274/274 [==============================] - 0s 95us/step - loss: 5.2908 - val_loss: 9.7787
Epoch 850/1000
274/274 [==============================] - 0s 94us/step - loss: 5.2796 - val_loss: 9.3888
Epoch 851/1000
274/274 [==============================] - 0s 95us/step - loss: 6.0993 - val_loss: 9.8683
Epoch 852/1000
274/274 [==========================

274/274 [==============================] - 0s 98us/step - loss: 5.1467 - val_loss: 9.9451
Epoch 921/1000
274/274 [==============================] - 0s 103us/step - loss: 5.2085 - val_loss: 9.8813
Epoch 922/1000
274/274 [==============================] - 0s 95us/step - loss: 5.1522 - val_loss: 10.0259
Epoch 923/1000
274/274 [==============================] - 0s 98us/step - loss: 5.2232 - val_loss: 9.7123
Epoch 924/1000
274/274 [==============================] - 0s 98us/step - loss: 5.1684 - val_loss: 9.9124
Epoch 925/1000
274/274 [==============================] - 0s 97us/step - loss: 5.2442 - val_loss: 10.0356
Epoch 926/1000
274/274 [==============================] - 0s 94us/step - loss: 5.1019 - val_loss: 9.8395
Epoch 927/1000
274/274 [==============================] - 0s 96us/step - loss: 5.1978 - val_loss: 10.0681
Epoch 928/1000
274/274 [==============================] - 0s 101us/step - loss: 5.2487 - val_loss: 9.8328
Epoch 929/1000
274/274 [==============================] - 0s 103u

274/274 [==============================] - 0s 113us/step - loss: 5.1418 - val_loss: 9.9105
Epoch 998/1000
274/274 [==============================] - 0s 113us/step - loss: 5.1294 - val_loss: 9.9781
Epoch 999/1000
274/274 [==============================] - 0s 102us/step - loss: 5.1873 - val_loss: 10.3142
Epoch 1000/1000
274/274 [==============================] - 0s 95us/step - loss: 5.3992 - val_loss: 10.0916
10.884357492802506


[array([[ 1.7288758 , -0.06541843,  0.58632547, -1.3629136 ,  0.46422407],
        [ 2.3081396 ,  0.7881358 , -0.62912446, -1.2251635 ,  0.19705184],
        [-0.501008  , -0.4680923 , -0.9469238 ,  0.3474041 , -0.12078792],
        [-2.3030055 , -1.2931288 ,  0.1704122 ,  1.382648  ,  0.05134116],
        [ 0.12318213,  0.22270057,  0.3667293 , -0.13824373,  0.64969444],
        [-2.633094  ,  0.81835854,  0.06489894,  2.4211962 ,  0.61502564],
        [-0.82157815,  0.2703321 , -2.1007023 ,  1.9908931 , -2.7727017 ]],
       dtype=float32),
 array([-1.3600553 , -1.4318937 , -0.59196126,  2.2808192 , -1.9100335 ],
       dtype=float32),
 array([[-0.353239  ,  0.58053946, -0.7805671 , -0.58372176,  0.28919235,
          0.21245453, -0.07810632, -0.36836976,  0.7225775 , -0.20783734,
          0.25532198,  0.14420606,  0.7811201 , -0.7235585 , -0.18971212],
        [-0.2553936 ,  0.92786413, -0.40111747, -0.9749811 , -0.6000833 ,
          0.2306046 , -0.3255578 , -0.36380255,  0.455874

In [43]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_structure3_3rd.h5')

In [44]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression_4(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_28 (Dense)             (None, 10)                80        
_________________________________________________________________
activation_19 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_29 (Dense)             (None, 5)                 55        
_________________________________________________________________
activation_20 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_30 (Dense)             (None, 1)                 6         
Total params: 141
Trainable params: 141
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 976us/step - loss

274/274 [==============================] - 0s 106us/step - loss: 6.3066 - val_loss: 9.5577
Epoch 69/1000
274/274 [==============================] - 0s 113us/step - loss: 6.3241 - val_loss: 9.5614
Epoch 70/1000
274/274 [==============================] - 0s 117us/step - loss: 6.2047 - val_loss: 9.7732
Epoch 71/1000
274/274 [==============================] - 0s 119us/step - loss: 6.1666 - val_loss: 9.7533
Epoch 72/1000
274/274 [==============================] - 0s 106us/step - loss: 6.3051 - val_loss: 9.5290
Epoch 73/1000
274/274 [==============================] - 0s 108us/step - loss: 6.2375 - val_loss: 9.6345
Epoch 74/1000
274/274 [==============================] - 0s 109us/step - loss: 6.3248 - val_loss: 9.6517
Epoch 75/1000
274/274 [==============================] - 0s 109us/step - loss: 6.3773 - val_loss: 9.5608
Epoch 76/1000
274/274 [==============================] - 0s 106us/step - loss: 6.1532 - val_loss: 9.6632
Epoch 77/1000
274/274 [==============================] - 0s 109us/ste

Epoch 146/1000
274/274 [==============================] - 0s 109us/step - loss: 5.7645 - val_loss: 9.1362
Epoch 147/1000
274/274 [==============================] - 0s 98us/step - loss: 5.5174 - val_loss: 9.0472
Epoch 148/1000
274/274 [==============================] - 0s 109us/step - loss: 5.6037 - val_loss: 8.9477
Epoch 149/1000
274/274 [==============================] - 0s 102us/step - loss: 5.5002 - val_loss: 8.9693
Epoch 150/1000
274/274 [==============================] - 0s 102us/step - loss: 5.4589 - val_loss: 9.0145
Epoch 151/1000
274/274 [==============================] - 0s 113us/step - loss: 5.4829 - val_loss: 9.1011
Epoch 152/1000
274/274 [==============================] - 0s 106us/step - loss: 5.4194 - val_loss: 8.9208
Epoch 153/1000
274/274 [==============================] - 0s 106us/step - loss: 5.4746 - val_loss: 8.6929
Epoch 154/1000
274/274 [==============================] - 0s 109us/step - loss: 5.3967 - val_loss: 8.7569
Epoch 155/1000
274/274 [=======================

274/274 [==============================] - 0s 102us/step - loss: 5.5197 - val_loss: 8.9674
Epoch 224/1000
274/274 [==============================] - 0s 106us/step - loss: 5.3732 - val_loss: 8.7299
Epoch 225/1000
274/274 [==============================] - 0s 105us/step - loss: 5.1179 - val_loss: 8.8994
Epoch 226/1000
274/274 [==============================] - 0s 102us/step - loss: 5.1038 - val_loss: 8.6563
Epoch 227/1000
274/274 [==============================] - 0s 105us/step - loss: 5.1480 - val_loss: 8.4371
Epoch 228/1000
274/274 [==============================] - 0s 105us/step - loss: 5.2229 - val_loss: 8.7720
Epoch 229/1000
274/274 [==============================] - 0s 104us/step - loss: 5.1745 - val_loss: 8.9798
Epoch 230/1000
274/274 [==============================] - 0s 105us/step - loss: 5.1273 - val_loss: 8.9683
Epoch 231/1000
274/274 [==============================] - 0s 106us/step - loss: 5.1338 - val_loss: 8.9089
Epoch 232/1000
274/274 [==============================] - 0s 

274/274 [==============================] - 0s 100us/step - loss: 4.9262 - val_loss: 8.9350
Epoch 301/1000
274/274 [==============================] - 0s 109us/step - loss: 4.9267 - val_loss: 9.0072
Epoch 302/1000
274/274 [==============================] - 0s 105us/step - loss: 4.9683 - val_loss: 8.8173
Epoch 303/1000
274/274 [==============================] - 0s 109us/step - loss: 5.0436 - val_loss: 9.3240
Epoch 304/1000
274/274 [==============================] - 0s 98us/step - loss: 5.0148 - val_loss: 9.2214
Epoch 305/1000
274/274 [==============================] - 0s 106us/step - loss: 5.1040 - val_loss: 9.1447
Epoch 306/1000
274/274 [==============================] - 0s 106us/step - loss: 4.8886 - val_loss: 8.9561
Epoch 307/1000
274/274 [==============================] - 0s 106us/step - loss: 5.0117 - val_loss: 8.7861
Epoch 308/1000
274/274 [==============================] - 0s 113us/step - loss: 4.8013 - val_loss: 8.7728
Epoch 309/1000
274/274 [==============================] - 0s 1

...

274/274 [==============================] - 0s 102us/step - loss: 3.7634 - val_loss: 9.2704
Epoch 765/1000
274/274 [==============================] - 0s 98us/step - loss: 3.9218 - val_loss: 7.9985
Epoch 766/1000
274/274 [==============================] - 0s 102us/step - loss: 3.9476 - val_loss: 8.9532
Epoch 767/1000
274/274 [==============================] - 0s 98us/step - loss: 3.9368 - val_loss: 7.8677
Epoch 768/1000
274/274 [==============================] - 0s 98us/step - loss: 4.2048 - val_loss: 7.9644
Epoch 769/1000
274/274 [==============================] - 0s 95us/step - loss: 3.7288 - val_loss: 7.8416
Epoch 770/1000
274/274 [==============================] - 0s 109us/step - loss: 3.8496 - val_loss: 7.7621
Epoch 771/1000
274/274 [==============================] - 0s 108us/step - loss: 3.7575 - val_loss: 7.6249
Epoch 772/1000
274/274 [==============================] - 0s 105us/step - loss: 3.7294 - val_loss: 7.6702
Epoch 773/1000
274/274 [==============================] - 0s 97us

Epoch 842/1000
274/274 [==============================] - 0s 95us/step - loss: 3.7980 - val_loss: 8.9372
Epoch 843/1000
274/274 [==============================] - 0s 102us/step - loss: 3.7411 - val_loss: 7.8922
Epoch 844/1000
274/274 [==============================] - 0s 98us/step - loss: 3.6795 - val_loss: 7.7710
Epoch 845/1000
274/274 [==============================] - 0s 95us/step - loss: 3.7554 - val_loss: 8.3610
Epoch 846/1000
274/274 [==============================] - 0s 103us/step - loss: 3.8062 - val_loss: 7.6537
Epoch 847/1000
274/274 [==============================] - 0s 102us/step - loss: 3.7222 - val_loss: 7.8854
Epoch 848/1000
274/274 [==============================] - 0s 95us/step - loss: 3.6229 - val_loss: 8.1056
Epoch 849/1000
274/274 [==============================] - 0s 102us/step - loss: 3.5946 - val_loss: 8.1154
Epoch 850/1000
274/274 [==============================] - 0s 98us/step - loss: 3.7046 - val_loss: 8.0058
Epoch 851/1000
274/274 [===========================

Epoch 920/1000
274/274 [==============================] - 0s 98us/step - loss: 3.7986 - val_loss: 8.9511
Epoch 921/1000
274/274 [==============================] - 0s 95us/step - loss: 3.5439 - val_loss: 8.2173
Epoch 922/1000
274/274 [==============================] - 0s 101us/step - loss: 3.6646 - val_loss: 8.1742
Epoch 923/1000
274/274 [==============================] - 0s 108us/step - loss: 3.6738 - val_loss: 8.2624
Epoch 924/1000
274/274 [==============================] - 0s 107us/step - loss: 3.7038 - val_loss: 8.1359
Epoch 925/1000
274/274 [==============================] - 0s 100us/step - loss: 3.6261 - val_loss: 8.5989
Epoch 926/1000
274/274 [==============================] - 0s 98us/step - loss: 3.6339 - val_loss: 8.2093
Epoch 927/1000
274/274 [==============================] - 0s 98us/step - loss: 3.6564 - val_loss: 8.2387
Epoch 928/1000
274/274 [==============================] - 0s 98us/step - loss: 3.7492 - val_loss: 8.5649
Epoch 929/1000
274/274 [===========================

274/274 [==============================] - 0s 102us/step - loss: 3.6346 - val_loss: 8.0744
Epoch 998/1000
274/274 [==============================] - 0s 106us/step - loss: 3.6963 - val_loss: 8.7638
Epoch 999/1000
274/274 [==============================] - 0s 95us/step - loss: 3.5383 - val_loss: 8.2997
Epoch 1000/1000
274/274 [==============================] - 0s 97us/step - loss: 3.5244 - val_loss: 8.4996
7.216272038928533


[array([[-0.16243517,  1.122077  ,  2.8560452 ,  2.2493777 ,  3.5669627 ,
         -0.9180955 ,  1.944857  ,  6.6755676 ,  0.27070794, -2.0343685 ],
        [ 0.96825826, -0.6077919 , -1.507651  ,  1.7450968 ,  0.04354353,
          0.38012785,  1.1624945 , -4.379269  , -0.4663922 ,  0.3645556 ],
        [ 0.8729432 ,  0.40877116, -1.3155353 ,  1.4644109 , -0.06413081,
         -0.5828039 , -0.769657  ,  1.1289961 , -0.9244863 , -0.21816698],
        [-0.67934567, -0.7120239 ,  1.6429641 , -0.1205655 , -2.005322  ,
          0.32881853, -0.2139117 ,  1.8799223 ,  1.635175  , -0.39642552],
        [ 1.6915435 ,  0.07351439, -1.5879179 , -0.12003101,  0.93923396,
         -0.2546069 , -0.06680035, -0.35054806, -1.2160046 ,  0.50890523],
        [ 0.11358173,  0.22145705, -0.8307466 ,  1.364433  , -1.108322  ,
         -1.1261615 ,  1.205211  , -1.2765306 ,  0.73458976,  0.083188  ],
        [-3.311329  , -0.26861703, -0.59079087,  0.938843  , -1.0738344 ,
         -0.7473333 ,  0.0655772

In [45]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_structure4_3rd.h5')

In [46]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression_5(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_31 (Dense)             (None, 10)                80        
_________________________________________________________________
activation_21 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_32 (Dense)             (None, 10)                110       
_________________________________________________________________
activation_22 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_33 (Dense)             (None, 1)                 11        
Total params: 201
Trainable params: 201
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 1ms/step - loss: 

Epoch 69/1000
274/274 [==============================] - 0s 105us/step - loss: 6.7261 - val_loss: 9.3775
Epoch 70/1000
274/274 [==============================] - 0s 102us/step - loss: 6.6706 - val_loss: 9.0821
Epoch 71/1000
274/274 [==============================] - 0s 95us/step - loss: 6.6105 - val_loss: 9.0744
Epoch 72/1000
274/274 [==============================] - 0s 90us/step - loss: 6.7518 - val_loss: 9.3980
Epoch 73/1000
274/274 [==============================] - 0s 102us/step - loss: 6.7966 - val_loss: 9.1649
Epoch 74/1000
274/274 [==============================] - 0s 95us/step - loss: 6.5614 - val_loss: 8.9878
Epoch 75/1000
274/274 [==============================] - 0s 95us/step - loss: 6.6941 - val_loss: 8.9845
Epoch 76/1000
274/274 [==============================] - 0s 98us/step - loss: 6.6667 - val_loss: 8.8530
Epoch 77/1000
274/274 [==============================] - 0s 98us/step - loss: 6.4499 - val_loss: 8.7536
Epoch 78/1000
274/274 [==============================] - 0s 1

Epoch 147/1000
274/274 [==============================] - 0s 100us/step - loss: 5.3785 - val_loss: 8.1387
Epoch 148/1000
274/274 [==============================] - 0s 102us/step - loss: 5.4601 - val_loss: 8.2010
Epoch 149/1000
274/274 [==============================] - 0s 102us/step - loss: 5.4149 - val_loss: 8.4140
Epoch 150/1000
274/274 [==============================] - 0s 102us/step - loss: 5.3480 - val_loss: 8.5003
Epoch 151/1000
274/274 [==============================] - 0s 95us/step - loss: 5.3900 - val_loss: 8.2270
Epoch 152/1000
274/274 [==============================] - 0s 99us/step - loss: 5.5123 - val_loss: 9.0630
Epoch 153/1000
274/274 [==============================] - 0s 95us/step - loss: 5.8789 - val_loss: 8.6660
Epoch 154/1000
274/274 [==============================] - 0s 96us/step - loss: 5.2660 - val_loss: 8.6250
Epoch 155/1000
274/274 [==============================] - 0s 102us/step - loss: 5.3506 - val_loss: 9.0132
Epoch 156/1000
274/274 [==========================

274/274 [==============================] - 0s 106us/step - loss: 5.3994 - val_loss: 8.8123
Epoch 225/1000
274/274 [==============================] - 0s 109us/step - loss: 5.0164 - val_loss: 8.1881
Epoch 226/1000
274/274 [==============================] - 0s 109us/step - loss: 5.1583 - val_loss: 8.9111
Epoch 227/1000
274/274 [==============================] - 0s 92us/step - loss: 5.1448 - val_loss: 8.5475
Epoch 228/1000
274/274 [==============================] - 0s 102us/step - loss: 4.7347 - val_loss: 8.2130
Epoch 229/1000
274/274 [==============================] - 0s 98us/step - loss: 4.7107 - val_loss: 8.1165
Epoch 230/1000
274/274 [==============================] - 0s 98us/step - loss: 5.0241 - val_loss: 8.4559
Epoch 231/1000
274/274 [==============================] - 0s 106us/step - loss: 4.9504 - val_loss: 8.5212
Epoch 232/1000
274/274 [==============================] - 0s 109us/step - loss: 4.7705 - val_loss: 8.0930
Epoch 233/1000
274/274 [==============================] - 0s 106

274/274 [==============================] - 0s 95us/step - loss: 4.6948 - val_loss: 8.9102
Epoch 302/1000
274/274 [==============================] - 0s 102us/step - loss: 4.6788 - val_loss: 8.9206
Epoch 303/1000
274/274 [==============================] - 0s 98us/step - loss: 4.7244 - val_loss: 8.5637
Epoch 304/1000
274/274 [==============================] - 0s 95us/step - loss: 4.5673 - val_loss: 8.8271
Epoch 305/1000
274/274 [==============================] - 0s 98us/step - loss: 4.5649 - val_loss: 8.8732
Epoch 306/1000
274/274 [==============================] - 0s 102us/step - loss: 4.5912 - val_loss: 8.6641
Epoch 307/1000
274/274 [==============================] - 0s 98us/step - loss: 4.7403 - val_loss: 8.8573
Epoch 308/1000
274/274 [==============================] - 0s 99us/step - loss: 4.6959 - val_loss: 8.3546
Epoch 309/1000
274/274 [==============================] - 0s 93us/step - loss: 4.7697 - val_loss: 8.7031
Epoch 310/1000
274/274 [==============================] - 0s 91us/st

...

274/274 [==============================] - 0s 87us/step - loss: 4.1372 - val_loss: 8.0419
Epoch 766/1000
274/274 [==============================] - 0s 87us/step - loss: 4.0940 - val_loss: 8.6536
Epoch 767/1000
274/274 [==============================] - 0s 91us/step - loss: 4.1801 - val_loss: 8.5226
Epoch 768/1000
274/274 [==============================] - 0s 95us/step - loss: 4.0457 - val_loss: 8.1184
Epoch 769/1000
274/274 [==============================] - 0s 98us/step - loss: 4.1447 - val_loss: 8.3406
Epoch 770/1000
274/274 [==============================] - 0s 94us/step - loss: 3.9983 - val_loss: 8.3507
Epoch 771/1000
274/274 [==============================] - 0s 95us/step - loss: 4.1477 - val_loss: 7.9985
Epoch 772/1000
274/274 [==============================] - 0s 95us/step - loss: 4.1212 - val_loss: 8.5596
Epoch 773/1000
274/274 [==============================] - 0s 91us/step - loss: 4.0699 - val_loss: 8.1252
Epoch 774/1000
274/274 [==============================] - 0s 87us/step

Epoch 843/1000
274/274 [==============================] - 0s 106us/step - loss: 4.0020 - val_loss: 8.5264
Epoch 844/1000
274/274 [==============================] - 0s 102us/step - loss: 4.0110 - val_loss: 8.4901
Epoch 845/1000
274/274 [==============================] - 0s 91us/step - loss: 4.0723 - val_loss: 8.5220
Epoch 846/1000
274/274 [==============================] - 0s 91us/step - loss: 4.3416 - val_loss: 8.5804
Epoch 847/1000
274/274 [==============================] - 0s 93us/step - loss: 4.2707 - val_loss: 8.3353
Epoch 848/1000
274/274 [==============================] - 0s 98us/step - loss: 4.1021 - val_loss: 9.0567
Epoch 849/1000
274/274 [==============================] - 0s 93us/step - loss: 4.4242 - val_loss: 8.8382
Epoch 850/1000
274/274 [==============================] - 0s 91us/step - loss: 4.1694 - val_loss: 9.1622
Epoch 851/1000
274/274 [==============================] - 0s 87us/step - loss: 4.5532 - val_loss: 8.1965
Epoch 852/1000
274/274 [=============================

274/274 [==============================] - 0s 106us/step - loss: 4.2673 - val_loss: 7.9803
Epoch 921/1000
274/274 [==============================] - 0s 110us/step - loss: 3.9404 - val_loss: 8.1637
Epoch 922/1000
274/274 [==============================] - 0s 95us/step - loss: 4.2731 - val_loss: 9.1257
Epoch 923/1000
274/274 [==============================] - 0s 98us/step - loss: 4.2851 - val_loss: 8.6010
Epoch 924/1000
274/274 [==============================] - 0s 109us/step - loss: 3.9183 - val_loss: 8.6013
Epoch 925/1000
274/274 [==============================] - 0s 98us/step - loss: 3.9575 - val_loss: 7.9371
Epoch 926/1000
274/274 [==============================] - 0s 103us/step - loss: 3.8723 - val_loss: 8.1852
Epoch 927/1000
274/274 [==============================] - 0s 104us/step - loss: 3.9501 - val_loss: 8.1283
Epoch 928/1000
274/274 [==============================] - 0s 104us/step - loss: 3.9159 - val_loss: 8.6616
Epoch 929/1000
274/274 [==============================] - 0s 109

274/274 [==============================] - 0s 102us/step - loss: 3.9187 - val_loss: 8.5390
Epoch 998/1000
274/274 [==============================] - 0s 102us/step - loss: 4.1245 - val_loss: 8.4916
Epoch 999/1000
274/274 [==============================] - 0s 102us/step - loss: 3.9757 - val_loss: 8.0658
Epoch 1000/1000
274/274 [==============================] - 0s 95us/step - loss: 3.8720 - val_loss: 8.2767
7.058238966990325


[array([[ 5.39351046e-01,  2.52797574e-01, -2.91687250e-01,
          3.10475469e+00, -1.72136676e+00,  7.69018590e-01,
          4.07023478e+00, -1.13834202e+00, -2.09590507e+00,
          2.30160117e+00],
        [ 3.92183781e-01, -2.30618167e+00,  1.96762800e+00,
          5.75860977e-01,  7.63407528e-01,  1.36196065e+00,
         -2.31887794e+00,  1.99672186e+00, -6.12289727e-01,
          5.58820292e-02],
        [-6.54923737e-01,  5.40605299e-02,  6.69640124e-01,
         -1.34657049e+00,  7.50180930e-02, -2.54732847e-01,
          4.66212668e-02, -7.82372296e-01, -5.17966413e+00,
         -6.72307730e-01],
        [ 3.13650757e-01,  9.98466074e-01,  1.78901744e+00,
          8.60540792e-02,  2.94427752e-01, -8.86224806e-01,
          2.59038520e+00, -1.97178200e-01,  9.90088582e-01,
         -1.30355406e+00],
        [-2.90753692e-02, -8.51105809e-01,  2.22427726e-01,
         -2.09354952e-01,  1.27487222e-03, -7.97286332e-02,
         -4.42562878e-01,  9.56673846e-02, -1.488263

In [47]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_structure5_3rd.h5')

In [48]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression_6(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_34 (Dense)             (None, 10)                80        
_________________________________________________________________
activation_23 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_35 (Dense)             (None, 15)                165       
_________________________________________________________________
activation_24 (Activation)   (None, 15)                0         
_________________________________________________________________
dense_36 (Dense)             (None, 1)                 16        
Total params: 261
Trainable params: 261
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 1ms/step - loss: 

274/274 [==============================] - 0s 117us/step - loss: 6.0614 - val_loss: 10.0177
Epoch 69/1000
274/274 [==============================] - 0s 100us/step - loss: 6.0055 - val_loss: 10.0504
Epoch 70/1000
274/274 [==============================] - 0s 98us/step - loss: 5.9849 - val_loss: 9.8993
Epoch 71/1000
274/274 [==============================] - 0s 105us/step - loss: 6.1066 - val_loss: 10.1476
Epoch 72/1000
274/274 [==============================] - 0s 102us/step - loss: 5.9857 - val_loss: 9.5616
Epoch 73/1000
274/274 [==============================] - 0s 103us/step - loss: 5.9612 - val_loss: 9.9004
Epoch 74/1000
274/274 [==============================] - 0s 102us/step - loss: 6.2359 - val_loss: 10.4981
Epoch 75/1000
274/274 [==============================] - 0s 100us/step - loss: 6.3187 - val_loss: 9.8857
Epoch 76/1000
274/274 [==============================] - 0s 98us/step - loss: 6.0451 - val_loss: 9.8106
Epoch 77/1000
274/274 [==============================] - 0s 101us/s

Epoch 146/1000
274/274 [==============================] - 0s 100us/step - loss: 5.7270 - val_loss: 9.9374
Epoch 147/1000
274/274 [==============================] - 0s 98us/step - loss: 6.0053 - val_loss: 9.3762
Epoch 148/1000
274/274 [==============================] - 0s 105us/step - loss: 6.0162 - val_loss: 9.3331
Epoch 149/1000
274/274 [==============================] - 0s 101us/step - loss: 5.8221 - val_loss: 9.6563
Epoch 150/1000
274/274 [==============================] - 0s 102us/step - loss: 5.7050 - val_loss: 9.2745
Epoch 151/1000
274/274 [==============================] - 0s 108us/step - loss: 5.7007 - val_loss: 10.3419
Epoch 152/1000
274/274 [==============================] - 0s 105us/step - loss: 5.8426 - val_loss: 9.2636
Epoch 153/1000
274/274 [==============================] - 0s 104us/step - loss: 5.7712 - val_loss: 10.5358
Epoch 154/1000
274/274 [==============================] - 0s 105us/step - loss: 5.8837 - val_loss: 9.4324
Epoch 155/1000
274/274 [=====================

274/274 [==============================] - 0s 98us/step - loss: 5.4236 - val_loss: 9.5647
Epoch 224/1000
274/274 [==============================] - 0s 98us/step - loss: 5.3441 - val_loss: 9.7945
Epoch 225/1000
274/274 [==============================] - 0s 102us/step - loss: 5.4392 - val_loss: 9.7096
Epoch 226/1000
274/274 [==============================] - 0s 98us/step - loss: 5.4102 - val_loss: 9.4880
Epoch 227/1000
274/274 [==============================] - 0s 97us/step - loss: 5.6271 - val_loss: 9.3974
Epoch 228/1000
274/274 [==============================] - 0s 99us/step - loss: 5.5378 - val_loss: 9.6647
Epoch 229/1000
274/274 [==============================] - 0s 101us/step - loss: 5.3846 - val_loss: 9.2341
Epoch 230/1000
274/274 [==============================] - 0s 98us/step - loss: 5.4057 - val_loss: 9.2605
Epoch 231/1000
274/274 [==============================] - 0s 104us/step - loss: 5.3046 - val_loss: 9.3541
Epoch 232/1000
274/274 [==============================] - 0s 103us/

274/274 [==============================] - 0s 101us/step - loss: 5.4240 - val_loss: 11.2457
Epoch 301/1000
274/274 [==============================] - 0s 99us/step - loss: 5.5873 - val_loss: 10.0238
Epoch 302/1000
274/274 [==============================] - 0s 95us/step - loss: 5.2687 - val_loss: 10.1603
Epoch 303/1000
274/274 [==============================] - 0s 100us/step - loss: 5.2246 - val_loss: 10.2935
Epoch 304/1000
274/274 [==============================] - 0s 98us/step - loss: 5.1661 - val_loss: 10.3696
Epoch 305/1000
274/274 [==============================] - 0s 127us/step - loss: 5.3598 - val_loss: 9.4998
Epoch 306/1000
274/274 [==============================] - 0s 109us/step - loss: 5.3125 - val_loss: 10.4662
Epoch 307/1000
274/274 [==============================] - 0s 113us/step - loss: 5.2331 - val_loss: 10.3752
Epoch 308/1000
274/274 [==============================] - 0s 113us/step - loss: 5.2912 - val_loss: 9.8928
Epoch 309/1000
274/274 [==============================] -

...

274/274 [==============================] - 0s 97us/step - loss: 4.1379 - val_loss: 11.4827
Epoch 763/1000
274/274 [==============================] - 0s 98us/step - loss: 4.3054 - val_loss: 12.7259
Epoch 764/1000
274/274 [==============================] - 0s 95us/step - loss: 4.1753 - val_loss: 11.8622
Epoch 765/1000
274/274 [==============================] - 0s 98us/step - loss: 4.2006 - val_loss: 12.1380
Epoch 766/1000
274/274 [==============================] - 0s 91us/step - loss: 4.2076 - val_loss: 11.8281
Epoch 767/1000
274/274 [==============================] - 0s 98us/step - loss: 4.2430 - val_loss: 11.5081
Epoch 768/1000
274/274 [==============================] - 0s 109us/step - loss: 4.2690 - val_loss: 11.7008
Epoch 769/1000
274/274 [==============================] - 0s 118us/step - loss: 4.1264 - val_loss: 12.0053
Epoch 770/1000
274/274 [==============================] - 0s 106us/step - loss: 4.1912 - val_loss: 11.7505
Epoch 771/1000
274/274 [==============================] - 

274/274 [==============================] - 0s 95us/step - loss: 4.1109 - val_loss: 11.7831
Epoch 840/1000
274/274 [==============================] - 0s 91us/step - loss: 4.2002 - val_loss: 11.5837
Epoch 841/1000
274/274 [==============================] - 0s 98us/step - loss: 4.0970 - val_loss: 11.5190
Epoch 842/1000
274/274 [==============================] - 0s 113us/step - loss: 4.4047 - val_loss: 13.0955
Epoch 843/1000
274/274 [==============================] - 0s 117us/step - loss: 4.2813 - val_loss: 11.6730
Epoch 844/1000
274/274 [==============================] - 0s 98us/step - loss: 4.1817 - val_loss: 11.9881
Epoch 845/1000
274/274 [==============================] - 0s 99us/step - loss: 4.6520 - val_loss: 13.2185
Epoch 846/1000
274/274 [==============================] - 0s 98us/step - loss: 4.3525 - val_loss: 11.5369
Epoch 847/1000
274/274 [==============================] - 0s 95us/step - loss: 4.5100 - val_loss: 11.7346
Epoch 848/1000
274/274 [==============================] - 0

274/274 [==============================] - 0s 87us/step - loss: 4.1791 - val_loss: 11.4413
Epoch 917/1000
274/274 [==============================] - 0s 93us/step - loss: 4.0698 - val_loss: 11.8726
Epoch 918/1000
274/274 [==============================] - 0s 91us/step - loss: 3.9965 - val_loss: 11.6507
Epoch 919/1000
274/274 [==============================] - 0s 95us/step - loss: 3.9069 - val_loss: 11.7773
Epoch 920/1000
274/274 [==============================] - 0s 108us/step - loss: 4.0943 - val_loss: 11.4035
Epoch 921/1000
274/274 [==============================] - 0s 96us/step - loss: 3.9727 - val_loss: 12.2578
Epoch 922/1000
274/274 [==============================] - 0s 91us/step - loss: 4.1656 - val_loss: 11.5200
Epoch 923/1000
274/274 [==============================] - 0s 95us/step - loss: 4.1133 - val_loss: 11.7195
Epoch 924/1000
274/274 [==============================] - 0s 102us/step - loss: 4.1127 - val_loss: 11.7376
Epoch 925/1000
274/274 [==============================] - 0

274/274 [==============================] - 0s 98us/step - loss: 3.9590 - val_loss: 11.3241
Epoch 994/1000
274/274 [==============================] - 0s 98us/step - loss: 4.0375 - val_loss: 11.9289
Epoch 995/1000
274/274 [==============================] - 0s 98us/step - loss: 3.8992 - val_loss: 11.7488
Epoch 996/1000
274/274 [==============================] - 0s 95us/step - loss: 3.9134 - val_loss: 12.0342
Epoch 997/1000
274/274 [==============================] - 0s 95us/step - loss: 3.8932 - val_loss: 11.2928
Epoch 998/1000
274/274 [==============================] - 0s 95us/step - loss: 3.9679 - val_loss: 11.3641
Epoch 999/1000
274/274 [==============================] - 0s 96us/step - loss: 4.0147 - val_loss: 11.5107
Epoch 1000/1000
274/274 [==============================] - 0s 98us/step - loss: 3.9446 - val_loss: 11.5352
7.679749149387166


[array([[-0.0727436 ,  0.56470597, -2.813895  ,  1.1301001 ,  0.75247073,
          2.5609727 , -1.1813607 , -1.4873018 ,  0.27920425,  1.8011307 ],
        [-0.09418276, -0.8559169 ,  2.0949564 ,  0.93121403,  0.46039432,
         -0.8900191 ,  1.0637549 , -1.1108001 ,  0.3112595 , -0.21977438],
        [ 0.0231803 , -2.0874476 ,  0.4391476 ,  2.8372698 , -0.32009083,
          0.129061  ,  0.28859258,  0.05034657,  0.6787449 , -0.32913455],
        [-0.44810745, -0.42586723, -2.250143  , -0.8543707 ,  0.51707757,
          0.00759366, -1.270443  ,  1.7206628 , -0.53953373, -0.77434003],
        [-0.67430294, -0.26849124,  1.354062  ,  0.9010869 , -0.01822092,
         -0.26219097,  0.27647364, -0.24194579,  0.90190095, -0.24712461],
        [-0.6872266 , -0.8108298 ,  1.5044708 ,  0.64178187, -1.4786613 ,
         -0.6757957 ,  1.7345508 ,  2.5526524 , -0.15979283,  0.3661914 ],
        [ 2.8715522 , -1.5260693 , -0.5081732 , -1.2227898 ,  0.36869976,
          1.0694298 , -0.0647867

In [49]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_structure6_3rd.h5')

In [50]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression_1(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_37 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_25 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_38 (Dense)             (None, 5)                 30        
_________________________________________________________________
activation_26 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_39 (Dense)             (None, 1)                 6         
Total params: 151
Trainable params: 151
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 2ms/step - loss: 

143/143 [==============================] - 0s 116us/step - loss: 0.0059 - val_loss: 0.0047
Epoch 70/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0063 - val_loss: 0.0041
Epoch 71/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0067 - val_loss: 0.0043
Epoch 72/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0061 - val_loss: 0.0041
Epoch 73/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0056 - val_loss: 0.0048
Epoch 74/1000
143/143 [==============================] - 0s 115us/step - loss: 0.0062 - val_loss: 0.0042
Epoch 75/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0056 - val_loss: 0.0036
Epoch 76/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0056 - val_loss: 0.0050
Epoch 77/1000
143/143 [==============================] - 0s 146us/step - loss: 0.0070 - val_loss: 0.0053
Epoch 78/1000
143/143 [==============================] - ETA: 0s - lo

143/143 [==============================] - 0s 119us/step - loss: 0.0033 - val_loss: 0.0030
Epoch 147/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0034 - val_loss: 0.0032
Epoch 148/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0037 - val_loss: 0.0030
Epoch 149/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0040 - val_loss: 0.0033
Epoch 150/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0035 - val_loss: 0.0038
Epoch 151/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0034 - val_loss: 0.0032
Epoch 152/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0032 - val_loss: 0.0031
Epoch 153/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0033 - val_loss: 0.0050
Epoch 154/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0042 - val_loss: 0.0032
Epoch 155/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 119us/step - loss: 0.0029 - val_loss: 0.0034
Epoch 224/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0030 - val_loss: 0.0031
Epoch 225/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0035 - val_loss: 0.0041
Epoch 226/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0031 - val_loss: 0.0032
Epoch 227/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0028 - val_loss: 0.0034
Epoch 228/1000
143/143 [==============================] - 0s 115us/step - loss: 0.0029 - val_loss: 0.0034
Epoch 229/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0029 - val_loss: 0.0031
Epoch 230/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0031 - val_loss: 0.0033
Epoch 231/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0030 - val_loss: 0.0044
Epoch 232/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 119us/step - loss: 0.0027 - val_loss: 0.0033
Epoch 301/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0025 - val_loss: 0.0039
Epoch 302/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0026 - val_loss: 0.0033
Epoch 303/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0027 - val_loss: 0.0041
Epoch 304/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0030 - val_loss: 0.0035
Epoch 305/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0027 - val_loss: 0.0033
Epoch 306/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0027 - val_loss: 0.0035
Epoch 307/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0027 - val_loss: 0.0042
Epoch 308/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0027 - val_loss: 0.0034
Epoch 309/1000
143/143 [==============================] - 0s 

...

143/143 [==============================] - 0s 126us/step - loss: 0.0017 - val_loss: 0.0042
Epoch 763/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0019 - val_loss: 0.0039
Epoch 764/1000
143/143 [==============================] - 0s 129us/step - loss: 0.0018 - val_loss: 0.0039
Epoch 765/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0017 - val_loss: 0.0041
Epoch 766/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0017 - val_loss: 0.0043
Epoch 767/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0017 - val_loss: 0.0043
Epoch 768/1000
143/143 [==============================] - 0s 125us/step - loss: 0.0018 - val_loss: 0.0042
Epoch 769/1000
143/143 [==============================] - 0s 120us/step - loss: 0.0019 - val_loss: 0.0049
Epoch 770/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0022 - val_loss: 0.0043
Epoch 771/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 98us/step - loss: 0.0025 - val_loss: 0.0043
Epoch 840/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0023 - val_loss: 0.0055
Epoch 841/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0025 - val_loss: 0.0046
Epoch 842/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0024 - val_loss: 0.0038
Epoch 843/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0018 - val_loss: 0.0041
Epoch 844/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0018 - val_loss: 0.0037
Epoch 845/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0017 - val_loss: 0.0041
Epoch 846/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0018 - val_loss: 0.0039
Epoch 847/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0020 - val_loss: 0.0052
Epoch 848/1000
143/143 [==============================] - 0s 99us/ste

143/143 [==============================] - 0s 105us/step - loss: 0.0020 - val_loss: 0.0040
Epoch 917/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0017 - val_loss: 0.0046
Epoch 918/1000
143/143 [==============================] - 0s 106us/step - loss: 0.0017 - val_loss: 0.0046
Epoch 919/1000
143/143 [==============================] - ETA: 0s - loss: 0.002 - 0s 112us/step - loss: 0.0016 - val_loss: 0.0048
Epoch 920/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0016 - val_loss: 0.0045
Epoch 921/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0018 - val_loss: 0.0052
Epoch 922/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0022 - val_loss: 0.0042
Epoch 923/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0019 - val_loss: 0.0043
Epoch 924/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0019 - val_loss: 0.0052
Epoch 925/1000
143/143 [=============

143/143 [==============================] - 0s 111us/step - loss: 0.0018 - val_loss: 0.0057
Epoch 994/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0018 - val_loss: 0.0041
Epoch 995/1000
143/143 [==============================] - 0s 106us/step - loss: 0.0014 - val_loss: 0.0044
Epoch 996/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0014 - val_loss: 0.0045
Epoch 997/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0014 - val_loss: 0.0045
Epoch 998/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0014 - val_loss: 0.0043
Epoch 999/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0014 - val_loss: 0.0047
Epoch 1000/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0014 - val_loss: 0.0046
0.010574804618954659


[array([[-0.42605963, -0.07777546,  0.01991159, -0.25458196, -0.20944428],
        [ 0.01476798,  0.27787945,  0.6385119 , -0.30709037,  0.40728468],
        [ 0.6436845 , -0.10575218, -0.24513009,  0.43868735,  0.36013085],
        [-0.7721344 , -0.91658956, -0.07400794,  1.1030083 , -0.6017021 ],
        [ 0.11562541,  0.5563142 ,  1.07536   , -0.07066115,  0.6955638 ],
        [ 0.54512054,  0.8538673 ,  0.48709026, -1.3089812 ,  0.7320952 ],
        [ 0.962849  , -0.04362402, -0.2506559 ,  0.23799028,  0.52421737],
        [ 0.30450827, -0.46097523,  0.13852961, -0.16888665, -0.20524688],
        [ 0.8619594 , -0.305783  ,  0.03242037, -0.3511745 ,  0.72221786],
        [-2.0347128 , -0.33124498,  0.2243542 ,  0.39747217, -0.80346715],
        [ 0.7102007 , -0.34486458, -0.42145637, -0.2874849 ,  0.48127884],
        [-0.81777394,  0.39722285, -0.95864487,  0.4399041 , -0.91013914],
        [-0.24695365,  0.7478601 ,  0.00490227, -0.6609163 , -0.96770114],
        [-1.0467479 , -1.

In [51]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_structure1_3rd.h5')

In [52]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression_2(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_40 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_27 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_41 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_28 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_42 (Dense)             (None, 1)                 11        
Total params: 186
Trainable params: 186
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 2ms/step - loss: 

143/143 [==============================] - 0s 119us/step - loss: 0.0061 - val_loss: 0.0047
Epoch 70/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0060 - val_loss: 0.0048
Epoch 71/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0060 - val_loss: 0.0048
Epoch 72/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0059 - val_loss: 0.0046
Epoch 73/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0058 - val_loss: 0.0047
Epoch 74/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0059 - val_loss: 0.0043
Epoch 75/1000
143/143 [==============================] - 0s 117us/step - loss: 0.0058 - val_loss: 0.0046
Epoch 76/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0060 - val_loss: 0.0048
Epoch 77/1000
143/143 [==============================] - 0s 124us/step - loss: 0.0059 - val_loss: 0.0044
Epoch 78/1000
143/143 [==============================] - 0s 115us/ste

143/143 [==============================] - 0s 105us/step - loss: 0.0050 - val_loss: 0.0040
Epoch 147/1000
143/143 [==============================] - 0s 123us/step - loss: 0.0051 - val_loss: 0.0046
Epoch 148/1000
143/143 [==============================] - 0s 125us/step - loss: 0.0052 - val_loss: 0.0040
Epoch 149/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0051 - val_loss: 0.0041
Epoch 150/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0049 - val_loss: 0.0041
Epoch 151/1000
143/143 [==============================] - 0s 110us/step - loss: 0.0050 - val_loss: 0.0040
Epoch 152/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0049 - val_loss: 0.0042
Epoch 153/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0051 - val_loss: 0.0042
Epoch 154/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0050 - val_loss: 0.0043
Epoch 155/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 123us/step - loss: 0.0049 - val_loss: 0.0038
Epoch 224/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0046 - val_loss: 0.0040
Epoch 225/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0045 - val_loss: 0.0037
Epoch 226/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0048 - val_loss: 0.0040
Epoch 227/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0046 - val_loss: 0.0035
Epoch 228/1000
143/143 [==============================] - 0s 120us/step - loss: 0.0047 - val_loss: 0.0040
Epoch 229/1000
143/143 [==============================] - 0s 124us/step - loss: 0.0044 - val_loss: 0.0036
Epoch 230/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0044 - val_loss: 0.0038
Epoch 231/1000
143/143 [==============================] - 0s 110us/step - loss: 0.0045 - val_loss: 0.0036
Epoch 232/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 116us/step - loss: 0.0043 - val_loss: 0.0036
Epoch 301/1000
143/143 [==============================] - 0s 121us/step - loss: 0.0044 - val_loss: 0.0036
Epoch 302/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0043 - val_loss: 0.0033
Epoch 303/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0043 - val_loss: 0.0039
Epoch 304/1000
143/143 [==============================] - 0s 108us/step - loss: 0.0044 - val_loss: 0.0035
Epoch 305/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0042 - val_loss: 0.0039
Epoch 306/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0043 - val_loss: 0.0036
Epoch 307/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0044 - val_loss: 0.0036
Epoch 308/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0044 - val_loss: 0.0038
Epoch 309/1000
143/143 [==============================] - 0s 

...

143/143 [==============================] - 0s 112us/step - loss: 0.0025 - val_loss: 0.0046
Epoch 763/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0025 - val_loss: 0.0032
Epoch 764/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0024 - val_loss: 0.0034
Epoch 765/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0024 - val_loss: 0.0034
Epoch 766/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0025 - val_loss: 0.0045
Epoch 767/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0025 - val_loss: 0.0033
Epoch 768/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0027 - val_loss: 0.0040
Epoch 769/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0027 - val_loss: 0.0051
Epoch 770/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0027 - val_loss: 0.0034
Epoch 771/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0022 - val_loss: 0.0039
Epoch 840/1000
143/143 [==============================] - 0s 108us/step - loss: 0.0023 - val_loss: 0.0053
Epoch 841/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0025 - val_loss: 0.0037
Epoch 842/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0024 - val_loss: 0.0034
Epoch 843/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0024 - val_loss: 0.0047
Epoch 844/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0022 - val_loss: 0.0038
Epoch 845/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0023 - val_loss: 0.0040
Epoch 846/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0022 - val_loss: 0.0044
Epoch 847/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0022 - val_loss: 0.0041
Epoch 848/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 105us/step - loss: 0.0029 - val_loss: 0.0076
Epoch 917/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0027 - val_loss: 0.0037
Epoch 918/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0027 - val_loss: 0.0050
Epoch 919/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0023 - val_loss: 0.0036
Epoch 920/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0025 - val_loss: 0.0035
Epoch 921/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0023 - val_loss: 0.0039
Epoch 922/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0023 - val_loss: 0.0044
Epoch 923/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0023 - val_loss: 0.0037
Epoch 924/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0025 - val_loss: 0.0035
Epoch 925/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0022 - val_loss: 0.0042
Epoch 994/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0022 - val_loss: 0.0040
Epoch 995/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0025 - val_loss: 0.0051
Epoch 996/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0022 - val_loss: 0.0038
Epoch 997/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0030 - val_loss: 0.0083
Epoch 998/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0031 - val_loss: 0.0035
Epoch 999/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0025 - val_loss: 0.0037
Epoch 1000/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0023 - val_loss: 0.0046
0.015278239734470844


[array([[ 1.01248734e-01, -3.90587375e-02,  4.54134762e-01,
         -3.48079592e-01,  8.65140617e-01],
        [-2.88793981e-01, -4.50984865e-01, -9.66663137e-02,
         -7.36611784e-01, -2.06296936e-01],
        [ 9.03432891e-02,  1.50342092e-01, -5.56128323e-01,
         -3.42928916e-01, -8.30271006e-01],
        [-8.82532597e-01,  1.31845906e-01,  4.36266780e-01,
         -6.63835704e-01, -3.46058130e-01],
        [-1.23831853e-01,  1.83544278e-01,  1.67308394e-02,
         -3.51047546e-01, -5.09675264e-01],
        [-3.57962638e-01,  2.95447975e-01,  3.86118084e-01,
         -1.11707095e-02,  7.00566649e-01],
        [-3.08032006e-01,  2.08132327e-01,  2.97880560e-01,
         -2.94365995e-02, -3.27254891e-01],
        [ 2.77151644e-01, -4.41821367e-01, -1.91900060e-02,
          1.07123882e-01,  3.82201165e-01],
        [ 7.87151873e-01, -3.59174401e-01, -6.70557082e-01,
         -2.30177402e-01,  1.27651647e-01],
        [-5.31766057e-01, -2.65929312e-01,  2.35073850e-01,
    

In [53]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_structure2_3rd.h5')

In [54]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression_3(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_43 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_29 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_44 (Dense)             (None, 15)                90        
_________________________________________________________________
activation_30 (Activation)   (None, 15)                0         
_________________________________________________________________
dense_45 (Dense)             (None, 1)                 16        
Total params: 221
Trainable params: 221
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 2ms/step - loss: 

143/143 [==============================] - 0s 112us/step - loss: 0.0057 - val_loss: 0.0045
Epoch 70/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0053 - val_loss: 0.0048
Epoch 71/1000
143/143 [==============================] - 0s 113us/step - loss: 0.0055 - val_loss: 0.0052
Epoch 72/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0052 - val_loss: 0.0043
Epoch 73/1000
143/143 [==============================] - 0s 115us/step - loss: 0.0059 - val_loss: 0.0060
Epoch 74/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0061 - val_loss: 0.0046
Epoch 75/1000
143/143 [==============================] - 0s 113us/step - loss: 0.0055 - val_loss: 0.0044
Epoch 76/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0055 - val_loss: 0.0069
Epoch 77/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0062 - val_loss: 0.0042
Epoch 78/1000
143/143 [==============================] - 0s 117us/ste

143/143 [==============================] - 0s 112us/step - loss: 0.0047 - val_loss: 0.0048
Epoch 147/1000
143/143 [==============================] - 0s 110us/step - loss: 0.0039 - val_loss: 0.0039
Epoch 148/1000
143/143 [==============================] - 0s 123us/step - loss: 0.0036 - val_loss: 0.0034
Epoch 149/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0034 - val_loss: 0.0036
Epoch 150/1000
143/143 [==============================] - 0s 123us/step - loss: 0.0035 - val_loss: 0.0036
Epoch 151/1000
143/143 [==============================] - 0s 115us/step - loss: 0.0034 - val_loss: 0.0039
Epoch 152/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0037 - val_loss: 0.0041
Epoch 153/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0046 - val_loss: 0.0062
Epoch 154/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0039 - val_loss: 0.0037
Epoch 155/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 119us/step - loss: 0.0028 - val_loss: 0.0037
Epoch 224/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0026 - val_loss: 0.0035
Epoch 225/1000
143/143 [==============================] - 0s 120us/step - loss: 0.0027 - val_loss: 0.0044
Epoch 226/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0031 - val_loss: 0.0039
Epoch 227/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0031 - val_loss: 0.0034
Epoch 228/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0030 - val_loss: 0.0035
Epoch 229/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0028 - val_loss: 0.0037
Epoch 230/1000
143/143 [==============================] - 0s 121us/step - loss: 0.0027 - val_loss: 0.0039
Epoch 231/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0027 - val_loss: 0.0035
Epoch 232/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 126us/step - loss: 0.0024 - val_loss: 0.0041
Epoch 301/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0024 - val_loss: 0.0039
Epoch 302/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0023 - val_loss: 0.0037
Epoch 303/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0024 - val_loss: 0.0037
Epoch 304/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0025 - val_loss: 0.0058
Epoch 305/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0032 - val_loss: 0.0036
Epoch 306/1000
143/143 [==============================] - 0s 132us/step - loss: 0.0025 - val_loss: 0.0036
Epoch 307/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0024 - val_loss: 0.0042
Epoch 308/1000
143/143 [==============================] - 0s 125us/step - loss: 0.0023 - val_loss: 0.0038
Epoch 309/1000
143/143 [==============================] - 0s 

...

143/143 [==============================] - 0s 126us/step - loss: 0.0016 - val_loss: 0.0044
Epoch 763/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0016 - val_loss: 0.0043
Epoch 764/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0016 - val_loss: 0.0045
Epoch 765/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0016 - val_loss: 0.0041
Epoch 766/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0015 - val_loss: 0.0044
Epoch 767/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0016 - val_loss: 0.0043
Epoch 768/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0018 - val_loss: 0.0041
Epoch 769/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0016 - val_loss: 0.0045
Epoch 770/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0016 - val_loss: 0.0042
Epoch 771/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 111us/step - loss: 0.0022 - val_loss: 0.0040
Epoch 840/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0021 - val_loss: 0.0038
Epoch 841/1000
143/143 [==============================] - 0s 115us/step - loss: 0.0018 - val_loss: 0.0046
Epoch 842/1000
143/143 [==============================] - 0s 121us/step - loss: 0.0017 - val_loss: 0.0037
Epoch 843/1000
143/143 [==============================] - 0s 123us/step - loss: 0.0017 - val_loss: 0.0043
Epoch 844/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0018 - val_loss: 0.0044
Epoch 845/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0018 - val_loss: 0.0042
Epoch 846/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0019 - val_loss: 0.0040
Epoch 847/1000
143/143 [==============================] - 0s 117us/step - loss: 0.0016 - val_loss: 0.0042
Epoch 848/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 118us/step - loss: 0.0017 - val_loss: 0.0043
Epoch 917/1000
143/143 [==============================] - 0s 110us/step - loss: 0.0014 - val_loss: 0.0040
Epoch 918/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0014 - val_loss: 0.0042
Epoch 919/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0015 - val_loss: 0.0045
Epoch 920/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0013 - val_loss: 0.0040
Epoch 921/1000
143/143 [==============================] - 0s 115us/step - loss: 0.0015 - val_loss: 0.0043
Epoch 922/1000
143/143 [==============================] - 0s 117us/step - loss: 0.0017 - val_loss: 0.0043
Epoch 923/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0015 - val_loss: 0.0041
Epoch 924/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0016 - val_loss: 0.0052
Epoch 925/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 119us/step - loss: 0.0014 - val_loss: 0.0048
Epoch 994/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0013 - val_loss: 0.0041
Epoch 995/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0013 - val_loss: 0.0042
Epoch 996/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0013 - val_loss: 0.0040
Epoch 997/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0014 - val_loss: 0.0048
Epoch 998/1000
143/143 [==============================] - ETA: 0s - loss: 0.001 - 0s 111us/step - loss: 0.0015 - val_loss: 0.0039
Epoch 999/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0013 - val_loss: 0.0038
Epoch 1000/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0013 - val_loss: 0.0041
0.010149426758289337


[array([[-1.5104556 , -1.3649668 ,  0.16585827,  0.20282786,  0.49746516],
        [ 0.6764522 , -0.14558627,  0.7936166 ,  0.7196441 ,  0.71913815],
        [ 0.27733368,  0.4032202 ,  0.81099224, -0.45604977, -0.27115762],
        [-0.08518778, -1.9339663 ,  1.5164855 ,  0.17174101,  0.4490696 ],
        [ 0.29391977,  0.31291568,  0.06588762,  0.24900812,  0.259849  ],
        [ 0.44609088, -0.3561649 , -1.3681657 ,  0.23819059,  0.82344115],
        [-0.07799527, -0.03031418,  0.6238308 , -0.29837093, -0.22517957],
        [ 0.2755493 , -0.17813592,  0.10340314, -0.01708427,  0.0929043 ],
        [ 0.21225977,  0.09164099, -0.40890488,  0.5774647 ,  0.4049441 ],
        [-0.7614256 , -0.8268645 ,  0.1616915 , -0.45591894, -0.7940281 ],
        [ 0.04979641,  1.4386493 , -0.5712316 ,  0.87113386, -0.21665843],
        [-0.34429568, -0.19074517, -0.07778696, -0.6864819 , -1.0254422 ],
        [-0.3641495 ,  0.25448275, -0.2415153 , -0.12336337, -0.789139  ],
        [-0.30248472,  0.

In [55]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_structure3_3rd.h5')

In [56]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression_4(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_46 (Dense)             (None, 10)                230       
_________________________________________________________________
activation_31 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_47 (Dense)             (None, 5)                 55        
_________________________________________________________________
activation_32 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_48 (Dense)             (None, 1)                 6         
Total params: 291
Trainable params: 291
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 2ms/step - loss: 

143/143 [==============================] - 0s 113us/step - loss: 0.0065 - val_loss: 0.0046
Epoch 70/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0064 - val_loss: 0.0042
Epoch 71/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0065 - val_loss: 0.0045
Epoch 72/1000
143/143 [==============================] - 0s 113us/step - loss: 0.0062 - val_loss: 0.0043
Epoch 73/1000
143/143 [==============================] - 0s 123us/step - loss: 0.0067 - val_loss: 0.0045
Epoch 74/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0063 - val_loss: 0.0043
Epoch 75/1000
143/143 [==============================] - 0s 106us/step - loss: 0.0062 - val_loss: 0.0047
Epoch 76/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0064 - val_loss: 0.0042
Epoch 77/1000
143/143 [==============================] - 0s 120us/step - loss: 0.0060 - val_loss: 0.0051
Epoch 78/1000
143/143 [==============================] - 0s 118us/ste

143/143 [==============================] - 0s 114us/step - loss: 0.0042 - val_loss: 0.0038
Epoch 147/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0044 - val_loss: 0.0039
Epoch 148/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0036 - val_loss: 0.0039
Epoch 149/1000
143/143 [==============================] - 0s 132us/step - loss: 0.0036 - val_loss: 0.0038
Epoch 150/1000
143/143 [==============================] - 0s 117us/step - loss: 0.0035 - val_loss: 0.0036
Epoch 151/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0034 - val_loss: 0.0044
Epoch 152/1000
143/143 [==============================] - 0s 124us/step - loss: 0.0035 - val_loss: 0.0036
Epoch 153/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0039 - val_loss: 0.0038
Epoch 154/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0036 - val_loss: 0.0040
Epoch 155/1000
143/143 [==============================] - 0s 

Epoch 223/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0041 - val_loss: 0.0042
Epoch 224/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0036 - val_loss: 0.0052
Epoch 225/1000
143/143 [==============================] - 0s 146us/step - loss: 0.0032 - val_loss: 0.0036
Epoch 226/1000
143/143 [==============================] - 0s 132us/step - loss: 0.0031 - val_loss: 0.0038
Epoch 227/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0032 - val_loss: 0.0054
Epoch 228/1000
143/143 [==============================] - 0s 128us/step - loss: 0.0032 - val_loss: 0.0037
Epoch 229/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0034 - val_loss: 0.0038
Epoch 230/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0033 - val_loss: 0.0054
Epoch 231/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0028 - val_loss: 0.0042
Epoch 232/1000
143/143 [======================

143/143 [==============================] - 0s 119us/step - loss: 0.0026 - val_loss: 0.0052
Epoch 301/1000
143/143 [==============================] - 0s 129us/step - loss: 0.0026 - val_loss: 0.0047
Epoch 302/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0025 - val_loss: 0.0040
Epoch 303/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0028 - val_loss: 0.0043
Epoch 304/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0024 - val_loss: 0.0041
Epoch 305/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0024 - val_loss: 0.0049
Epoch 306/1000
143/143 [==============================] - 0s 123us/step - loss: 0.0025 - val_loss: 0.0042
Epoch 307/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0024 - val_loss: 0.0044
Epoch 308/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0023 - val_loss: 0.0039
Epoch 309/1000
143/143 [==============================] - 0s 

...

143/143 [==============================] - 0s 139us/step - loss: 0.0012 - val_loss: 0.0054
Epoch 763/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0013 - val_loss: 0.0063
Epoch 764/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0012 - val_loss: 0.0060
Epoch 765/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0012 - val_loss: 0.0066
Epoch 766/1000
143/143 [==============================] - 0s 128us/step - loss: 0.0020 - val_loss: 0.0068
Epoch 767/1000
143/143 [==============================] - 0s 132us/step - loss: 0.0017 - val_loss: 0.0067
Epoch 768/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0017 - val_loss: 0.0054
Epoch 769/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0015 - val_loss: 0.0062
Epoch 770/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0017 - val_loss: 0.0068
Epoch 771/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0015 - val_loss: 0.0066
Epoch 840/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0013 - val_loss: 0.0067
Epoch 841/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0016 - val_loss: 0.0059
Epoch 842/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0011 - val_loss: 0.0068
Epoch 843/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0012 - val_loss: 0.0065
Epoch 844/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0011 - val_loss: 0.0064
Epoch 845/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0011 - val_loss: 0.0064
Epoch 846/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0011 - val_loss: 0.0064
Epoch 847/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0011 - val_loss: 0.0064
Epoch 848/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 133us/step - loss: 0.0011 - val_loss: 0.0067
Epoch 917/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0012 - val_loss: 0.0067
Epoch 918/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0013 - val_loss: 0.0068
Epoch 919/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0014 - val_loss: 0.0071
Epoch 920/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0010 - val_loss: 0.0069
Epoch 921/1000
143/143 [==============================] - 0s 119us/step - loss: 9.8522e-04 - val_loss: 0.0066
Epoch 922/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0010 - val_loss: 0.0070
Epoch 923/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0010 - val_loss: 0.0072
Epoch 924/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0012 - val_loss: 0.0069
Epoch 925/1000
143/143 [==============================] -

Epoch 993/1000
143/143 [==============================] - 0s 110us/step - loss: 0.0010 - val_loss: 0.0070
Epoch 994/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0010 - val_loss: 0.0072
Epoch 995/1000
143/143 [==============================] - 0s 119us/step - loss: 9.8413e-04 - val_loss: 0.0070
Epoch 996/1000
143/143 [==============================] - 0s 112us/step - loss: 9.3137e-04 - val_loss: 0.0074
Epoch 997/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0011 - val_loss: 0.0069
Epoch 998/1000
143/143 [==============================] - 0s 111us/step - loss: 9.8988e-04 - val_loss: 0.0071
Epoch 999/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0010 - val_loss: 0.0074
Epoch 1000/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0013 - val_loss: 0.0077
0.01167001947760582


[array([[ 0.66158277,  0.4425873 , -1.1086903 ,  0.10679191, -0.10305981,
          0.18635064, -0.03261758, -0.42856145,  0.293184  , -0.52332664],
        [-0.546337  , -0.13943301,  0.20290516, -1.3790962 ,  0.37715313,
          0.58803797, -0.0724607 , -0.09307076, -0.16117214,  0.25323528],
        [ 0.5214689 , -0.10949008,  0.8612542 , -0.19324389,  0.21184772,
         -0.20438097, -0.5215213 ,  0.15538877, -0.27269676, -0.30890936],
        [ 0.60288984,  0.24934107,  1.4024686 ,  0.92994565,  0.08463224,
         -0.23654194, -0.6247185 , -0.00417121, -0.1580552 , -1.6997945 ],
        [ 0.32997617, -0.22422571, -0.4207746 , -0.11020105, -0.21649693,
         -0.01926761, -0.6244146 ,  0.24130294, -0.21946259,  0.45718363],
        [-1.3655093 ,  0.6933503 , -0.13095807, -0.2212351 ,  0.66821015,
          0.24088503, -0.04546517, -0.4768003 ,  0.00571786,  0.5434827 ],
        [-0.1426752 ,  0.09581514,  0.5837343 ,  0.10552273,  0.6665544 ,
          0.15189697, -0.6956318

In [57]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_structure4_3rd.h5')

In [58]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression_5(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_49 (Dense)             (None, 10)                230       
_________________________________________________________________
activation_33 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_50 (Dense)             (None, 10)                110       
_________________________________________________________________
activation_34 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_51 (Dense)             (None, 1)                 11        
Total params: 351
Trainable params: 351
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 3ms/step - loss: 

143/143 [==============================] - 0s 113us/step - loss: 0.0064 - val_loss: 0.0044
Epoch 70/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0063 - val_loss: 0.0038
Epoch 71/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0066 - val_loss: 0.0039
Epoch 72/1000
143/143 [==============================] - 0s 117us/step - loss: 0.0062 - val_loss: 0.0045
Epoch 73/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0071 - val_loss: 0.0048
Epoch 74/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0074 - val_loss: 0.0047
Epoch 75/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0071 - val_loss: 0.0039
Epoch 76/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0078 - val_loss: 0.0052
Epoch 77/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0071 - val_loss: 0.0040
Epoch 78/1000
143/143 [==============================] - 0s 111us/ste

143/143 [==============================] - 0s 126us/step - loss: 0.0047 - val_loss: 0.0038
Epoch 147/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0048 - val_loss: 0.0032
Epoch 148/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0046 - val_loss: 0.0039
Epoch 149/1000
143/143 [==============================] - 0s 132us/step - loss: 0.0044 - val_loss: 0.0042
Epoch 150/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0046 - val_loss: 0.0045
Epoch 151/1000
143/143 [==============================] - 0s 125us/step - loss: 0.0046 - val_loss: 0.0033
Epoch 152/1000
143/143 [==============================] - 0s 146us/step - loss: 0.0046 - val_loss: 0.0032
Epoch 153/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0045 - val_loss: 0.0038
Epoch 154/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0046 - val_loss: 0.0040
Epoch 155/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 114us/step - loss: 0.0034 - val_loss: 0.0033
Epoch 224/1000
143/143 [==============================] - 0s 120us/step - loss: 0.0037 - val_loss: 0.0034
Epoch 225/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0032 - val_loss: 0.0032
Epoch 226/1000
143/143 [==============================] - 0s 120us/step - loss: 0.0036 - val_loss: 0.0030
Epoch 227/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0035 - val_loss: 0.0049
Epoch 228/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0035 - val_loss: 0.0032
Epoch 229/1000
143/143 [==============================] - 0s 117us/step - loss: 0.0035 - val_loss: 0.0039
Epoch 230/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0035 - val_loss: 0.0036
Epoch 231/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0034 - val_loss: 0.0030
Epoch 232/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0028 - val_loss: 0.0032
Epoch 301/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0028 - val_loss: 0.0026
Epoch 302/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0031 - val_loss: 0.0029
Epoch 303/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0027 - val_loss: 0.0027
Epoch 304/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0028 - val_loss: 0.0027
Epoch 305/1000
143/143 [==============================] - 0s 120us/step - loss: 0.0030 - val_loss: 0.0033
Epoch 306/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0028 - val_loss: 0.0026
Epoch 307/1000
143/143 [==============================] - 0s 121us/step - loss: 0.0029 - val_loss: 0.0039
Epoch 308/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0031 - val_loss: 0.0028
Epoch 309/1000
143/143 [==============================] - 0s 

...

143/143 [==============================] - 0s 132us/step - loss: 0.0014 - val_loss: 0.0024
Epoch 763/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0014 - val_loss: 0.0031
Epoch 764/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0014 - val_loss: 0.0021
Epoch 765/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0015 - val_loss: 0.0022
Epoch 766/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0015 - val_loss: 0.0026
Epoch 767/1000
143/143 [==============================] - 0s 125us/step - loss: 0.0014 - val_loss: 0.0028
Epoch 768/1000
143/143 [==============================] - 0s 125us/step - loss: 0.0013 - val_loss: 0.0025
Epoch 769/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0012 - val_loss: 0.0022
Epoch 770/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0013 - val_loss: 0.0025
Epoch 771/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 126us/step - loss: 0.0012 - val_loss: 0.0023
Epoch 840/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0011 - val_loss: 0.0025
Epoch 841/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0014 - val_loss: 0.0025
Epoch 842/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0012 - val_loss: 0.0022
Epoch 843/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0014 - val_loss: 0.0021
Epoch 844/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0012 - val_loss: 0.0020
Epoch 845/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0011 - val_loss: 0.0026
Epoch 846/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0011 - val_loss: 0.0027
Epoch 847/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0012 - val_loss: 0.0020
Epoch 848/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 133us/step - loss: 0.0013 - val_loss: 0.0022
Epoch 917/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0014 - val_loss: 0.0034
Epoch 918/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0016 - val_loss: 0.0022
Epoch 919/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0013 - val_loss: 0.0022
Epoch 920/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0013 - val_loss: 0.0020
Epoch 921/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0011 - val_loss: 0.0025
Epoch 922/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0010 - val_loss: 0.0021
Epoch 923/1000
143/143 [==============================] - 0s 132us/step - loss: 0.0012 - val_loss: 0.0022
Epoch 924/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0011 - val_loss: 0.0028
Epoch 925/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0013 - val_loss: 0.0025
Epoch 994/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0014 - val_loss: 0.0023
Epoch 995/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0014 - val_loss: 0.0023
Epoch 996/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0013 - val_loss: 0.0023
Epoch 997/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0011 - val_loss: 0.0035
Epoch 998/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0014 - val_loss: 0.0023
Epoch 999/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0011 - val_loss: 0.0021
Epoch 1000/1000
143/143 [==============================] - 0s 146us/step - loss: 0.0013 - val_loss: 0.0022
0.007543719373643398


[array([[ 0.8610001 , -0.69259506, -0.04879162,  0.49990648,  0.12087695,
         -0.3130062 , -0.2902251 ,  0.13304886,  0.10281271,  0.34344184],
        [-1.0500197 , -0.75020295,  0.0891478 , -0.1620969 ,  0.28636977,
         -0.03386784,  0.29471952,  0.44975516,  0.40577102,  0.40337408],
        [-0.8181066 , -0.3465711 , -0.04623673, -0.3937001 , -0.4681498 ,
          0.20688802, -0.1088808 , -0.15312329, -0.11984669,  0.1848564 ],
        [-0.8765797 , -0.7483909 ,  1.0353211 ,  0.13621739,  0.3792853 ,
          0.23240094, -0.36602482,  0.5326473 ,  1.639029  , -0.4720744 ],
        [-0.34530672, -0.33807015, -0.14380863,  0.12248819, -0.31446236,
          0.2176863 , -0.29968452,  0.6476523 , -0.3387815 ,  0.35390815],
        [ 0.96023905, -0.45538405, -0.3042542 , -0.13684462, -0.3438831 ,
         -0.7934102 , -0.14345106,  0.40933284, -0.07168973, -0.19313207],
        [-0.16871041, -0.18558829,  0.61611503,  0.17277129,  0.01124904,
         -0.2866169 ,  0.5564643

In [59]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_structure5_3rd.h5')

In [60]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression_6(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_52 (Dense)             (None, 10)                230       
_________________________________________________________________
activation_35 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_53 (Dense)             (None, 15)                165       
_________________________________________________________________
activation_36 (Activation)   (None, 15)                0         
_________________________________________________________________
dense_54 (Dense)             (None, 1)                 16        
Total params: 411
Trainable params: 411
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 3ms/step - loss: 

143/143 [==============================] - 0s 139us/step - loss: 0.0082 - val_loss: 0.0080
Epoch 70/1000
143/143 [==============================] - 0s 125us/step - loss: 0.0077 - val_loss: 0.0085
Epoch 71/1000
143/143 [==============================] - 0s 133us/step - loss: 0.0080 - val_loss: 0.0069
Epoch 72/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0076 - val_loss: 0.0062
Epoch 73/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0069 - val_loss: 0.0062
Epoch 74/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0070 - val_loss: 0.0069
Epoch 75/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0073 - val_loss: 0.0065
Epoch 76/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0069 - val_loss: 0.0071
Epoch 77/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0079 - val_loss: 0.0096
Epoch 78/1000
143/143 [==============================] - 0s 119us/ste

143/143 [==============================] - 0s 119us/step - loss: 0.0062 - val_loss: 0.0109
Epoch 147/1000
143/143 [==============================] - 0s 116us/step - loss: 0.0070 - val_loss: 0.0080
Epoch 148/1000
143/143 [==============================] - 0s 113us/step - loss: 0.0073 - val_loss: 0.0094
Epoch 149/1000
143/143 [==============================] - 0s 115us/step - loss: 0.0104 - val_loss: 0.0106
Epoch 150/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0081 - val_loss: 0.0071
Epoch 151/1000
143/143 [==============================] - 0s 127us/step - loss: 0.0072 - val_loss: 0.0077
Epoch 152/1000
143/143 [==============================] - 0s 117us/step - loss: 0.0056 - val_loss: 0.0068
Epoch 153/1000
143/143 [==============================] - 0s 117us/step - loss: 0.0057 - val_loss: 0.0069
Epoch 154/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0052 - val_loss: 0.0072
Epoch 155/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 110us/step - loss: 0.0049 - val_loss: 0.0065
Epoch 224/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0054 - val_loss: 0.0075
Epoch 225/1000
143/143 [==============================] - 0s 129us/step - loss: 0.0045 - val_loss: 0.0073
Epoch 226/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0050 - val_loss: 0.0067
Epoch 227/1000
143/143 [==============================] - 0s 139us/step - loss: 0.0044 - val_loss: 0.0064
Epoch 228/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0044 - val_loss: 0.0067
Epoch 229/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0045 - val_loss: 0.0070
Epoch 230/1000
143/143 [==============================] - 0s 115us/step - loss: 0.0052 - val_loss: 0.0063
Epoch 231/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0044 - val_loss: 0.0063
Epoch 232/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 139us/step - loss: 0.0039 - val_loss: 0.0061
Epoch 301/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0039 - val_loss: 0.0060
Epoch 302/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0040 - val_loss: 0.0063
Epoch 303/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0038 - val_loss: 0.0062
Epoch 304/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0039 - val_loss: 0.0061
Epoch 305/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0039 - val_loss: 0.0062
Epoch 306/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0044 - val_loss: 0.0077
Epoch 307/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0040 - val_loss: 0.0061
Epoch 308/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0043 - val_loss: 0.0073
Epoch 309/1000
143/143 [==============================] - 0s 

...

143/143 [==============================] - 0s 111us/step - loss: 0.0018 - val_loss: 0.0063
Epoch 763/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0018 - val_loss: 0.0059
Epoch 764/1000
143/143 [==============================] - 0s 108us/step - loss: 0.0018 - val_loss: 0.0091
Epoch 765/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0020 - val_loss: 0.0067
Epoch 766/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0020 - val_loss: 0.0066
Epoch 767/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0018 - val_loss: 0.0080
Epoch 768/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0019 - val_loss: 0.0065
Epoch 769/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0017 - val_loss: 0.0069
Epoch 770/1000
143/143 [==============================] - 0s 117us/step - loss: 0.0016 - val_loss: 0.0072
Epoch 771/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 115us/step - loss: 0.0017 - val_loss: 0.0072
Epoch 840/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0015 - val_loss: 0.0066
Epoch 841/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0014 - val_loss: 0.0072
Epoch 842/1000
143/143 [==============================] - 0s 108us/step - loss: 0.0015 - val_loss: 0.0071
Epoch 843/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0015 - val_loss: 0.0088
Epoch 844/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0018 - val_loss: 0.0063
Epoch 845/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0014 - val_loss: 0.0069
Epoch 846/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0014 - val_loss: 0.0072
Epoch 847/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0015 - val_loss: 0.0067
Epoch 848/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 110us/step - loss: 0.0013 - val_loss: 0.0070
Epoch 917/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0014 - val_loss: 0.0066
Epoch 918/1000
143/143 [==============================] - 0s 110us/step - loss: 0.0019 - val_loss: 0.0068
Epoch 919/1000
143/143 [==============================] - 0s 122us/step - loss: 0.0017 - val_loss: 0.0077
Epoch 920/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0015 - val_loss: 0.0085
Epoch 921/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0020 - val_loss: 0.0071
Epoch 922/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0016 - val_loss: 0.0067
Epoch 923/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0020 - val_loss: 0.0061
Epoch 924/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0015 - val_loss: 0.0072
Epoch 925/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 105us/step - loss: 0.0013 - val_loss: 0.0073
Epoch 994/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0014 - val_loss: 0.0067
Epoch 995/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0012 - val_loss: 0.0085
Epoch 996/1000
143/143 [==============================] - 0s 108us/step - loss: 0.0012 - val_loss: 0.0083
Epoch 997/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0011 - val_loss: 0.0072
Epoch 998/1000
143/143 [==============================] - 0s 108us/step - loss: 0.0016 - val_loss: 0.0073
Epoch 999/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0013 - val_loss: 0.0087
Epoch 1000/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0015 - val_loss: 0.0074
0.005262305494397879


[array([[ 6.52661920e-01, -3.57384346e-02,  1.63505271e-01,
         -8.23379338e-01, -9.21211317e-02,  6.90834224e-01,
         -3.70481938e-01,  7.61913419e-01,  5.49882174e-01,
         -2.37085834e-01],
        [-2.14445651e-01, -5.85439622e-01,  5.29419601e-01,
          2.80065089e-01,  5.94026625e-01,  4.15281832e-01,
         -3.56058687e-01, -9.22980487e-01, -2.32882574e-01,
          3.86679381e-01],
        [-1.48893637e-03, -2.51165461e-02,  4.74239774e-02,
          2.45677143e-01,  4.59122807e-01, -3.36362958e-01,
         -3.29041988e-01,  1.38766274e-01,  2.73531586e-01,
         -2.65145749e-01],
        [ 5.02472520e-01, -1.52574599e-01,  4.05685455e-01,
          1.57966897e-01,  6.63309574e-01,  9.51064050e-01,
         -7.93316543e-01, -7.98670292e-01,  6.82790101e-01,
         -3.70312572e-01],
        [ 2.41264567e-01,  4.01149035e-01, -7.93255642e-02,
         -4.23042148e-01, -1.02782790e-02,  3.50716472e-01,
          6.28901303e-01, -6.63010299e-01,  5.067836

In [61]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_structure6_3rd.h5')

In [62]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_1(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_55 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_37 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_56 (Dense)             (None, 5)                 30        
_________________________________________________________________
activation_38 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_57 (Dense)             (None, 1)                 6         
Total params: 56
Trainable params: 56
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 12ms/step - loss: 35.64

Epoch 71/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0194 - val_loss: 0.0561
Epoch 72/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0193 - val_loss: 0.0536
Epoch 73/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0191 - val_loss: 0.0505
Epoch 74/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0188 - val_loss: 0.0469
Epoch 75/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0184 - val_loss: 0.0434
Epoch 76/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0178 - val_loss: 0.0399
Epoch 77/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0172 - val_loss: 0.0366
Epoch 78/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0165 - val_loss: 0.0336
Epoch 79/1000
32/32 [==============================] - 0s 126us/step - loss: 0.0158 - val_loss: 0.0309
Epoch 80/1000
32/32 [==============================] - 0s 124us/step - lo

Epoch 151/1000
32/32 [==============================] - 0s 139us/step - loss: 0.0049 - val_loss: 0.0086
Epoch 152/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0048 - val_loss: 0.0085
Epoch 153/1000
32/32 [==============================] - 0s 186us/step - loss: 0.0047 - val_loss: 0.0084
Epoch 154/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0047 - val_loss: 0.0082
Epoch 155/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0046 - val_loss: 0.0081
Epoch 156/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0046 - val_loss: 0.0080
Epoch 157/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0045 - val_loss: 0.0079
Epoch 158/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0045 - val_loss: 0.0078
Epoch 159/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0044 - val_loss: 0.0077
Epoch 160/1000
32/32 [==============================] - 0s 124us

Epoch 230/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0023 - val_loss: 0.0033
Epoch 231/1000
32/32 [==============================] - 0s 217us/step - loss: 0.0023 - val_loss: 0.0032
Epoch 232/1000
32/32 [==============================] - 0s 218us/step - loss: 0.0023 - val_loss: 0.0032
Epoch 233/1000
32/32 [==============================] - 0s 218us/step - loss: 0.0023 - val_loss: 0.0032
Epoch 234/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0023 - val_loss: 0.0031
Epoch 235/1000
32/32 [==============================] - 0s 159us/step - loss: 0.0023 - val_loss: 0.0031
Epoch 236/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0022 - val_loss: 0.0031
Epoch 237/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0022 - val_loss: 0.0030
Epoch 238/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0022 - val_loss: 0.0030
Epoch 239/1000
32/32 [==============================] - 0s 155us

Epoch 309/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0015 - val_loss: 0.0015
Epoch 310/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0015 - val_loss: 0.0014
Epoch 311/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0015 - val_loss: 0.0014
Epoch 312/1000
32/32 [==============================] - 0s 153us/step - loss: 0.0015 - val_loss: 0.0014
Epoch 313/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0015 - val_loss: 0.0014
Epoch 314/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0015 - val_loss: 0.0014
Epoch 315/1000
32/32 [==============================] - 0s 153us/step - loss: 0.0014 - val_loss: 0.0014
Epoch 316/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0014 - val_loss: 0.0014
Epoch 317/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0014 - val_loss: 0.0013
Epoch 318/1000
32/32 [==============================] - 0s 143us

...

Epoch 757/1000
32/32 [==============================] - 0s 156us/step - loss: 4.0127e-04 - val_loss: 2.1628e-05
Epoch 758/1000
32/32 [==============================] - 0s 156us/step - loss: 4.0067e-04 - val_loss: 2.1677e-05
Epoch 759/1000
32/32 [==============================] - 0s 156us/step - loss: 4.0006e-04 - val_loss: 2.1729e-05
Epoch 760/1000
32/32 [==============================] - 0s 156us/step - loss: 3.9947e-04 - val_loss: 2.1782e-05
Epoch 761/1000
32/32 [==============================] - 0s 124us/step - loss: 3.9886e-04 - val_loss: 2.1831e-05
Epoch 762/1000
32/32 [==============================] - 0s 156us/step - loss: 3.9827e-04 - val_loss: 2.1882e-05
Epoch 763/1000
32/32 [==============================] - 0s 129us/step - loss: 3.9767e-04 - val_loss: 2.1932e-05
Epoch 764/1000
32/32 [==============================] - 0s 156us/step - loss: 3.9708e-04 - val_loss: 2.1980e-05
Epoch 765/1000
32/32 [==============================] - 0s 156us/step - loss: 3.9649e-04 - val_loss: 2.2

Epoch 831/1000
32/32 [==============================] - 0s 125us/step - loss: 3.6032e-04 - val_loss: 2.4274e-05
Epoch 832/1000
32/32 [==============================] - 0s 125us/step - loss: 3.5980e-04 - val_loss: 2.4296e-05
Epoch 833/1000
32/32 [==============================] - 0s 156us/step - loss: 3.5930e-04 - val_loss: 2.4318e-05
Epoch 834/1000
32/32 [==============================] - 0s 156us/step - loss: 3.5879e-04 - val_loss: 2.4339e-05
Epoch 835/1000
32/32 [==============================] - 0s 125us/step - loss: 3.5829e-04 - val_loss: 2.4360e-05
Epoch 836/1000
32/32 [==============================] - 0s 156us/step - loss: 3.5779e-04 - val_loss: 2.4380e-05
Epoch 837/1000
32/32 [==============================] - 0s 148us/step - loss: 3.5728e-04 - val_loss: 2.4397e-05
Epoch 838/1000
32/32 [==============================] - 0s 156us/step - loss: 3.5678e-04 - val_loss: 2.4414e-05
Epoch 839/1000
32/32 [==============================] - 0s 156us/step - loss: 3.5628e-04 - val_loss: 2.4

Epoch 905/1000
32/32 [==============================] - 0s 156us/step - loss: 3.2547e-04 - val_loss: 2.5067e-05
Epoch 906/1000
32/32 [==============================] - 0s 156us/step - loss: 3.2503e-04 - val_loss: 2.5070e-05
Epoch 907/1000
32/32 [==============================] - 0s 156us/step - loss: 3.2459e-04 - val_loss: 2.5073e-05
Epoch 908/1000
32/32 [==============================] - 0s 156us/step - loss: 3.2415e-04 - val_loss: 2.5076e-05
Epoch 909/1000
32/32 [==============================] - 0s 156us/step - loss: 3.2372e-04 - val_loss: 2.5076e-05
Epoch 910/1000
32/32 [==============================] - 0s 187us/step - loss: 3.2329e-04 - val_loss: 2.5076e-05
Epoch 911/1000
32/32 [==============================] - 0s 156us/step - loss: 3.2285e-04 - val_loss: 2.5078e-05
Epoch 912/1000
32/32 [==============================] - 0s 125us/step - loss: 3.2242e-04 - val_loss: 2.5081e-05
Epoch 913/1000
32/32 [==============================] - 0s 187us/step - loss: 3.2199e-04 - val_loss: 2.5

Epoch 979/1000
32/32 [==============================] - 0s 156us/step - loss: 2.9526e-04 - val_loss: 2.4892e-05
Epoch 980/1000
32/32 [==============================] - 0s 156us/step - loss: 2.9487e-04 - val_loss: 2.4886e-05
Epoch 981/1000
32/32 [==============================] - 0s 187us/step - loss: 2.9450e-04 - val_loss: 2.4878e-05
Epoch 982/1000
32/32 [==============================] - 0s 156us/step - loss: 2.9411e-04 - val_loss: 2.4873e-05
Epoch 983/1000
32/32 [==============================] - 0s 156us/step - loss: 2.9373e-04 - val_loss: 2.4869e-05
Epoch 984/1000
32/32 [==============================] - 0s 187us/step - loss: 2.9336e-04 - val_loss: 2.4862e-05
Epoch 985/1000
32/32 [==============================] - 0s 156us/step - loss: 2.9298e-04 - val_loss: 2.4857e-05
Epoch 986/1000
32/32 [==============================] - 0s 156us/step - loss: 2.9260e-04 - val_loss: 2.4851e-05
Epoch 987/1000
32/32 [==============================] - 0s 156us/step - loss: 2.9222e-04 - val_loss: 2.4

[array([[ 0.4972712 ,  1.1983103 ,  0.55198175,  0.49735886, -0.9871544 ],
        [-0.516381  , -0.7493354 ,  0.09029235, -0.53057796,  0.60803044],
        [-0.7899559 ,  1.4216826 ,  1.5982524 ,  1.0730485 , -1.0867842 ]],
       dtype=float32),
 array([ 0.8497863 , -0.7726063 ,  0.61374   , -0.9131135 ,  0.43090382],
       dtype=float32),
 array([[-3.48383784e-01,  8.10980618e-01, -1.10070489e-01,
          4.41076368e-01, -1.31641611e-01],
        [-9.51579452e-01,  3.21457475e-01,  2.55592197e-01,
         -2.05613095e-02,  7.18704017e-04],
        [-5.17058253e-01,  9.68577713e-02, -1.64514184e-01,
          3.34117442e-01,  3.56699467e-01],
        [ 5.80054581e-01,  1.13027126e-01, -4.32037890e-01,
         -5.12755573e-01,  4.35394682e-02],
        [-1.07791328e+00,  6.31578267e-02,  2.09732100e-01,
          1.06566995e-01,  6.93198502e-01]], dtype=float32),
 array([-0.90925735,  0.87682956, -0.8281077 ,  0.87066793,  0.9061792 ],
       dtype=float32),
 array([[-1.0660119]

In [63]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_structure1_3rd.h5')

In [64]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_2(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_58 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_39 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_59 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_40 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_60 (Dense)             (None, 1)                 11        
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 14ms/step - loss: 36.55

Epoch 71/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0219 - val_loss: 0.0387
Epoch 72/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0211 - val_loss: 0.0371
Epoch 73/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0201 - val_loss: 0.0351
Epoch 74/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0190 - val_loss: 0.0331
Epoch 75/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0180 - val_loss: 0.0309
Epoch 76/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0170 - val_loss: 0.0288
Epoch 77/1000
32/32 [==============================] - 0s 129us/step - loss: 0.0161 - val_loss: 0.0268
Epoch 78/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0154 - val_loss: 0.0250
Epoch 79/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0149 - val_loss: 0.0234
Epoch 80/1000
32/32 [==============================] - 0s 137us/step - lo

32/32 [==============================] - 0s 136us/step - loss: 0.0055 - val_loss: 0.0084
Epoch 151/1000
32/32 [==============================] - 0s 138us/step - loss: 0.0054 - val_loss: 0.0083
Epoch 152/1000
32/32 [==============================] - 0s 186us/step - loss: 0.0054 - val_loss: 0.0082
Epoch 153/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0053 - val_loss: 0.0081
Epoch 154/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0053 - val_loss: 0.0081
Epoch 155/1000
32/32 [==============================] - 0s 186us/step - loss: 0.0052 - val_loss: 0.0080
Epoch 156/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0052 - val_loss: 0.0079
Epoch 157/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0051 - val_loss: 0.0078
Epoch 158/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0051 - val_loss: 0.0078
Epoch 159/1000
32/32 [==============================] - 0s 125us/step - loss: 0

Epoch 229/1000
32/32 [==============================] - 0s 218us/step - loss: 0.0029 - val_loss: 0.0049
Epoch 230/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0029 - val_loss: 0.0048
Epoch 231/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0029 - val_loss: 0.0048
Epoch 232/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0029 - val_loss: 0.0048
Epoch 233/1000
32/32 [==============================] - 0s 186us/step - loss: 0.0029 - val_loss: 0.0048
Epoch 234/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0028 - val_loss: 0.0047
Epoch 235/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0028 - val_loss: 0.0047
Epoch 236/1000
32/32 [==============================] - 0s 218us/step - loss: 0.0028 - val_loss: 0.0047
Epoch 237/1000
32/32 [==============================] - 0s 218us/step - loss: 0.0028 - val_loss: 0.0046
Epoch 238/1000
32/32 [==============================] - 0s 249us

Epoch 308/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0018 - val_loss: 0.0032
Epoch 309/1000
32/32 [==============================] - 0s 218us/step - loss: 0.0018 - val_loss: 0.0032
Epoch 310/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0018 - val_loss: 0.0032
Epoch 311/1000
32/32 [==============================] - 0s 218us/step - loss: 0.0018 - val_loss: 0.0032
Epoch 312/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0018 - val_loss: 0.0032
Epoch 313/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0018 - val_loss: 0.0032
Epoch 314/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0018 - val_loss: 0.0032
Epoch 315/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0018 - val_loss: 0.0031
Epoch 316/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0018 - val_loss: 0.0031
Epoch 317/1000
32/32 [==============================] - 0s 156us

...

Epoch 766/1000
32/32 [==============================] - 0s 156us/step - loss: 3.5192e-04 - val_loss: 7.3442e-04
Epoch 767/1000
32/32 [==============================] - 0s 125us/step - loss: 3.5102e-04 - val_loss: 7.3269e-04
Epoch 768/1000
32/32 [==============================] - 0s 156us/step - loss: 3.5012e-04 - val_loss: 7.3095e-04
Epoch 769/1000
32/32 [==============================] - 0s 156us/step - loss: 3.4922e-04 - val_loss: 7.2921e-04
Epoch 770/1000
32/32 [==============================] - 0s 156us/step - loss: 3.4832e-04 - val_loss: 7.2748e-04
Epoch 771/1000
32/32 [==============================] - 0s 156us/step - loss: 3.4743e-04 - val_loss: 7.2575e-04
Epoch 772/1000
32/32 [==============================] - 0s 156us/step - loss: 3.4653e-04 - val_loss: 7.2404e-04
Epoch 773/1000
32/32 [==============================] - 0s 187us/step - loss: 3.4564e-04 - val_loss: 7.2233e-04
Epoch 774/1000
32/32 [==============================] - 0s 187us/step - loss: 3.4476e-04 - val_loss: 7.2

Epoch 840/1000
32/32 [==============================] - 0s 155us/step - loss: 2.9268e-04 - val_loss: 6.1933e-04
Epoch 841/1000
32/32 [==============================] - 0s 125us/step - loss: 2.9198e-04 - val_loss: 6.1796e-04
Epoch 842/1000
32/32 [==============================] - 0s 156us/step - loss: 2.9128e-04 - val_loss: 6.1659e-04
Epoch 843/1000
32/32 [==============================] - 0s 155us/step - loss: 2.9058e-04 - val_loss: 6.1522e-04
Epoch 844/1000
32/32 [==============================] - 0s 187us/step - loss: 2.8989e-04 - val_loss: 6.1383e-04
Epoch 845/1000
32/32 [==============================] - 0s 156us/step - loss: 2.8919e-04 - val_loss: 6.1248e-04
Epoch 846/1000
32/32 [==============================] - 0s 125us/step - loss: 2.8850e-04 - val_loss: 6.1111e-04
Epoch 847/1000
32/32 [==============================] - 0s 156us/step - loss: 2.8781e-04 - val_loss: 6.0977e-04
Epoch 848/1000
32/32 [==============================] - 0s 187us/step - loss: 2.8712e-04 - val_loss: 6.0

Epoch 914/1000
32/32 [==============================] - 0s 125us/step - loss: 2.4630e-04 - val_loss: 5.2762e-04
Epoch 915/1000
32/32 [==============================] - 0s 187us/step - loss: 2.4574e-04 - val_loss: 5.2650e-04
Epoch 916/1000
32/32 [==============================] - 0s 156us/step - loss: 2.4518e-04 - val_loss: 5.2539e-04
Epoch 917/1000
32/32 [==============================] - 0s 125us/step - loss: 2.4463e-04 - val_loss: 5.2429e-04
Epoch 918/1000
32/32 [==============================] - 0s 156us/step - loss: 2.4408e-04 - val_loss: 5.2318e-04
Epoch 919/1000
32/32 [==============================] - 0s 125us/step - loss: 2.4353e-04 - val_loss: 5.2209e-04
Epoch 920/1000
32/32 [==============================] - 0s 218us/step - loss: 2.4298e-04 - val_loss: 5.2101e-04
Epoch 921/1000
32/32 [==============================] - 0s 156us/step - loss: 2.4244e-04 - val_loss: 5.1990e-04
Epoch 922/1000
32/32 [==============================] - 0s 156us/step - loss: 2.4189e-04 - val_loss: 5.1

Epoch 988/1000
32/32 [==============================] - 0s 156us/step - loss: 2.0926e-04 - val_loss: 4.5321e-04
Epoch 989/1000
32/32 [==============================] - 0s 156us/step - loss: 2.0881e-04 - val_loss: 4.5230e-04
Epoch 990/1000
32/32 [==============================] - 0s 155us/step - loss: 2.0837e-04 - val_loss: 4.5140e-04
Epoch 991/1000
32/32 [==============================] - 0s 125us/step - loss: 2.0792e-04 - val_loss: 4.5049e-04
Epoch 992/1000
32/32 [==============================] - 0s 156us/step - loss: 2.0748e-04 - val_loss: 4.4960e-04
Epoch 993/1000
32/32 [==============================] - 0s 187us/step - loss: 2.0703e-04 - val_loss: 4.4869e-04
Epoch 994/1000
32/32 [==============================] - 0s 125us/step - loss: 2.0659e-04 - val_loss: 4.4781e-04
Epoch 995/1000
32/32 [==============================] - 0s 126us/step - loss: 2.0615e-04 - val_loss: 4.4689e-04
Epoch 996/1000
32/32 [==============================] - 0s 156us/step - loss: 2.0571e-04 - val_loss: 4.4

[array([[ 0.03889734,  0.5113609 , -0.6847888 ,  0.5183343 , -0.75870466],
        [-0.8815038 ,  0.291253  , -0.1051743 ,  1.1593301 , -0.62455904],
        [ 0.81181574,  0.9476168 ,  0.36905318,  0.24579082, -0.64575255]],
       dtype=float32),
 array([-0.6093154 ,  0.35221583, -0.6423176 , -0.48661128,  0.42522877],
       dtype=float32),
 array([[-0.18379413,  0.43037397, -0.4279939 , -0.06893022, -0.3152057 ,
          0.30165404, -0.36938602, -0.00873825, -0.4966687 , -0.26341173],
        [-0.43110305, -0.49818295, -0.5748151 , -0.38305098, -0.20579858,
         -0.6117527 ,  0.21773547,  0.21822439,  0.38463446, -0.5446202 ],
        [-0.13812755, -0.34262168,  0.3010797 , -0.2698826 , -0.4322621 ,
         -0.29632366, -0.79574126,  0.6050327 , -0.27839202, -0.06004401],
        [ 0.35037553,  0.49517697,  0.20357603, -0.50358856,  0.28758958,
          0.35600626,  0.03560254,  0.09661929,  0.634248  ,  0.0106471 ],
        [-0.0188462 ,  0.45207652, -0.32104805, -0.0418817

In [65]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_structure2_3rd.h5')

In [66]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_3(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_61 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_41 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_62 (Dense)             (None, 15)                90        
_________________________________________________________________
activation_42 (Activation)   (None, 15)                0         
_________________________________________________________________
dense_63 (Dense)             (None, 1)                 16        
Total params: 126
Trainable params: 126
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 13ms/step - loss: 36.

Epoch 71/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0196 - val_loss: 0.0273
Epoch 72/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0191 - val_loss: 0.0279
Epoch 73/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0177 - val_loss: 0.0283
Epoch 74/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0159 - val_loss: 0.0286
Epoch 75/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0143 - val_loss: 0.0290
Epoch 76/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0134 - val_loss: 0.0295
Epoch 77/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0134 - val_loss: 0.0299
Epoch 78/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0139 - val_loss: 0.0300
Epoch 79/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0145 - val_loss: 0.0296
Epoch 80/1000
32/32 [==============================] - 0s 156us/step - lo

Epoch 151/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0069 - val_loss: 0.0088
Epoch 152/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0069 - val_loss: 0.0087
Epoch 153/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0068 - val_loss: 0.0087
Epoch 154/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0068 - val_loss: 0.0086
Epoch 155/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0068 - val_loss: 0.0085
Epoch 156/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0067 - val_loss: 0.0084
Epoch 157/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0067 - val_loss: 0.0084
Epoch 158/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0067 - val_loss: 0.0083
Epoch 159/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0066 - val_loss: 0.0082
Epoch 160/1000
32/32 [==============================] - 0s 155us

Epoch 230/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0047 - val_loss: 0.0050
Epoch 231/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0047 - val_loss: 0.0050
Epoch 232/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0047 - val_loss: 0.0050
Epoch 233/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0046 - val_loss: 0.0049
Epoch 234/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0046 - val_loss: 0.0049
Epoch 235/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0046 - val_loss: 0.0049
Epoch 236/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0046 - val_loss: 0.0049
Epoch 237/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0046 - val_loss: 0.0048
Epoch 238/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0045 - val_loss: 0.0048
Epoch 239/1000
32/32 [==============================] - 0s 157us

Epoch 309/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0034 - val_loss: 0.0034
Epoch 310/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0034 - val_loss: 0.0033
Epoch 311/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0034 - val_loss: 0.0033
Epoch 312/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0034 - val_loss: 0.0033
Epoch 313/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0033 - val_loss: 0.0033
Epoch 314/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0033 - val_loss: 0.0033
Epoch 315/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0033 - val_loss: 0.0033
Epoch 316/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0033 - val_loss: 0.0033
Epoch 317/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0033 - val_loss: 0.0032
Epoch 318/1000
32/32 [==============================] - 0s 156us

...

Epoch 777/1000
32/32 [==============================] - 0s 155us/step - loss: 8.1549e-04 - val_loss: 7.8044e-04
Epoch 778/1000
32/32 [==============================] - 0s 156us/step - loss: 8.1345e-04 - val_loss: 7.7871e-04
Epoch 779/1000
32/32 [==============================] - 0s 155us/step - loss: 8.1141e-04 - val_loss: 7.7697e-04
Epoch 780/1000
32/32 [==============================] - 0s 186us/step - loss: 8.0939e-04 - val_loss: 7.7524e-04
Epoch 781/1000
32/32 [==============================] - 0s 156us/step - loss: 8.0736e-04 - val_loss: 7.7353e-04
Epoch 782/1000
32/32 [==============================] - 0s 155us/step - loss: 8.0536e-04 - val_loss: 7.7182e-04
Epoch 783/1000
32/32 [==============================] - 0s 125us/step - loss: 8.0336e-04 - val_loss: 7.7010e-04
Epoch 784/1000
32/32 [==============================] - 0s 124us/step - loss: 8.0135e-04 - val_loss: 7.6838e-04
Epoch 785/1000
32/32 [==============================] - 0s 186us/step - loss: 7.9935e-04 - val_loss: 7.6

Epoch 851/1000
32/32 [==============================] - 0s 156us/step - loss: 6.8059e-04 - val_loss: 6.6580e-04
Epoch 852/1000
32/32 [==============================] - 0s 187us/step - loss: 6.7897e-04 - val_loss: 6.6439e-04
Epoch 853/1000
32/32 [==============================] - 0s 156us/step - loss: 6.7736e-04 - val_loss: 6.6304e-04
Epoch 854/1000
32/32 [==============================] - 0s 156us/step - loss: 6.7574e-04 - val_loss: 6.6167e-04
Epoch 855/1000
32/32 [==============================] - 0s 156us/step - loss: 6.7414e-04 - val_loss: 6.6031e-04
Epoch 856/1000
32/32 [==============================] - 0s 156us/step - loss: 6.7253e-04 - val_loss: 6.5895e-04
Epoch 857/1000
32/32 [==============================] - 0s 156us/step - loss: 6.7093e-04 - val_loss: 6.5758e-04
Epoch 858/1000
32/32 [==============================] - 0s 156us/step - loss: 6.6934e-04 - val_loss: 6.5622e-04
Epoch 859/1000
32/32 [==============================] - 0s 187us/step - loss: 6.6776e-04 - val_loss: 6.5

Epoch 925/1000
32/32 [==============================] - 0s 156us/step - loss: 5.7275e-04 - val_loss: 5.7393e-04
Epoch 926/1000
32/32 [==============================] - 0s 156us/step - loss: 5.7145e-04 - val_loss: 5.7284e-04
Epoch 927/1000
32/32 [==============================] - 0s 156us/step - loss: 5.7015e-04 - val_loss: 5.7170e-04
Epoch 928/1000
32/32 [==============================] - 0s 187us/step - loss: 5.6884e-04 - val_loss: 5.7062e-04
Epoch 929/1000
32/32 [==============================] - 0s 187us/step - loss: 5.6756e-04 - val_loss: 5.6950e-04
Epoch 930/1000
32/32 [==============================] - 0s 156us/step - loss: 5.6627e-04 - val_loss: 5.6839e-04
Epoch 931/1000
32/32 [==============================] - 0s 156us/step - loss: 5.6498e-04 - val_loss: 5.6729e-04
Epoch 932/1000
32/32 [==============================] - 0s 156us/step - loss: 5.6370e-04 - val_loss: 5.6620e-04
Epoch 933/1000
32/32 [==============================] - 0s 187us/step - loss: 5.6243e-04 - val_loss: 5.6

Epoch 999/1000
32/32 [==============================] - 0s 157us/step - loss: 4.8552e-04 - val_loss: 4.9865e-04
Epoch 1000/1000
32/32 [==============================] - 0s 156us/step - loss: 4.8446e-04 - val_loss: 4.9773e-04
0.0002449593448545784


[array([[ 0.7244419 , -0.18053065, -0.52981913, -0.43697247,  0.53630316],
        [-0.94422174, -0.00260216,  0.33968464,  0.60096806, -0.44988677],
        [ 0.8580506 ,  1.584395  ,  0.8190745 ,  0.52542037,  0.65288067]],
       dtype=float32),
 array([-0.5093216 ,  0.37297165, -0.65194654, -0.36437142, -0.6290139 ],
       dtype=float32),
 array([[ 0.0578681 ,  0.2785509 , -0.4462231 , -0.40439898, -0.5536354 ,
         -0.22892714, -0.18569808,  0.1688244 , -0.15997669,  0.23754506,
         -0.08420013, -0.15355483, -0.37878224, -0.1059102 , -0.0822348 ],
        [-0.0954055 , -0.15297212, -0.1094649 ,  0.00628059, -0.06464136,
          0.5020361 ,  0.3382464 ,  0.22174034, -0.52526903, -0.5246862 ,
          0.21151787, -0.2588147 , -0.10743792,  0.12558886,  0.35835662],
        [ 0.05023582, -0.55573136,  0.03018558,  0.16093665, -0.01578087,
         -0.0438478 ,  0.3578779 , -0.42825553, -0.50969064,  0.1254739 ,
          0.331195  ,  0.20409352,  0.3858541 ,  0.28731725,

In [67]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_structure3_3rd.h5')

In [68]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_4(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_64 (Dense)             (None, 10)                40        
_________________________________________________________________
activation_43 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_65 (Dense)             (None, 5)                 55        
_________________________________________________________________
activation_44 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_66 (Dense)             (None, 1)                 6         
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 13ms/step - loss: 36.

Epoch 71/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0200 - val_loss: 0.0185
Epoch 72/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0213 - val_loss: 0.0190
Epoch 73/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0221 - val_loss: 0.0193
Epoch 74/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0221 - val_loss: 0.0191
Epoch 75/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0213 - val_loss: 0.0187
Epoch 76/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0199 - val_loss: 0.0185
Epoch 77/1000
32/32 [==============================] - 0s 174us/step - loss: 0.0184 - val_loss: 0.0186
Epoch 78/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0171 - val_loss: 0.0191
Epoch 79/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0162 - val_loss: 0.0200
Epoch 80/1000
32/32 [==============================] - 0s 187us/step - lo

Epoch 151/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0071 - val_loss: 0.0093
Epoch 152/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0070 - val_loss: 0.0092
Epoch 153/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0069 - val_loss: 0.0091
Epoch 154/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0069 - val_loss: 0.0091
Epoch 155/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0068 - val_loss: 0.0090
Epoch 156/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0068 - val_loss: 0.0089
Epoch 157/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0067 - val_loss: 0.0088
Epoch 158/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0067 - val_loss: 0.0088
Epoch 159/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0066 - val_loss: 0.0087
Epoch 160/1000
32/32 [==============================] - 0s 125us

Epoch 230/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0040 - val_loss: 0.0054
Epoch 231/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0039 - val_loss: 0.0053
Epoch 232/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0039 - val_loss: 0.0053
Epoch 233/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0039 - val_loss: 0.0053
Epoch 234/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0039 - val_loss: 0.0052
Epoch 235/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0038 - val_loss: 0.0052
Epoch 236/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0038 - val_loss: 0.0052
Epoch 237/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0038 - val_loss: 0.0051
Epoch 238/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0038 - val_loss: 0.0051
Epoch 239/1000
32/32 [==============================] - 0s 156us

Epoch 309/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0025 - val_loss: 0.0034
Epoch 310/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0025 - val_loss: 0.0034
Epoch 311/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0025 - val_loss: 0.0034
Epoch 312/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0024 - val_loss: 0.0034
Epoch 313/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0024 - val_loss: 0.0034
Epoch 314/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0024 - val_loss: 0.0034
Epoch 315/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0024 - val_loss: 0.0033
Epoch 316/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0024 - val_loss: 0.0033
Epoch 317/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0024 - val_loss: 0.0033
Epoch 318/1000
32/32 [==============================] - 0s 156us

...

Epoch 769/1000
32/32 [==============================] - 0s 156us/step - loss: 4.2338e-04 - val_loss: 6.1695e-04
Epoch 770/1000
32/32 [==============================] - 0s 156us/step - loss: 4.2235e-04 - val_loss: 6.1531e-04
Epoch 771/1000
32/32 [==============================] - 0s 157us/step - loss: 4.2132e-04 - val_loss: 6.1370e-04
Epoch 772/1000
32/32 [==============================] - 0s 156us/step - loss: 4.2028e-04 - val_loss: 6.1207e-04
Epoch 773/1000
32/32 [==============================] - 0s 156us/step - loss: 4.1926e-04 - val_loss: 6.1046e-04
Epoch 774/1000
32/32 [==============================] - 0s 126us/step - loss: 4.1824e-04 - val_loss: 6.0886e-04
Epoch 775/1000
32/32 [==============================] - 0s 156us/step - loss: 4.1723e-04 - val_loss: 6.0725e-04
Epoch 776/1000
32/32 [==============================] - 0s 156us/step - loss: 4.1622e-04 - val_loss: 6.0566e-04
Epoch 777/1000
32/32 [==============================] - 0s 125us/step - loss: 4.1521e-04 - val_loss: 6.0

Epoch 843/1000
32/32 [==============================] - 0s 156us/step - loss: 3.5737e-04 - val_loss: 5.1181e-04
Epoch 844/1000
32/32 [==============================] - 0s 125us/step - loss: 3.5661e-04 - val_loss: 5.1058e-04
Epoch 845/1000
32/32 [==============================] - 0s 187us/step - loss: 3.5586e-04 - val_loss: 5.0937e-04
Epoch 846/1000
32/32 [==============================] - 0s 125us/step - loss: 3.5510e-04 - val_loss: 5.0814e-04
Epoch 847/1000
32/32 [==============================] - 0s 125us/step - loss: 3.5436e-04 - val_loss: 5.0692e-04
Epoch 848/1000
32/32 [==============================] - 0s 156us/step - loss: 3.5360e-04 - val_loss: 5.0571e-04
Epoch 849/1000
32/32 [==============================] - 0s 156us/step - loss: 3.5286e-04 - val_loss: 5.0448e-04
Epoch 850/1000
32/32 [==============================] - 0s 125us/step - loss: 3.5211e-04 - val_loss: 5.0328e-04
Epoch 851/1000
32/32 [==============================] - 0s 156us/step - loss: 3.5138e-04 - val_loss: 5.0

Epoch 917/1000
32/32 [==============================] - 0s 125us/step - loss: 3.0835e-04 - val_loss: 4.3178e-04
Epoch 918/1000
32/32 [==============================] - 0s 187us/step - loss: 3.0778e-04 - val_loss: 4.3084e-04
Epoch 919/1000
32/32 [==============================] - 0s 218us/step - loss: 3.0721e-04 - val_loss: 4.2988e-04
Epoch 920/1000
32/32 [==============================] - 0s 187us/step - loss: 3.0664e-04 - val_loss: 4.2895e-04
Epoch 921/1000
32/32 [==============================] - 0s 187us/step - loss: 3.0607e-04 - val_loss: 4.2802e-04
Epoch 922/1000
32/32 [==============================] - 0s 187us/step - loss: 3.0551e-04 - val_loss: 4.2708e-04
Epoch 923/1000
32/32 [==============================] - 0s 187us/step - loss: 3.0494e-04 - val_loss: 4.2616e-04
Epoch 924/1000
32/32 [==============================] - 0s 156us/step - loss: 3.0438e-04 - val_loss: 4.2523e-04
Epoch 925/1000
32/32 [==============================] - 0s 187us/step - loss: 3.0382e-04 - val_loss: 4.2

Epoch 991/1000
32/32 [==============================] - 0s 156us/step - loss: 2.7093e-04 - val_loss: 3.6965e-04
Epoch 992/1000
32/32 [==============================] - 0s 156us/step - loss: 2.7048e-04 - val_loss: 3.6890e-04
Epoch 993/1000
32/32 [==============================] - 0s 156us/step - loss: 2.7005e-04 - val_loss: 3.6815e-04
Epoch 994/1000
32/32 [==============================] - 0s 125us/step - loss: 2.6960e-04 - val_loss: 3.6744e-04
Epoch 995/1000
32/32 [==============================] - 0s 156us/step - loss: 2.6916e-04 - val_loss: 3.6669e-04
Epoch 996/1000
32/32 [==============================] - 0s 156us/step - loss: 2.6872e-04 - val_loss: 3.6597e-04
Epoch 997/1000
32/32 [==============================] - 0s 155us/step - loss: 2.6828e-04 - val_loss: 3.6522e-04
Epoch 998/1000
32/32 [==============================] - 0s 155us/step - loss: 2.6785e-04 - val_loss: 3.6451e-04
Epoch 999/1000
32/32 [==============================] - 0s 156us/step - loss: 2.6742e-04 - val_loss: 3.6

[array([[-0.23901601, -0.7780296 ,  0.06455855, -0.74722993, -0.6184778 ,
          0.60119754,  0.7595726 , -0.06485737, -0.33488598,  1.0843974 ],
        [-0.18823282,  0.24822621,  0.2831763 , -0.12071633, -0.50832045,
         -0.6487418 , -0.46411273,  1.3642317 ,  0.8754057 , -0.72211814],
        [-0.13148028, -1.0566496 ,  1.8635312 , -0.8834052 , -1.3848641 ,
         -0.92668873, -0.7310501 , -0.5974123 , -0.25896022,  0.17282324]],
       dtype=float32),
 array([ 0.54873466,  0.4324164 ,  0.47355974, -0.33128324, -0.7161277 ,
         0.5859835 ,  0.420777  , -0.23802078,  0.49295434, -0.6019282 ],
       dtype=float32),
 array([[-0.42225942,  0.26033983,  0.38283774, -0.51377064,  0.31146583],
        [-0.72460186,  0.57049286, -0.25379714, -0.8270029 ,  0.01559149],
        [ 0.23676133, -0.025828  ,  0.1314389 , -0.5800296 ,  0.20281239],
        [ 0.2880134 , -0.07955879,  0.1326491 ,  0.13191925,  0.44558495],
        [-0.11466061, -0.88041776, -0.86336607,  0.7871846 

In [69]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_structure4_3rd.h5')

In [70]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_5(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_67 (Dense)             (None, 10)                40        
_________________________________________________________________
activation_45 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_68 (Dense)             (None, 10)                110       
_________________________________________________________________
activation_46 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_69 (Dense)             (None, 1)                 11        
Total params: 161
Trainable params: 161
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 14ms/step - loss: 36.

Epoch 71/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0133 - val_loss: 0.0082
Epoch 72/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0147 - val_loss: 0.0099
Epoch 73/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0159 - val_loss: 0.0106
Epoch 74/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0162 - val_loss: 0.0102
Epoch 75/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0153 - val_loss: 0.0089
Epoch 76/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0135 - val_loss: 0.0071
Epoch 77/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0115 - val_loss: 0.0054
Epoch 78/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0098 - val_loss: 0.0041
Epoch 79/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0088 - val_loss: 0.0034
Epoch 80/1000
32/32 [==============================] - 0s 156us/step - lo

Epoch 151/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0026 - val_loss: 0.0032
Epoch 152/1000
32/32 [==============================] - 0s 155us/step - loss: 0.0026 - val_loss: 0.0032
Epoch 153/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0025 - val_loss: 0.0031
Epoch 154/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0025 - val_loss: 0.0031
Epoch 155/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0025 - val_loss: 0.0031
Epoch 156/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0025 - val_loss: 0.0031
Epoch 157/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0025 - val_loss: 0.0030
Epoch 158/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0025 - val_loss: 0.0030
Epoch 159/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0025 - val_loss: 0.0030
Epoch 160/1000
32/32 [==============================] - 0s 155us

Epoch 230/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0018 - val_loss: 0.0020
Epoch 231/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0018 - val_loss: 0.0020
Epoch 232/1000
32/32 [==============================] - 0s 186us/step - loss: 0.0018 - val_loss: 0.0020
Epoch 233/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0018 - val_loss: 0.0020
Epoch 234/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0018 - val_loss: 0.0020
Epoch 235/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0018 - val_loss: 0.0020
Epoch 236/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0018 - val_loss: 0.0020
Epoch 237/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0018 - val_loss: 0.0020
Epoch 238/1000
32/32 [==============================] - 0s 166us/step - loss: 0.0017 - val_loss: 0.0020
Epoch 239/1000
32/32 [==============================] - 0s 125us

Epoch 309/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0014 - val_loss: 0.0014
Epoch 310/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0014 - val_loss: 0.0014
Epoch 311/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0014 - val_loss: 0.0014
Epoch 312/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0014 - val_loss: 0.0014
Epoch 313/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0014 - val_loss: 0.0014
Epoch 314/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0014 - val_loss: 0.0013
Epoch 315/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0014 - val_loss: 0.0013
Epoch 316/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0013 - val_loss: 0.0013
Epoch 317/1000
32/32 [==============================] - 0s 126us/step - loss: 0.0013 - val_loss: 0.0013
Epoch 318/1000
32/32 [==============================] - 0s 155us

...

Epoch 759/1000
32/32 [==============================] - 0s 156us/step - loss: 4.7413e-04 - val_loss: 3.8193e-04
Epoch 760/1000
32/32 [==============================] - 0s 156us/step - loss: 4.7322e-04 - val_loss: 3.8130e-04
Epoch 761/1000
32/32 [==============================] - 0s 125us/step - loss: 4.7233e-04 - val_loss: 3.8067e-04
Epoch 762/1000
32/32 [==============================] - 0s 125us/step - loss: 4.7144e-04 - val_loss: 3.8004e-04
Epoch 763/1000
32/32 [==============================] - 0s 125us/step - loss: 4.7054e-04 - val_loss: 3.7941e-04
Epoch 764/1000
32/32 [==============================] - 0s 156us/step - loss: 4.6965e-04 - val_loss: 3.7876e-04
Epoch 765/1000
32/32 [==============================] - 0s 156us/step - loss: 4.6876e-04 - val_loss: 3.7814e-04
Epoch 766/1000
32/32 [==============================] - 0s 125us/step - loss: 4.6787e-04 - val_loss: 3.7753e-04
Epoch 767/1000
32/32 [==============================] - 0s 125us/step - loss: 4.6698e-04 - val_loss: 3.7

Epoch 833/1000
32/32 [==============================] - 0s 125us/step - loss: 4.1363e-04 - val_loss: 3.4010e-04
Epoch 834/1000
32/32 [==============================] - 0s 125us/step - loss: 4.1289e-04 - val_loss: 3.3960e-04
Epoch 835/1000
32/32 [==============================] - 0s 156us/step - loss: 4.1214e-04 - val_loss: 3.3910e-04
Epoch 836/1000
32/32 [==============================] - 0s 125us/step - loss: 4.1141e-04 - val_loss: 3.3861e-04
Epoch 837/1000
32/32 [==============================] - 0s 156us/step - loss: 4.1068e-04 - val_loss: 3.3812e-04
Epoch 838/1000
32/32 [==============================] - 0s 156us/step - loss: 4.0995e-04 - val_loss: 3.3762e-04
Epoch 839/1000
32/32 [==============================] - 0s 156us/step - loss: 4.0921e-04 - val_loss: 3.3712e-04
Epoch 840/1000
32/32 [==============================] - 0s 187us/step - loss: 4.0850e-04 - val_loss: 3.3664e-04
Epoch 841/1000
32/32 [==============================] - 0s 156us/step - loss: 4.0777e-04 - val_loss: 3.3

Epoch 907/1000
32/32 [==============================] - 0s 125us/step - loss: 3.6385e-04 - val_loss: 3.0675e-04
Epoch 908/1000
32/32 [==============================] - 0s 156us/step - loss: 3.6324e-04 - val_loss: 3.0635e-04
Epoch 909/1000
32/32 [==============================] - 0s 125us/step - loss: 3.6262e-04 - val_loss: 3.0593e-04
Epoch 910/1000
32/32 [==============================] - 0s 125us/step - loss: 3.6202e-04 - val_loss: 3.0552e-04
Epoch 911/1000
32/32 [==============================] - 0s 155us/step - loss: 3.6140e-04 - val_loss: 3.0513e-04
Epoch 912/1000
32/32 [==============================] - 0s 156us/step - loss: 3.6081e-04 - val_loss: 3.0472e-04
Epoch 913/1000
32/32 [==============================] - 0s 156us/step - loss: 3.6020e-04 - val_loss: 3.0432e-04
Epoch 914/1000
32/32 [==============================] - 0s 156us/step - loss: 3.5960e-04 - val_loss: 3.0392e-04
Epoch 915/1000
32/32 [==============================] - 0s 125us/step - loss: 3.5900e-04 - val_loss: 3.0

Epoch 981/1000
32/32 [==============================] - 0s 187us/step - loss: 3.2249e-04 - val_loss: 2.7930e-04
Epoch 982/1000
32/32 [==============================] - 0s 156us/step - loss: 3.2198e-04 - val_loss: 2.7896e-04
Epoch 983/1000
32/32 [==============================] - 0s 156us/step - loss: 3.2147e-04 - val_loss: 2.7862e-04
Epoch 984/1000
32/32 [==============================] - 0s 186us/step - loss: 3.2096e-04 - val_loss: 2.7829e-04
Epoch 985/1000
32/32 [==============================] - 0s 156us/step - loss: 3.2045e-04 - val_loss: 2.7796e-04
Epoch 986/1000
32/32 [==============================] - 0s 156us/step - loss: 3.1995e-04 - val_loss: 2.7763e-04
Epoch 987/1000
32/32 [==============================] - 0s 156us/step - loss: 3.1944e-04 - val_loss: 2.7729e-04
Epoch 988/1000
32/32 [==============================] - 0s 156us/step - loss: 3.1894e-04 - val_loss: 2.7696e-04
Epoch 989/1000
32/32 [==============================] - 0s 187us/step - loss: 3.1844e-04 - val_loss: 2.7

[array([[-0.596877  , -0.26509124, -0.32265404,  0.01187365, -0.31117406,
          0.62543154, -0.73705244,  0.3214254 , -0.23465015,  1.111962  ],
        [-0.0937803 ,  0.69218284,  0.5723254 ,  0.00946737,  0.4522415 ,
         -0.02764956, -0.20428444,  0.33471343,  0.81658024, -0.04199678],
        [ 0.12242565, -0.13139273, -0.69210124,  0.08854216,  0.69533813,
          1.4135716 , -0.00516706,  1.0838466 ,  0.4192919 ,  0.5363443 ]],
       dtype=float32),
 array([-0.44346634,  0.45568496,  0.44752863,  0.25776002, -0.51009536,
         0.42257187,  0.22863944,  0.4557526 , -0.30164236, -0.42065176],
       dtype=float32),
 array([[ 0.08705856,  0.46992967,  0.09966096,  0.5038854 , -0.22098967,
         -0.1879343 ,  0.4054801 , -0.5418108 ,  0.14028645, -0.02369673],
        [-0.32400087, -0.38921672, -0.37722284,  0.1679618 , -0.24201676,
         -0.4271226 , -0.33022496,  0.0986876 , -0.18372993,  0.14371309],
        [-0.07586323, -0.6229004 , -0.27781314,  0.56258744, 

In [71]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_structure5_3rd.h5')

In [72]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression_6(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_70 (Dense)             (None, 10)                40        
_________________________________________________________________
activation_47 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_71 (Dense)             (None, 15)                165       
_________________________________________________________________
activation_48 (Activation)   (None, 15)                0         
_________________________________________________________________
dense_72 (Dense)             (None, 1)                 16        
Total params: 221
Trainable params: 221
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 14ms/step - loss: 36.

Epoch 71/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0046 - val_loss: 0.0028
Epoch 72/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0056 - val_loss: 0.0038
Epoch 73/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0069 - val_loss: 0.0045
Epoch 74/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0079 - val_loss: 0.0046
Epoch 75/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0082 - val_loss: 0.0041
Epoch 76/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0077 - val_loss: 0.0031
Epoch 77/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0066 - val_loss: 0.0021
Epoch 78/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0054 - val_loss: 0.0012
Epoch 79/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0044 - val_loss: 6.8011e-04
Epoch 80/1000
32/32 [==============================] - 0s 156us/step 

Epoch 149/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0016 - val_loss: 7.1738e-04
Epoch 150/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0016 - val_loss: 7.2174e-04
Epoch 151/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0016 - val_loss: 7.2057e-04
Epoch 152/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0016 - val_loss: 7.1392e-04
Epoch 153/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0016 - val_loss: 7.0234e-04
Epoch 154/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0016 - val_loss: 6.8699e-04
Epoch 155/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0016 - val_loss: 6.6917e-04
Epoch 156/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0015 - val_loss: 6.5023e-04
Epoch 157/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0015 - val_loss: 6.3145e-04
Epoch 158/1000
32/32 [======

Epoch 225/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0011 - val_loss: 4.0738e-04
Epoch 226/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0011 - val_loss: 4.0510e-04
Epoch 227/1000
32/32 [==============================] - 0s 218us/step - loss: 0.0011 - val_loss: 4.0282e-04
Epoch 228/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0011 - val_loss: 4.0049e-04
Epoch 229/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0011 - val_loss: 3.9814e-04
Epoch 230/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0011 - val_loss: 3.9569e-04
Epoch 231/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0011 - val_loss: 3.9322e-04
Epoch 232/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0011 - val_loss: 3.9069e-04
Epoch 233/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0011 - val_loss: 3.8813e-04
Epoch 234/1000
32/32 [======

Epoch 300/1000
32/32 [==============================] - 0s 156us/step - loss: 8.2604e-04 - val_loss: 2.5943e-04
Epoch 301/1000
32/32 [==============================] - 0s 156us/step - loss: 8.2280e-04 - val_loss: 2.5798e-04
Epoch 302/1000
32/32 [==============================] - 0s 187us/step - loss: 8.1957e-04 - val_loss: 2.5654e-04
Epoch 303/1000
32/32 [==============================] - 0s 156us/step - loss: 8.1637e-04 - val_loss: 2.5511e-04
Epoch 304/1000
32/32 [==============================] - 0s 156us/step - loss: 8.1318e-04 - val_loss: 2.5369e-04
Epoch 305/1000
32/32 [==============================] - 0s 156us/step - loss: 8.0999e-04 - val_loss: 2.5228e-04
Epoch 306/1000
32/32 [==============================] - 0s 156us/step - loss: 8.0681e-04 - val_loss: 2.5089e-04
Epoch 307/1000
32/32 [==============================] - 0s 156us/step - loss: 8.0366e-04 - val_loss: 2.4951e-04
Epoch 308/1000
32/32 [==============================] - 0s 187us/step - loss: 8.0053e-04 - val_loss: 2.4

...

Epoch 744/1000
32/32 [==============================] - 0s 125us/step - loss: 1.8550e-04 - val_loss: 6.3289e-05
Epoch 745/1000
32/32 [==============================] - 0s 156us/step - loss: 1.8498e-04 - val_loss: 6.3170e-05
Epoch 746/1000
32/32 [==============================] - 0s 187us/step - loss: 1.8448e-04 - val_loss: 6.3047e-05
Epoch 747/1000
32/32 [==============================] - 0s 156us/step - loss: 1.8398e-04 - val_loss: 6.2921e-05
Epoch 748/1000
32/32 [==============================] - 0s 156us/step - loss: 1.8348e-04 - val_loss: 6.2795e-05
Epoch 749/1000
32/32 [==============================] - 0s 156us/step - loss: 1.8298e-04 - val_loss: 6.2675e-05
Epoch 750/1000
32/32 [==============================] - 0s 156us/step - loss: 1.8249e-04 - val_loss: 6.2557e-05
Epoch 751/1000
32/32 [==============================] - 0s 156us/step - loss: 1.8199e-04 - val_loss: 6.2443e-05
Epoch 752/1000
32/32 [==============================] - 0s 125us/step - loss: 1.8150e-04 - val_loss: 6.2

Epoch 818/1000
32/32 [==============================] - 0s 156us/step - loss: 1.5257e-04 - val_loss: 5.5006e-05
Epoch 819/1000
32/32 [==============================] - 0s 125us/step - loss: 1.5219e-04 - val_loss: 5.4897e-05
Epoch 820/1000
32/32 [==============================] - 0s 156us/step - loss: 1.5180e-04 - val_loss: 5.4803e-05
Epoch 821/1000
32/32 [==============================] - 0s 312us/step - loss: 1.5142e-04 - val_loss: 5.4699e-05
Epoch 822/1000
32/32 [==============================] - 0s 187us/step - loss: 1.5103e-04 - val_loss: 5.4596e-05
Epoch 823/1000
32/32 [==============================] - 0s 156us/step - loss: 1.5065e-04 - val_loss: 5.4495e-05
Epoch 824/1000
32/32 [==============================] - 0s 156us/step - loss: 1.5027e-04 - val_loss: 5.4397e-05
Epoch 825/1000
32/32 [==============================] - 0s 156us/step - loss: 1.4989e-04 - val_loss: 5.4292e-05
Epoch 826/1000
32/32 [==============================] - 0s 156us/step - loss: 1.4951e-04 - val_loss: 5.4

32/32 [==============================] - 0s 125us/step - loss: 1.2741e-04 - val_loss: 4.8115e-05
Epoch 892/1000
32/32 [==============================] - 0s 156us/step - loss: 1.2710e-04 - val_loss: 4.8028e-05
Epoch 893/1000
32/32 [==============================] - 0s 156us/step - loss: 1.2680e-04 - val_loss: 4.7941e-05
Epoch 894/1000
32/32 [==============================] - 0s 125us/step - loss: 1.2650e-04 - val_loss: 4.7855e-05
Epoch 895/1000
32/32 [==============================] - 0s 156us/step - loss: 1.2619e-04 - val_loss: 4.7771e-05
Epoch 896/1000
32/32 [==============================] - 0s 187us/step - loss: 1.2590e-04 - val_loss: 4.7684e-05
Epoch 897/1000
32/32 [==============================] - 0s 156us/step - loss: 1.2560e-04 - val_loss: 4.7599e-05
Epoch 898/1000
32/32 [==============================] - 0s 156us/step - loss: 1.2530e-04 - val_loss: 4.7510e-05
Epoch 899/1000
32/32 [==============================] - 0s 156us/step - loss: 1.2500e-04 - val_loss: 4.7429e-05
Epoch 9

Epoch 965/1000
32/32 [==============================] - 0s 156us/step - loss: 1.0732e-04 - val_loss: 4.2170e-05
Epoch 966/1000
32/32 [==============================] - 0s 156us/step - loss: 1.0708e-04 - val_loss: 4.2096e-05
Epoch 967/1000
32/32 [==============================] - 0s 156us/step - loss: 1.0684e-04 - val_loss: 4.2023e-05
Epoch 968/1000
32/32 [==============================] - 0s 156us/step - loss: 1.0661e-04 - val_loss: 4.1953e-05
Epoch 969/1000
32/32 [==============================] - 0s 156us/step - loss: 1.0636e-04 - val_loss: 4.1878e-05
Epoch 970/1000
32/32 [==============================] - 0s 156us/step - loss: 1.0613e-04 - val_loss: 4.1805e-05
Epoch 971/1000
32/32 [==============================] - 0s 187us/step - loss: 1.0589e-04 - val_loss: 4.1726e-05
Epoch 972/1000
32/32 [==============================] - 0s 156us/step - loss: 1.0566e-04 - val_loss: 4.1655e-05
Epoch 973/1000
32/32 [==============================] - 0s 187us/step - loss: 1.0542e-04 - val_loss: 4.1

[array([[-0.7115876 ,  0.28496602,  0.8704008 , -0.2489118 ,  0.8805716 ,
         -0.17566952,  0.57391965,  0.32014492, -0.92153245,  0.25783142],
        [-0.0235457 , -1.4158196 , -0.7305759 ,  0.5334021 ,  0.07287913,
         -0.93123466, -0.5096066 ,  0.7152222 ,  0.7974308 , -0.8070359 ],
        [-0.52847797,  0.21350288,  0.26874876,  0.6758865 ,  1.2268987 ,
         -0.57691526, -1.0479962 ,  0.712552  , -0.42414522,  0.4681817 ]],
       dtype=float32),
 array([ 0.15485458, -0.10824323,  0.06342434,  0.21577683,  0.37591812,
        -0.1565053 ,  0.5715749 , -0.17358758,  0.4250253 , -0.19761698],
       dtype=float32),
 array([[-0.24552092,  0.37862003, -0.17195041,  0.20588355,  0.08291757,
          0.21875459,  0.10731257,  0.07907016,  0.15094492,  0.31025666,
         -0.11159744, -0.1751987 ,  0.22116084, -0.27511403, -0.43708834],
        [-0.4189066 , -0.27132753,  0.09392748,  0.23210411, -0.37294394,
         -0.17324091, -0.30917874, -0.4935607 , -0.24403867, -

In [73]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_structure6_3rd.h5')